# 데이터 확장

기존에 전라 방언에 적용했던 처리를 확장하여 다른 4개 방언에도 적용

In [1]:
import pandas as pd
import numpy as np
import pathlib
import random
import re
import seaborn as sns

RANDOM_SEED = 19439
random.seed(RANDOM_SEED)

## 전라 방언 SentencePiece

In [27]:
df = pd.read_csv("../datas/output/jeonla_dialect_data_age_sampled.csv", index_col=0)
df

,방언,표준어,연령대,출처 파일,길이
0,막상 완전 돈이 급하게 필요한 것도 아니라서 그냥 용돈벌이 정도로만 할려고 생각을 ...,막상 완전 돈이 급하게 필요한 것도 아니라서 그냥 용돈벌이 정도로만 하려고 생각을 ...,20대,DJSX20002683.json,15
1,좋은 추억이 있어가지고 다시 키운다면 진돗개를 키우고 싶고,좋은 추억이 있어가지고 다시 키운다면 진돗개를 키우고 싶고,30대,DJDG20002125.json,8
2,쫌 보면 인제 세계 인제,조금 보면 인제 세계 인제,30대,DJSX20004304.json,5
3,가시나들 있으니까 뭐~ 일부러 뭐~ 쫌 져주는 척 또 한다고 그랬는진 모르겄는디,계집아이들 있으니까 뭐~ 일부러 뭐~ 조금 져주는 척 또 한다고 그랬는진 모르겠는데,20대,DJDG20001833.json,12
4,거기서 어 저 새끼 왜 미니언 안 먹는데 씨 때려야지 이러지는 않잖아,거기서 어 저 새끼 왜 미니언 안 먹는데 씨 때려야지 이러지는 않잖아,20대,DJDD20000034.json,12
...,...,...,...,...,...
508409,그~ 마음이나 생각이 조금 달라지나 봐,그~ 마음이나 생각이 조금 달라지나 봐,40대,DJSX20004708.json,6
508410,우리는 그 절에 가서 빌 수 있다는 것 내가 건강하기 때문에,우리는 그 절에 가서 빌 수 있다는 것 내가 건강하기 때문에,60대 이상,DJSX20004083.json,11
508411,방구석에 이케 이케 이불 뒤집고 이렇게 오열을 했었었는데,방구석에 이렇게 이렇게 이불 뒤집고 이렇게 오열을 했었었는데,20대,DJDG20001151.json,8
508412,뭐 없던 원래 없던 꿈도 아니라 이제 한창,뭐 없던 원래 없던 꿈도 아니라 이제 한창,20대,DJSX20000555.json,8


In [29]:
import sentencepiece as spm

spm.SentencePieceTrainer.train(input='../datas/output/jeonla_dialect_data_text_sampled.txt', model_prefix='jeonla_sp', vocab_size=30000)

In [30]:
sp_tokenizer = spm.SentencePieceProcessor(model_file='jeonla_sp.model')

code_result_dialect = sp_tokenizer.encode(df["방언"].to_list(), out_type=str)
code_result_standard = sp_tokenizer.encode(df["표준어"].to_list(), out_type=str)

In [31]:
df["방언"] = [' '.join(res) for res in code_result_dialect]
df["방언"]

0         ▁막상 ▁완전 ▁돈이 ▁급하게 ▁필요한 ▁것도 ▁아니라서 ▁그냥 ▁용돈벌이 ▁정도로...
1                ▁좋은 ▁추억이 ▁있어가지고 ▁다시 ▁키운다 면 ▁진돗개 를 ▁키우고 ▁싶고
2                                        ▁쫌 ▁보면 ▁인제 ▁세계 ▁인제
3         ▁가시나 들 ▁있으니까 ▁뭐 ~ ▁일부러 ▁뭐 ~ ▁쫌 ▁져 주는 ▁척 ▁또 ▁한다...
4         ▁거기서 ▁어 ▁저 ▁새끼 ▁왜 ▁미니언 ▁안 ▁먹는데 ▁씨 ▁때려 야지 ▁이러지 ...
                                ...                        
508409                       ▁그 ~ ▁마음이 나 ▁생각이 ▁조금 ▁달라지 나 ▁봐
508410        ▁우리는 ▁그 ▁절에 ▁가서 ▁빌 ▁수 ▁있다는 ▁것 ▁내가 ▁건강 하기 ▁때문에
508411          ▁방구석 에 ▁이케 ▁이케 ▁이불 ▁뒤집 고 ▁이렇게 ▁오 열 을 ▁했었었는데
508412                      ▁뭐 ▁없던 ▁원래 ▁없던 ▁꿈도 ▁아니라 ▁이제 ▁한창
508413             ▁너 ▁말대로 ▁알고리즘 ▁타고 ▁오니까 ▁쫌 ▁괴 아 는 게 ▁많아 .
Name: 방언, Length: 508414, dtype: object

In [32]:
df["표준어"] = [' '.join(res) for res in code_result_standard]
df["표준어"]

0         ▁막상 ▁완전 ▁돈이 ▁급하게 ▁필요한 ▁것도 ▁아니라서 ▁그냥 ▁용돈벌이 ▁정도로...
1                ▁좋은 ▁추억이 ▁있어가지고 ▁다시 ▁키운다 면 ▁진돗개 를 ▁키우고 ▁싶고
2                                       ▁조금 ▁보면 ▁인제 ▁세계 ▁인제
3         ▁계집아이 들 ▁있으니까 ▁뭐 ~ ▁일부러 ▁뭐 ~ ▁조금 ▁져 주는 ▁척 ▁또 ▁...
4         ▁거기서 ▁어 ▁저 ▁새끼 ▁왜 ▁미니언 ▁안 ▁먹는데 ▁씨 ▁때려 야지 ▁이러지 ...
                                ...                        
508409                       ▁그 ~ ▁마음이 나 ▁생각이 ▁조금 ▁달라지 나 ▁봐
508410        ▁우리는 ▁그 ▁절에 ▁가서 ▁빌 ▁수 ▁있다는 ▁것 ▁내가 ▁건강 하기 ▁때문에
508411        ▁방구석 에 ▁이렇게 ▁이렇게 ▁이불 ▁뒤집 고 ▁이렇게 ▁오 열 을 ▁했었었는데
508412                      ▁뭐 ▁없던 ▁원래 ▁없던 ▁꿈도 ▁아니라 ▁이제 ▁한창
508413              ▁너 ▁말대로 ▁알고리즘 ▁타고 ▁오니까 ▁조금 ▁괜찮은 게 ▁많아 .
Name: 표준어, Length: 508414, dtype: object

In [33]:
df.to_csv("../datas/output/jeonla_dialect_data_SentencePiece.csv")

### 테스트셋 SP

In [34]:
df = pd.read_csv("../datas/output/jeonla_dialect_test_age_processed.csv", index_col=0)
df

,방언,표준어,연령대,출처 파일,길이
0,나는 회사 학교의 이야기를 하고 싶어 형은 있잖아요 친구들 같은 거 다 동창 친구들...,나는 회사 학교의 이야기를 하고 싶어 형은 있잖아요 친구들 같은 거 다 동창 친구들...,20대,DJDD20000247.json,14
2,동창 친구들은 이제 초등학교 때부터 친구인 거예요?,동창 친구들은 이제 초등학교 때부터 친구인 거예요?,20대,DJDD20000247.json,7
3,음 친구들은 거의 이제 초등학교?,음 친구들은 거의 이제 초등학교?,20대,DJDD20000247.json,5
4,이렇게 친구인 애 있고 중학교에도 친구들 있고 한데 근데 형은 솔직히 말하면 그거 있잖아,이렇게 친구인 애 있고 중학교에도 친구들 있고 한데 근데 형은 솔직히 말하면 그거 있잖아,20대,DJDD20000247.json,14
5,친구들도 이제 촤승 아 초등학교 중학교 이런 것보다는 고등학교에서 진짜 친구 나뉜다...,친구들도 이제 촤승 아 초등학교 중학교 이런 것보다는 고등학교에서 진짜 친구 나뉜다...,20대,DJDD20000247.json,13
...,...,...,...,...,...
275123,어 택시가 있을 거 같애.,어 택시가 있을 거 같애.,30대,DJSX20003150.json,5
275125,우리나라도 완전 자율주행차를 이 천 이 십 칠 년에 최초로 상용화한다고 발표했대.,우리나라도 완전 자율주행차를 이 천 이 십 칠 년에 최초로 상용화한다고 발표했대.,30대,DJSX20003150.json,12
275126,대박이다 이제 칠 년 남았네 칠 년,대박이다 이제 칠 년 남았네 칠 년,30대,DJSX20003150.json,7
275133,이 천 이 십 칠 년이 이 천 이 십 칠 년에,이 천 이 십 칠 년이 이 천 이 십 칠 년에,30대,DJSX20003150.json,12


In [36]:
spm.SentencePieceTrainer.train(input='../datas/output/jeonla_dialect_test_text.txt', model_prefix='jeonla_test_sp', vocab_size=30000)

sp_tokenizer = spm.SentencePieceProcessor(model_file='jeonla_test_sp.model')

code_result_dialect = sp_tokenizer.encode(df["방언"].to_list(), out_type=str)
code_result_standard = sp_tokenizer.encode(df["표준어"].to_list(), out_type=str)

In [37]:
df["방언"] = [' '.join(res) for res in code_result_dialect]
df["방언"]

0         ▁나는 ▁회사 ▁학교 의 ▁이야기를 ▁하고 ▁싶어 ▁형은 ▁있잖아요 ▁친구들 ▁같은...
2                      ▁동창 ▁친구들은 ▁이제 ▁초등학교 ▁때부터 ▁친구인 ▁거예요 ?
3                                  ▁음 ▁친구들은 ▁거의 ▁이제 ▁초등학교 ?
4         ▁이렇게 ▁친구인 ▁애 ▁있고 ▁중학교 에도 ▁친구들 ▁있고 ▁한데 ▁근데 ▁형은 ...
5         ▁친구들도 ▁이제 ▁ 촤 승 ▁아 ▁초등학교 ▁중학교 ▁이런 ▁것보다는 ▁고등학교 ...
                                ...                        
275123                                ▁어 ▁택시 가 ▁있을 ▁거 ▁같애 .
275125    ▁우리나라도 ▁완전 ▁자율주행차 를 ▁이 ▁천 ▁이 ▁십 ▁칠 ▁년에 ▁최초 로 ▁...
275126                          ▁대박이다 ▁이제 ▁칠 ▁년 ▁남았 네 ▁칠 ▁년
275133                ▁이 ▁천 ▁이 ▁십 ▁칠 ▁년이 ▁이 ▁천 ▁이 ▁십 ▁칠 ▁년에
275134                            ▁근데 ▁발표 를 ▁한다고 ▁해서 ▁이게 ▁딱
Name: 방언, Length: 220917, dtype: object

In [38]:
df["표준어"] = [' '.join(res) for res in code_result_standard]
df["표준어"]

0         ▁나는 ▁회사 ▁학교 의 ▁이야기를 ▁하고 ▁싶어 ▁형은 ▁있잖아요 ▁친구들 ▁같은...
2                      ▁동창 ▁친구들은 ▁이제 ▁초등학교 ▁때부터 ▁친구인 ▁거예요 ?
3                                  ▁음 ▁친구들은 ▁거의 ▁이제 ▁초등학교 ?
4         ▁이렇게 ▁친구인 ▁애 ▁있고 ▁중학교 에도 ▁친구들 ▁있고 ▁한데 ▁근데 ▁형은 ...
5         ▁친구들도 ▁이제 ▁ 촤 승 ▁아 ▁초등학교 ▁중학교 ▁이런 ▁것보다는 ▁고등학교 ...
                                ...                        
275123                                ▁어 ▁택시 가 ▁있을 ▁거 ▁같애 .
275125    ▁우리나라도 ▁완전 ▁자율주행차 를 ▁이 ▁천 ▁이 ▁십 ▁칠 ▁년에 ▁최초 로 ▁...
275126                          ▁대박이다 ▁이제 ▁칠 ▁년 ▁남았 네 ▁칠 ▁년
275133                ▁이 ▁천 ▁이 ▁십 ▁칠 ▁년이 ▁이 ▁천 ▁이 ▁십 ▁칠 ▁년에
275134                            ▁근데 ▁발표 를 ▁한다고 ▁해서 ▁이게 ▁딱
Name: 표준어, Length: 220917, dtype: object

In [39]:
df.to_csv("../datas/output/jeonla_dialect_test_SentencePiece.csv")

## 강원 방언

In [23]:
df = pd.read_csv("../datas/output/gangwon_dialect_train_age.csv", index_col=0)
df

,방언,표준어,연령대,출처 파일
0,글쎄요,글쎄요,50대,DGDQ20000020.json
1,예?,예?,60대 이상,DGDQ20000020.json
2,뭔 얘기부터 할까유?,뭔 얘기부터 할까요?,60대 이상,DGDQ20000020.json
3,바다 얘기 좀 해주세요 {laughing},바다 얘기 좀 해주세요 {laughing},50대,DGDQ20000020.json
4,바다에요?,바다에요?,60대 이상,DGDQ20000020.json
...,...,...,...,...
1573232,식당에 삐집고 들어갔나?,식당에 비집고 들어갔나?,20대,DGTS20004276.json
1573233,음식점에 삐집고 들어갔나?,음식점에 비집고 들어갔나?,20대,DGTS20004276.json
1573234,쉰 음석이네요.,쉰 음식이네요.,20대,DGTS20004276.json
1573235,이 음석은 쉬었어요.,이 음식은 쉬었어요.,20대,DGTS20004276.json


In [2]:
# 테스트 데이터로 동일한 과정을 진행하고 싶을 때
df = pd.read_csv("../datas/output/gangwon_dialect_test_age.csv", index_col=0)
df

,방언,표준어,연령대,출처 파일
0,나는 사실 내 인생 자체가 증말 다이어트라고 해도 과언이 아닐 정도인데,나는 사실 내 인생 자체가 정말 다이어트라고 해도 과언이 아닐 정도인데,20대,DGIN21500168.json
1,일단 다이어트를 하면은 사람이 예민이 해지고 짜증을 많이 내잖아.,일단 다이어트를 하면은 사람이 예민이 해지고 짜증을 많이 내잖아.,20대,DGIN21500168.json
2,그런데 이제 나는 무조근 기분 나쁜 다이어트가 아니라 쪼끔 내 몸이 건강해지는,그런데 이제 나는 무조건 기분 나쁜 다이어트가 아니라 쪼끔 내 몸이 건강해지는,20대,DGIN21500168.json
3,기분 좋은 다이어트를 하고 싶은데,기분 좋은 다이어트를 하고 싶은데,20대,DGIN21500168.json
4,너도 다이어트를 많이 하잖아.,너도 다이어트를 많이 하잖아.,20대,DGIN21500168.json
...,...,...,...,...
276794,어렸을 때 삼겹살이라던가 거기 항상 냉동이긴 했는데 어~ 그거를 많이 먹었었던 것 같애.,어렸을 때 삼겹살이라던가 거기 항상 냉동이긴 했는데 어~ 그거를 많이 먹었었던 것 같아.,30대,DGIN21801133.json
276795,소괴기는 사실 어~ 나이가 들고 어~,소고기는 사실 어~ 나이가 들고 어~,30대,DGIN21801133.json
276796,먹었었던 것 같애 사실 어~ 젊었을 때나 어렸을 때라던가 이럴 때는 워낙 소괴기가 ...,먹었었던 것 같아 사실 어~ 젊었을 때나 어렸을 때라던가 이럴 때는 워낙 소고기가 ...,30대,DGIN21801133.json
276797,내가 일을 하면서 뭔가 쉽사리 접근할 수 있었던 이제 부분은 아니었어서,내가 일을 하면서 뭔가 쉽사리 접근할 수 있었던 이제 부분은 아니었어서,30대,DGIN21801133.json


### 데이터 전처리

In [24]:
df["방언"] = df["방언"].str.replace(r"\(\(\)\)", '', regex=True)
df["표준어"] = df["표준어"].str.replace(r"\(\(\)\)", '', regex=True)

df[df["방언"].str.contains("\(\(\)\)")|df["표준어"].str.contains("\(\(\)\)")]

,방언,표준어,연령대,출처 파일


In [25]:
phrase_words = df["방언"].str.extract(r"\(\((.*)\)\)", expand=False).dropna()
for i in phrase_words.index:
    sentence = df.loc[i, "방언"]
    sentence = re.sub(r"\(\(.*\)\)", phrase_words[i], sentence)
    df.loc[i, "방언"] = sentence

phrase_words = df["표준어"].str.extract(r"\(\((.*)\)\)", expand=False).dropna()
for i in phrase_words.index:
    sentence = df.loc[i, "표준어"]
    sentence = re.sub(r"\(\(.*\)\)", phrase_words[i], sentence)
    df.loc[i, "표준어"] = sentence

df.loc[phrase_words.index]

,방언,표준어,연령대,출처 파일
1192,까 여기까지가 이제 한줄로 요약하면,까 여기까지가 이제 한줄로 요약하면,20대,DGDQ20000032.json
3473,아 요즘에 가방 속 먼지를 청소해주는 더스트 부 뭐 막 이런 게 있대.,아 요즘에 가방 속 먼지를 청소해주는 더스트 부 뭐 막 이런 게 있대.,20대,DGDQ20000053.json
3580,그루일 야자 할 때마다 졸잖아,그루일 야자 할 때마다 졸잖아,20대,DGDQ20000061.json
3698,옵 다 배드민턴만 치더라?,옵 다 배드민턴만 치더라?,20대,DGDQ20000061.json
4643,제가 내년 선택 과목에 인제 탐구 과목 세 개를 고르는데,제가 내년 선택 과목에 인제 탐구 과목 세 개를 고르는데,10대,DGDQ20000067.json
...,...,...,...,...
1057576,다 고년 때민이지.,다 고년 때문이지.,50대,DGIN21920661.json
1057732,그 년이네 니 웬수.,그 년이네 네 웬수.,50대,DGIN21920662.json
1057745,저년 아니 쟤 때미네,저년 아니 쟤 때문에,50대,DGIN21920662.json
1058137,츠녀가 꼬 택시 안에서 잠이 들 정도로 술을 마시고 다녀?,처녀가 꼬 택시 안에서 잠이 들 정도로 술을 마시고 다녀?,50대,DGIN21920664.json


In [26]:
phrase_words = df["방언"].str.extract(r"\((.*)\)\/\(.*\)", expand=False).dropna()
for i in phrase_words.index:
    sentence = df.loc[i, "방언"]
    sentence = re.sub(r"\(.*\)\/\(.*\)", phrase_words[i], sentence)
    df.loc[i, "방언"] = sentence

phrase_words = df["표준어"].str.extract(r"\(.*\)\/\((.*)\)", expand=False).dropna()
for i in phrase_words.index:
    sentence = df.loc[i, "표준어"]
    sentence = re.sub(r"\(.*\)\/\(.*\)", phrase_words[i], sentence)
    df.loc[i, "표준어"] = sentence

df.loc[phrase_words.index]

,방언,표준어,연령대,출처 파일
9583,답장을 근데 뭐 이 학기 돼가주구 우리가 그냥 계속 연락하다 보니까,답장을 근데 뭐 이 학기 돼돼가지고 우리가 그냥 계속 연락하다 보니까,20대,DGDQ20000090.json
11452,엄마는 아이를 놀이기구에태꿨다.,엄마는 아이를 놀이기구에태웠다.,50대,DGDQ20000103.json
11517,알면서도 모른척하문서 시치미를 뗐다.,알면서도 모른척하면서 시치미를 뗐다.,50대,DGDQ20000103.json
11584,나는 간장에 절인 마늘장쩨이를 즐겨 먹는다.,나는 간장에 절인 마늘짱아찌를 즐겨 먹는다.,50대,DGDQ20000103.json
11586,이 집은 더덕구이와 감장쩨이로 유명하다.,이 집은 더덕구이와 감짱아찌로 유명하다.,50대,DGDQ20000103.json
...,...,...,...,...
1148029,먹고 살기가 힘들어 몇 시간이 되었어도 마수거리도 못해어.,먹고 살기가 힘들어 몇 시간이 되었어도 마수걸이 못했어.,60대 이상,DGTS20000149.json
1388300,아이 괜찮아요.거낭 구경하고 있아요.,아니 괜찮아요.그냥 구경하고 있어요.,40대,DGTS20000601.json
1388409,아이 멋있다고 생각했어 .하물미 치약을 밑에서부터 짜는 거 부터해서.,아니 멋있다고 생각했어 .하물며 치약을 밑에서부터 짜는 거 부터해서.,40대,DGTS20000601.json
1487513,감이 바로 내영감아이와.,감이 바로 내아니오.,40대,DGTS20000941.json


In [27]:
laugh_index = df["방언"].str.contains("\{laughing\}")

df["방언"] = df["방언"].str.replace(r"\{laughing\}", "", regex=True)
df["표준어"] = df["표준어"].str.replace(r"\{laughing\}", "", regex=True)
df[laugh_index]

,방언,표준어,연령대,출처 파일
3,바다 얘기 좀 해주세요,바다 얘기 좀 해주세요,50대,DGDQ20000020.json
51,그건 맞어 .,그건 맞어 .,50대,DGDQ20000020.json
115,김치도 진딱 해 가지고 여기저기 퍼주니,김치도 #진딱 해 가지고 여기저기 퍼주니,60대 이상,DGDQ20000020.json
183,강원도 사투리를 내가 많이 한대.,강원도 사투리를 내가 많이 한대.,60대 이상,DGDQ20000020.json
1630,기독교 아닌 사람들이 더 많아.,기독교 아닌 사람들이 더 많아.,30대,DGDQ20000034.json
...,...,...,...,...
1051062,.,.,40대,DGIN21920532.json
1051649,.,.,60대 이상,DGIN21920534.json
1051698,&name9&씨가 그렇게 말해주이시이까 열심히 조사해서 온 보람이 있는데요?,&name9&씨가 그렇게 말해주시니까 열심히 조사해서 온 보람이 있는데요?,60대 이상,DGIN21920534.json
1052437,,,60대 이상,DGIN21920634.json


In [28]:
stutter_sentence = df["방언"].str.contains(r"\-.*\-", regex=True)
df["방언"] = df["방언"].str.replace(r"\-.*\-", "", regex=True)
df["표준어"] = df["표준어"].str.replace(r"\-.*\-", "", regex=True)

df.loc[stutter_sentence]

,방언,표준어,연령대,출처 파일
17,근데 그 티비에 나오는데 보니까,근데 그 티비에 나오는데 보니까,60대 이상,DGDQ20000020.json
22,지금도 누가 울어도 하고 저 명파 그 사람 죽은 사람 그 누구야?,지금도 누가 울어도 하고 저 명파 그 사람 죽은 사람 그 누구야?,60대 이상,DGDQ20000020.json
26,어떠한 자신감도 있겠지만,어떠한 자신감도 있겠지만,60대 이상,DGDQ20000020.json
32,형님 말 빨라.,형님 말 빨라.,50대,DGDQ20000020.json
46,알았다고 고개 끄덕끄덕 대면서 그렇게 해서 내가 목소리가 큰 줄 알았다니까?,알았다고 고개 끄덕끄덕 대면서 그렇게 해서 내가 목소리가 큰 줄 알았다니까?,60대 이상,DGDQ20000020.json
...,...,...,...,...
1111025,국민들은 두 당 간의 전쟁에 아쭈 무관심한 태도를 보였어.,국민들은 두 당 간의 전쟁에 아주 무관심한 태도를 보였어.,50대,DGTS20000089.json
1111533,그는 항상 지 장모가 좀 짜증스럽다고 그택 생각했어. 한편으로는 어머니와 딸의 관...,그는 항상 자기 장모가 좀 짜증스럽다고 늘 생각했어. 한편으로는 어머니와 딸의 관...,30대,DGTS20000091.json
1144408,가난한 처녀는 지 누디기 드레스를 부끄러워했어.,가난한 처녀는 자기 누더기 드레스를 부끄러워했어.,60대 이상,DGTS20000144.json
1147210,누 빨간 옷 입은 긴 까만 머리?,누구 빨간 옷 입은 긴 까만 머리?,40대,DGTS20000148.json


### 대체어 처리

In [29]:
대체어_위치_방언 = df["방언"].str.contains(r"\&\S+[0-9]\&", regex=True)
대체어_위치_표준어 = df["표준어"].str.contains(r"\&\S+[0-9]\&", regex=True)

In [30]:
df["방언"] = df["방언"].str.replace(r"\&name[0-9]+\&", "name", regex=True)
df["방언"] = df["방언"].str.replace(r"\&address[0-9]+\&", "address", regex=True)
df["방언"] = df["방언"].str.replace(r"\&company-name[0-9]+\&", "companyname", regex=True)
df["방언"] = df["방언"].str.replace(r"\&companyname[0-9]+\&", "companyname", regex=True)

df[대체어_위치_방언]

,방언,표준어,연령대,출처 파일
69,아니 내가 제일 젊진 않고 여기 저 저 name이 엄마 name이 엄마가 더 젊지.,아니 내가 제일 젊진 않고 여기 저 저 &name1&이 엄마 &name1&이 엄마가...,60대 이상,DGDQ20000020.json
70,name이 엄마가 나보다 한 살 아래지 나이로 따지면,&name1&이 엄마가 나보다 한 살 아래지 나이로 따지면,60대 이상,DGDQ20000020.json
288,강릉시 주문진읍에 사는 스무살 name이라고 합니다.,강릉시 주문진읍에 사는 스무살 &name1&이라고 합니다.,20대,DGDQ20000027.json
487,저희가 name 선생님이라는 캡틴을 잘 만나 가지고,저희가 &name1& 선생님이라는 캡틴을 잘 만나 가지고,20대,DGDQ20000027.json
609,name이라는 애가 있습니다.,&name2&이라는 애가 있습니다.,20대,DGDQ20000027.json
...,...,...,...,...
1063038,그러다 name가 저보고 너 다 들었지?,그러다 &name3&가 저보고 너 다 들었지?,50대,DGIN21920687.json
1063044,name보다 행복한 아인 거 같아요.,&name3&보다 행복한 아인 거 같아요.,50대,DGIN21920687.json
1063045,name는 엄마랑 아빠가 다같이 못산다 하는데,&name3&는 엄마랑 아빠가 다같이 못산다 하는데,50대,DGIN21920687.json
1063047,그래 사램은 말이다 name아.,그래 사람은 말이다 &name4&아.,50대,DGIN21920687.json


In [31]:
df["표준어"] = df["표준어"].str.replace(r"\&name[0-9]\&", "name", regex=True)
df["표준어"] = df["표준어"].str.replace(r"\&address[0-9]\&", "address", regex=True)
df["표준어"] = df["표준어"].str.replace(r"\&company-name[0-9]\&", "companyname", regex=True)
df["표준어"] = df["표준어"].str.replace(r"\&companyname[0-9]\&", "companyname", regex=True)

df[대체어_위치_표준어]

,방언,표준어,연령대,출처 파일
69,아니 내가 제일 젊진 않고 여기 저 저 name이 엄마 name이 엄마가 더 젊지.,아니 내가 제일 젊진 않고 여기 저 저 name이 엄마 name이 엄마가 더 젊지.,60대 이상,DGDQ20000020.json
70,name이 엄마가 나보다 한 살 아래지 나이로 따지면,name이 엄마가 나보다 한 살 아래지 나이로 따지면,60대 이상,DGDQ20000020.json
288,강릉시 주문진읍에 사는 스무살 name이라고 합니다.,강릉시 주문진읍에 사는 스무살 name이라고 합니다.,20대,DGDQ20000027.json
487,저희가 name 선생님이라는 캡틴을 잘 만나 가지고,저희가 name 선생님이라는 캡틴을 잘 만나 가지고,20대,DGDQ20000027.json
609,name이라는 애가 있습니다.,name이라는 애가 있습니다.,20대,DGDQ20000027.json
...,...,...,...,...
1063038,그러다 name가 저보고 너 다 들었지?,그러다 name가 저보고 너 다 들었지?,50대,DGIN21920687.json
1063044,name보다 행복한 아인 거 같아요.,name보다 행복한 아인 거 같아요.,50대,DGIN21920687.json
1063045,name는 엄마랑 아빠가 다같이 못산다 하는데,name는 엄마랑 아빠가 다같이 못산다 하는데,50대,DGIN21920687.json
1063047,그래 사램은 말이다 name아.,그래 사람은 말이다 name아.,50대,DGIN21920687.json


### 데이터 정제

In [32]:
df["길이"] = [min(len(dia.split()), len(stan.split())) for dia, stan in zip(df["방언"], df["표준어"])]
df["길이"]

0          1
1          1
2          3
3          4
4          1
          ..
1573232    3
1573233    3
1573234    2
1573235    3
1573236    2
Name: 길이, Length: 1573237, dtype: int64

In [33]:
suf_len_df = df[df["길이"] > 4]
len(suf_len_df)

841540

In [34]:
dia_sur_len = len(suf_len_df[suf_len_df["방언"] != suf_len_df["표준어"]])
print("길이로 제외한 후 남은 방언 샘플 수: {0}".format(dia_sur_len))

길이로 제외한 후 남은 방언 샘플 수: 557969


방언 데이터의 순도가 전라 방언에 비해 훨씬 높다고 봐야할 것 같다.

In [35]:
suf_len_df.to_csv("../datas/output/gangwon_dialect_data_age_processed.csv")

In [43]:
len(suf_len_df)

841540

In [44]:
sur_len_df = pd.read_csv("../datas/output/gangwon_dialect_data_age_processed.csv", index_col=0)
len(suf_len_df)

841540

### 데이터 샘플링

In [16]:
dial_df = suf_len_df[suf_len_df["방언"] != suf_len_df["표준어"]]
stan_df = suf_len_df[suf_len_df["방언"] == suf_len_df["표준어"]]

In [17]:
len(stan_df)

134312

강원 방언는 표준어 샘플이 방언 샘플보다 훨씬 작으니 훈련 셋을 그냥 사용하도록 하겠다.

* 강원 방언 필터링 후 훈련 샘플 수: 841540
* 강원 방언 필터링 후 테스트 샘플 수: 134312

In [45]:
sampled_df = suf_len_df

In [46]:
sampled_df.to_csv("../datas/output/gangwon_dialect_data_age_sampled.csv")

### 형태소 토큰화

In [21]:
import MeCab

def pos(sentence: str):
    """한국어 토큰을 분리합니다. 토큰과 품사를 튜플 리스트로 반환합니다.
        
        매개변수:
            sentence (str): 토큰화할 문장 
        반환값:
            token list (list[tuple]): 토큰과 품사 리스트
    """
    t = MeCab.Tagger()
    tag_result = t.parse(sentence)
    tag_result = tag_result.replace("\t", ".@!").replace("\n", ".@!").split(".@!")
    tag_word = tag_result[::2][:-1] # 마지막 EOS는 자른다
    tag_info = tag_result[1::2][:-1] # 마지막 EOS는 자른다
    return [(word, info.split(',')[0]) for word, info in zip(tag_word, tag_info)]

def morphs(sentence: str):
    """한국어 토큰을 분리합니다. 토큰의 리스트를 반환합니다.
        
        매개변수:
            sentence (str): 토큰화할 문장 
        반환값:
            token list (list): 토큰 리스트
    """
    t = MeCab.Tagger()
    tag_result = t.parse(sentence)
    tag_result = tag_result.replace("\t", ".@!").replace("\n", ".@!").split(".@!")
    return tag_result[::2][:-1]

def space_tokenize(original_sentence, token_list):
    """
    문장을 decode할 때 한국어의 띄어쓰기도 되살릴 수 있도록 token에 이를 반영하도록 처리하는 함수
    """
    
    space_token = original_sentence.split()

    if len(space_token) == 0:
        return token_list

    token_idx = 0
    cur_token = space_token[token_idx]
    cum_word = ""
    for i, word in enumerate(token_list):
        cum_word += word

        if len(cum_word) > len(word):
            token_list[i] = "##" + token_list[i]
            
        if cum_word == cur_token:
            token_idx += 1
            cum_word = ""
            cur_token = space_token[min(token_idx, len(space_token) - 1)]
    
    return token_list

def morph_and_preprocess(sentence: str):
    """한국어 토큰을 분리하고 전처리합니다. 토큰의 리스트를 반환합니다.
        
        매개변수:
            sentence (str): 토큰화할 문장 
        반환값:
            token list (list): 토큰 리스트
    """
    
    pos_result = pos(sentence)
    word_list = [word for word, _ in pos_result]
    word_list = space_tokenize(sentence, word_list) # 공백 복원 토큰화
    
    return word_list

In [22]:
dialect_list = sampled_df["방언"].to_list()
dialect_list = list(map(morph_and_preprocess, dialect_list))

standard_list = sampled_df["표준어"].to_list()
standard_list = list(map(morph_and_preprocess, standard_list))

In [23]:
list_to_spaced_sentence = lambda li : " ".join(li)
sampled_df["방언"] = list(map(list_to_spaced_sentence, dialect_list))
sampled_df["표준어"] = list(map(list_to_spaced_sentence, standard_list))

sampled_df.head()

C:\Users\wnsgh\AppData\Local\Temp\ipykernel_31188\2846202428.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sampled_df["방언"] = list(map(list_to_spaced_sentence, dialect_list))
C:\Users\wnsgh\AppData\Local\Temp\ipykernel_31188\2846202428.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sampled_df["표준어"] = list(map(list_to_spaced_sentence, standard_list))


,방언,표준어,연령대,출처 파일,길이
0,나 ##는 사실 내 인생 자체 ##가 증말 다이어트 ##라고 해도 과언 ##이 아닐...,나 ##는 사실 내 인생 자체 ##가 정말 다이어트 ##라고 해도 과언 ##이 아닐...,20대,DGIN21500168.json,11
1,일단 다이어트 ##를 하 ##면은 사람 ##이 예민 ##이 해 ##지 ##고 짜증 ...,일단 다이어트 ##를 하 ##면은 사람 ##이 예민 ##이 해 ##지 ##고 짜증 ...,20대,DGIN21500168.json,9
2,그런데 이제 나 ##는 무 ##조근 기분 나쁜 다이어트 ##가 아니 ##라 쪼끔 내...,그런데 이제 나 ##는 무조건 기분 나쁜 다이어트 ##가 아니 ##라 쪼끔 내 몸 ...,20대,DGIN21500168.json,12
3,기분 좋 ##은 다이어트 ##를 하 ##고 싶 ##은데,기분 좋 ##은 다이어트 ##를 하 ##고 싶 ##은데,20대,DGIN21500168.json,5
6,뭐 ##~ 어떤 다이어트 ##가 너 ##한테 도움 ##이 됐 ##는지,뭐 ##~ 어떤 다이어트 ##가 너 ##한테 도움 ##이 됐 ##는지,20대,DGIN21500168.json,6


In [24]:
sampled_df.to_csv("../datas/output/gangwon_dialect_test_형태소.csv")

### SentencePiece 토큰화

In [47]:
df = pd.read_csv("../datas/output/gangwon_dialect_data_age_sampled.csv", index_col=0)
df

,방언,표준어,연령대,출처 파일,길이
6,바다의 무슨 얘기를 해야 되나?,바다의 무슨 얘기를 해야 되나?,60대 이상,DGDQ20000020.json,5
10,옛날에 바다에서 내가 잘한다고 엠비씨까지 갔다 왔거든요 제가.,옛날에 바다에서 내가 잘한다고 엠비씨까지 갔다 왔거든요 제가.,60대 이상,DGDQ20000020.json,8
13,구십 몇 년 도에 했는데요?,구십 몇 년 도에 했는데요?,60대 이상,DGDQ20000020.json,5
14,그때가 엠비씨 방송국에서 해녀 하라고 또 인터뷰 하자 하대?,그때가 엠비씨 방송국에서 해녀 하라고 또 인터뷰 하자 하더라?,60대 이상,DGDQ20000020.json,9
15,그래서 엠비씨 방송 공개 홀 강릉 가 가지고 노래도 부르구,그래서 엠비씨 방송 공개 홀 강릉 가 가지고 노래도 부르구,60대 이상,DGDQ20000020.json,10
...,...,...,...,...,...
1573194,감자 썩은 데는 삐제 내세요.,감자 썩은 데는 도려 내세요.,20대,DGTS20004276.json,5
1573195,멀쩡한 것만 골래 밥에 넣어주세요.,멀쩡한 것만 골라 밥에 넣어주세요.,20대,DGTS20004276.json,5
1573196,저 녀석은 도대체 왜 자주 삐지냐?,저 녀석은 도대체 왜 자주 토라지나?,20대,DGTS20004276.json,6
1573197,사소한 것에도 삐지니 말을 섞기가 싫어.,사소한 것에도 토라지니 말을 섞기가 싫어.,20대,DGTS20004276.json,6


In [48]:
df = df.dropna(how='any')
with open('../datas/output/gangwon_dialect_text.txt', 'w', encoding='utf8') as f:
    f.write('\n'.join(df['방언'] + " " + df["표준어"]))

In [50]:
import sentencepiece as spm

spm.SentencePieceTrainer.train(input='../datas/output/gangwon_dialect_text.txt', model_prefix='gangwon_sp', vocab_size=30000)

In [53]:
sp_tokenizer = spm.SentencePieceProcessor(model_file='gangwon_sp.model')

code_result_dialect = sp_tokenizer.encode(df["방언"].to_list(), out_type=str)
code_result_standard = sp_tokenizer.encode(df["표준어"].to_list(), out_type=str)

In [58]:
df["방언"] = [' '.join(res) for res in code_result_dialect]
df["방언"]

6                                ▁바다 의 ▁무슨 ▁얘기를 ▁해야 ▁되나 ?
10         ▁옛날에 ▁바다에서 ▁내가 ▁잘한다고 ▁엠 비 씨 까지 ▁갔다 ▁왔거든요 ▁제가 .
13                                ▁구십 ▁몇 ▁년 ▁ 도 에 ▁했는데요 ?
14         ▁그때가 ▁엠 비 씨 ▁방송국 에서 ▁해녀 ▁하라고 ▁또 ▁인터뷰 ▁하자 ▁하대 ?
15         ▁그래서 ▁엠 비 씨 ▁방송 ▁공개 ▁홀 ▁강릉 ▁가 ▁가지고 ▁노래 도 ▁부르 구
                                ...                      
1573194                           ▁감자 ▁썩은 ▁데는 ▁삐 제 ▁내세요 .
1573195                       ▁멀쩡한 ▁것만 ▁골 래 ▁밥에 ▁넣어 주세요 .
1573196                       ▁저 ▁녀석은 ▁도대체 ▁왜 ▁자주 ▁삐지 냐 ?
1573197                 ▁사소한 ▁것에 도 ▁삐지 니 ▁말을 ▁ 섞 기가 ▁싫어 .
1573198                 ▁저 ▁아이는 ▁삐지 기 ▁쉬우니 ▁잘 ▁달래 줘야 ▁돼 .
Name: 방언, Length: 841540, dtype: object

In [59]:
df["표준어"] = [' '.join(res) for res in code_result_standard]
df["표준어"]

6                                 ▁바다 의 ▁무슨 ▁얘기를 ▁해야 ▁되나 ?
10          ▁옛날에 ▁바다에서 ▁내가 ▁잘한다고 ▁엠 비 씨 까지 ▁갔다 ▁왔거든요 ▁제가 .
13                                 ▁구십 ▁몇 ▁년 ▁ 도 에 ▁했는데요 ?
14         ▁그때가 ▁엠 비 씨 ▁방송국 에서 ▁해녀 ▁하라고 ▁또 ▁인터뷰 ▁하자 ▁하더라 ?
15          ▁그래서 ▁엠 비 씨 ▁방송 ▁공개 ▁홀 ▁강릉 ▁가 ▁가지고 ▁노래 도 ▁부르 구
                                ...                       
1573194                           ▁감자 ▁썩은 ▁데는 ▁ 도 려 ▁내세요 .
1573195                         ▁멀쩡한 ▁것만 ▁골라 ▁밥에 ▁넣어 주세요 .
1573196                       ▁저 ▁녀석은 ▁도대체 ▁왜 ▁자주 ▁토라지 나 ?
1573197                 ▁사소한 ▁것에 도 ▁토라지 니 ▁말을 ▁ 섞 기가 ▁싫어 .
1573198                 ▁저 ▁아이는 ▁토라지 기 ▁쉬우니 ▁잘 ▁달래 줘야 ▁돼 .
Name: 표준어, Length: 841540, dtype: object

In [60]:
df.to_csv("../datas/output/gangwon_dialect_data_SentencePiece.csv")

### 테스트셋 SentencePiece

In [62]:
df = pd.read_csv("../datas/output/gangwon_dialect_test_age_processed.csv", index_col=0)
df

,방언,표준어,연령대,출처 파일,길이
0,나는 사실 내 인생 자체가 증말 다이어트라고 해도 과언이 아닐 정도인데,나는 사실 내 인생 자체가 정말 다이어트라고 해도 과언이 아닐 정도인데,20대,DGIN21500168.json,11
1,일단 다이어트를 하면은 사람이 예민이 해지고 짜증을 많이 내잖아.,일단 다이어트를 하면은 사람이 예민이 해지고 짜증을 많이 내잖아.,20대,DGIN21500168.json,9
2,그런데 이제 나는 무조근 기분 나쁜 다이어트가 아니라 쪼끔 내 몸이 건강해지는,그런데 이제 나는 무조건 기분 나쁜 다이어트가 아니라 쪼끔 내 몸이 건강해지는,20대,DGIN21500168.json,12
3,기분 좋은 다이어트를 하고 싶은데,기분 좋은 다이어트를 하고 싶은데,20대,DGIN21500168.json,5
6,뭐~ 어떤 다이어트가 너한테 도움이 됐는지,뭐~ 어떤 다이어트가 너한테 도움이 됐는지,20대,DGIN21500168.json,6
...,...,...,...,...,...
276793,사실 뭐~ 어~ 기억은 안 나고 그때는 난 돼지고기를 먹기 위해서 많이 갔었지.,사실 뭐~ 어~ 기억은 안 나고 그때는 난 돼지고기를 먹기 위해서 많이 갔었지.,30대,DGIN21801133.json,13
276794,어렸을 때 삼겹살이라던가 거기 항상 냉동이긴 했는데 어~ 그거를 많이 먹었었던 것 같애.,어렸을 때 삼겹살이라던가 거기 항상 냉동이긴 했는데 어~ 그거를 많이 먹었었던 것 같아.,30대,DGIN21801133.json,13
276795,소괴기는 사실 어~ 나이가 들고 어~,소고기는 사실 어~ 나이가 들고 어~,30대,DGIN21801133.json,6
276796,먹었었던 것 같애 사실 어~ 젊었을 때나 어렸을 때라던가 이럴 때는 워낙 소괴기가 ...,먹었었던 것 같아 사실 어~ 젊었을 때나 어렸을 때라던가 이럴 때는 워낙 소고기가 ...,30대,DGIN21801133.json,15


In [63]:
df = df.dropna(how='any')
with open('../datas/output/gangwon_dialect_text.txt', 'w', encoding='utf8') as f:
    f.write('\n'.join(df['방언'] + " " + df["표준어"]))

In [65]:
import sentencepiece as spm

spm.SentencePieceTrainer.train(input='../datas/output/gangwon_dialect_text.txt', model_prefix='gangwon_sp_test', vocab_size=30000)

In [66]:
sp_tokenizer = spm.SentencePieceProcessor(model_file='gangwon_sp_test.model')

code_result_dialect = sp_tokenizer.encode(df["방언"].to_list(), out_type=str)
code_result_standard = sp_tokenizer.encode(df["표준어"].to_list(), out_type=str)

In [67]:
df["방언"] = [' '.join(res) for res in code_result_dialect]
df["표준어"] = [' '.join(res) for res in code_result_standard]

In [68]:
df.to_csv("../datas/output/gangwon_dialect_test_SentencePiece.csv")

### 자모 토큰화

In [69]:
df = pd.read_csv("../datas/output/gangwon_dialect_data_age_sampled.csv", index_col=0)
df

,방언,표준어,연령대,출처 파일,길이
6,바다의 무슨 얘기를 해야 되나?,바다의 무슨 얘기를 해야 되나?,60대 이상,DGDQ20000020.json,5
10,옛날에 바다에서 내가 잘한다고 엠비씨까지 갔다 왔거든요 제가.,옛날에 바다에서 내가 잘한다고 엠비씨까지 갔다 왔거든요 제가.,60대 이상,DGDQ20000020.json,8
13,구십 몇 년 도에 했는데요?,구십 몇 년 도에 했는데요?,60대 이상,DGDQ20000020.json,5
14,그때가 엠비씨 방송국에서 해녀 하라고 또 인터뷰 하자 하대?,그때가 엠비씨 방송국에서 해녀 하라고 또 인터뷰 하자 하더라?,60대 이상,DGDQ20000020.json,9
15,그래서 엠비씨 방송 공개 홀 강릉 가 가지고 노래도 부르구,그래서 엠비씨 방송 공개 홀 강릉 가 가지고 노래도 부르구,60대 이상,DGDQ20000020.json,10
...,...,...,...,...,...
1573194,감자 썩은 데는 삐제 내세요.,감자 썩은 데는 도려 내세요.,20대,DGTS20004276.json,5
1573195,멀쩡한 것만 골래 밥에 넣어주세요.,멀쩡한 것만 골라 밥에 넣어주세요.,20대,DGTS20004276.json,5
1573196,저 녀석은 도대체 왜 자주 삐지냐?,저 녀석은 도대체 왜 자주 토라지나?,20대,DGTS20004276.json,6
1573197,사소한 것에도 삐지니 말을 섞기가 싫어.,사소한 것에도 토라지니 말을 섞기가 싫어.,20대,DGTS20004276.json,6


In [70]:
from jamo import h2j, j2hcj # 자모화
from jamo import hcj2j, j2h

text = df.iloc[0, 0][0]
jamo_str = j2hcj(h2j(text))
print(jamo_str)

ㅂㅏ


In [2]:
from jamo import h2j, j2hcj # 자모화

def jamo_tokenization(sentence):
    token_list = []

    for c in sentence:
        if c == ' ':
            token_list.append("<SPC>")
        else:
            jamo_str = j2hcj(h2j(c))
            token_list.extend(jamo_str)
            token_list.append("<SEP>")
    
    return token_list

In [72]:
dialect_list = df["방언"].to_list()
dialect_list = list(map(jamo_tokenization, dialect_list))

standard_list = df["표준어"].to_list()
standard_list = list(map(jamo_tokenization, standard_list))

In [73]:
list_to_spaced_sentence = lambda li : " ".join(li)
df.loc[df.index, "방언"] = list(map(list_to_spaced_sentence, dialect_list))
df.loc[df.index, "표준어"] = list(map(list_to_spaced_sentence, standard_list))

df

,방언,표준어,연령대,출처 파일,길이
6,ㅂ ㅏ <SEP> ㄷ ㅏ <SEP> ㅇ ㅢ <SEP> <SPC> ㅁ ㅜ <SEP> ...,ㅂ ㅏ <SEP> ㄷ ㅏ <SEP> ㅇ ㅢ <SEP> <SPC> ㅁ ㅜ <SEP> ...,60대 이상,DGDQ20000020.json,5
10,ㅇ ㅖ ㅅ <SEP> ㄴ ㅏ ㄹ <SEP> ㅇ ㅔ <SEP> <SPC> ㅂ ㅏ <S...,ㅇ ㅖ ㅅ <SEP> ㄴ ㅏ ㄹ <SEP> ㅇ ㅔ <SEP> <SPC> ㅂ ㅏ <S...,60대 이상,DGDQ20000020.json,8
13,ㄱ ㅜ <SEP> ㅅ ㅣ ㅂ <SEP> <SPC> ㅁ ㅕ ㅊ <SEP> <SPC> ...,ㄱ ㅜ <SEP> ㅅ ㅣ ㅂ <SEP> <SPC> ㅁ ㅕ ㅊ <SEP> <SPC> ...,60대 이상,DGDQ20000020.json,5
14,ㄱ ㅡ <SEP> ㄸ ㅐ <SEP> ㄱ ㅏ <SEP> <SPC> ㅇ ㅔ ㅁ <SEP...,ㄱ ㅡ <SEP> ㄸ ㅐ <SEP> ㄱ ㅏ <SEP> <SPC> ㅇ ㅔ ㅁ <SEP...,60대 이상,DGDQ20000020.json,9
15,ㄱ ㅡ <SEP> ㄹ ㅐ <SEP> ㅅ ㅓ <SEP> <SPC> ㅇ ㅔ ㅁ <SEP...,ㄱ ㅡ <SEP> ㄹ ㅐ <SEP> ㅅ ㅓ <SEP> <SPC> ㅇ ㅔ ㅁ <SEP...,60대 이상,DGDQ20000020.json,10
...,...,...,...,...,...
1573194,ㄱ ㅏ ㅁ <SEP> ㅈ ㅏ <SEP> <SPC> ㅆ ㅓ ㄱ <SEP> ㅇ ㅡ ㄴ ...,ㄱ ㅏ ㅁ <SEP> ㅈ ㅏ <SEP> <SPC> ㅆ ㅓ ㄱ <SEP> ㅇ ㅡ ㄴ ...,20대,DGTS20004276.json,5
1573195,ㅁ ㅓ ㄹ <SEP> ㅉ ㅓ ㅇ <SEP> ㅎ ㅏ ㄴ <SEP> <SPC> ㄱ ㅓ ...,ㅁ ㅓ ㄹ <SEP> ㅉ ㅓ ㅇ <SEP> ㅎ ㅏ ㄴ <SEP> <SPC> ㄱ ㅓ ...,20대,DGTS20004276.json,5
1573196,ㅈ ㅓ <SEP> <SPC> ㄴ ㅕ <SEP> ㅅ ㅓ ㄱ <SEP> ㅇ ㅡ ㄴ <S...,ㅈ ㅓ <SEP> <SPC> ㄴ ㅕ <SEP> ㅅ ㅓ ㄱ <SEP> ㅇ ㅡ ㄴ <S...,20대,DGTS20004276.json,6
1573197,ㅅ ㅏ <SEP> ㅅ ㅗ <SEP> ㅎ ㅏ ㄴ <SEP> <SPC> ㄱ ㅓ ㅅ <S...,ㅅ ㅏ <SEP> ㅅ ㅗ <SEP> ㅎ ㅏ ㄴ <SEP> <SPC> ㄱ ㅓ ㅅ <S...,20대,DGTS20004276.json,6


In [74]:
df.to_csv("../datas/output/gangwon_dialect_data_jamo.csv")

### 테스트셋 자모

In [3]:
df = pd.read_csv("../datas/output/gangwon_dialect_test_age_processed.csv", index_col=0)
df

,방언,표준어,연령대,출처 파일,길이
0,나는 사실 내 인생 자체가 증말 다이어트라고 해도 과언이 아닐 정도인데,나는 사실 내 인생 자체가 정말 다이어트라고 해도 과언이 아닐 정도인데,20대,DGIN21500168.json,11
1,일단 다이어트를 하면은 사람이 예민이 해지고 짜증을 많이 내잖아.,일단 다이어트를 하면은 사람이 예민이 해지고 짜증을 많이 내잖아.,20대,DGIN21500168.json,9
2,그런데 이제 나는 무조근 기분 나쁜 다이어트가 아니라 쪼끔 내 몸이 건강해지는,그런데 이제 나는 무조건 기분 나쁜 다이어트가 아니라 쪼끔 내 몸이 건강해지는,20대,DGIN21500168.json,12
3,기분 좋은 다이어트를 하고 싶은데,기분 좋은 다이어트를 하고 싶은데,20대,DGIN21500168.json,5
6,뭐~ 어떤 다이어트가 너한테 도움이 됐는지,뭐~ 어떤 다이어트가 너한테 도움이 됐는지,20대,DGIN21500168.json,6
...,...,...,...,...,...
276793,사실 뭐~ 어~ 기억은 안 나고 그때는 난 돼지고기를 먹기 위해서 많이 갔었지.,사실 뭐~ 어~ 기억은 안 나고 그때는 난 돼지고기를 먹기 위해서 많이 갔었지.,30대,DGIN21801133.json,13
276794,어렸을 때 삼겹살이라던가 거기 항상 냉동이긴 했는데 어~ 그거를 많이 먹었었던 것 같애.,어렸을 때 삼겹살이라던가 거기 항상 냉동이긴 했는데 어~ 그거를 많이 먹었었던 것 같아.,30대,DGIN21801133.json,13
276795,소괴기는 사실 어~ 나이가 들고 어~,소고기는 사실 어~ 나이가 들고 어~,30대,DGIN21801133.json,6
276796,먹었었던 것 같애 사실 어~ 젊었을 때나 어렸을 때라던가 이럴 때는 워낙 소괴기가 ...,먹었었던 것 같아 사실 어~ 젊었을 때나 어렸을 때라던가 이럴 때는 워낙 소고기가 ...,30대,DGIN21801133.json,15


In [4]:
dialect_list = df["방언"].to_list()
dialect_list = list(map(jamo_tokenization, dialect_list))

standard_list = df["표준어"].to_list()
standard_list = list(map(jamo_tokenization, standard_list))

In [5]:
list_to_spaced_sentence = lambda li : " ".join(li)
df.loc[df.index, "방언"] = list(map(list_to_spaced_sentence, dialect_list))
df.loc[df.index, "표준어"] = list(map(list_to_spaced_sentence, standard_list))

df

,방언,표준어,연령대,출처 파일,길이
0,ㄴ ㅏ <SEP> ㄴ ㅡ ㄴ <SEP> <SPC> ㅅ ㅏ <SEP> ㅅ ㅣ ㄹ <S...,ㄴ ㅏ <SEP> ㄴ ㅡ ㄴ <SEP> <SPC> ㅅ ㅏ <SEP> ㅅ ㅣ ㄹ <S...,20대,DGIN21500168.json,11
1,ㅇ ㅣ ㄹ <SEP> ㄷ ㅏ ㄴ <SEP> <SPC> ㄷ ㅏ <SEP> ㅇ ㅣ <S...,ㅇ ㅣ ㄹ <SEP> ㄷ ㅏ ㄴ <SEP> <SPC> ㄷ ㅏ <SEP> ㅇ ㅣ <S...,20대,DGIN21500168.json,9
2,ㄱ ㅡ <SEP> ㄹ ㅓ ㄴ <SEP> ㄷ ㅔ <SEP> <SPC> ㅇ ㅣ <SEP...,ㄱ ㅡ <SEP> ㄹ ㅓ ㄴ <SEP> ㄷ ㅔ <SEP> <SPC> ㅇ ㅣ <SEP...,20대,DGIN21500168.json,12
3,ㄱ ㅣ <SEP> ㅂ ㅜ ㄴ <SEP> <SPC> ㅈ ㅗ ㅎ <SEP> ㅇ ㅡ ㄴ ...,ㄱ ㅣ <SEP> ㅂ ㅜ ㄴ <SEP> <SPC> ㅈ ㅗ ㅎ <SEP> ㅇ ㅡ ㄴ ...,20대,DGIN21500168.json,5
6,ㅁ ㅝ <SEP> ~ <SEP> <SPC> ㅇ ㅓ <SEP> ㄸ ㅓ ㄴ <SEP> ...,ㅁ ㅝ <SEP> ~ <SEP> <SPC> ㅇ ㅓ <SEP> ㄸ ㅓ ㄴ <SEP> ...,20대,DGIN21500168.json,6
...,...,...,...,...,...
276793,ㅅ ㅏ <SEP> ㅅ ㅣ ㄹ <SEP> <SPC> ㅁ ㅝ <SEP> ~ <SEP> ...,ㅅ ㅏ <SEP> ㅅ ㅣ ㄹ <SEP> <SPC> ㅁ ㅝ <SEP> ~ <SEP> ...,30대,DGIN21801133.json,13
276794,ㅇ ㅓ <SEP> ㄹ ㅕ ㅆ <SEP> ㅇ ㅡ ㄹ <SEP> <SPC> ㄸ ㅐ <S...,ㅇ ㅓ <SEP> ㄹ ㅕ ㅆ <SEP> ㅇ ㅡ ㄹ <SEP> <SPC> ㄸ ㅐ <S...,30대,DGIN21801133.json,13
276795,ㅅ ㅗ <SEP> ㄱ ㅚ <SEP> ㄱ ㅣ <SEP> ㄴ ㅡ ㄴ <SEP> <SPC...,ㅅ ㅗ <SEP> ㄱ ㅗ <SEP> ㄱ ㅣ <SEP> ㄴ ㅡ ㄴ <SEP> <SPC...,30대,DGIN21801133.json,6
276796,ㅁ ㅓ ㄱ <SEP> ㅇ ㅓ ㅆ <SEP> ㅇ ㅓ ㅆ <SEP> ㄷ ㅓ ㄴ <SEP...,ㅁ ㅓ ㄱ <SEP> ㅇ ㅓ ㅆ <SEP> ㅇ ㅓ ㅆ <SEP> ㄷ ㅓ ㄴ <SEP...,30대,DGIN21801133.json,15


In [6]:
df.to_csv("../datas/output/gangwon_dialect_test_jamo.csv")

### Byte 수준 BPE

In [12]:
df = pd.read_csv("../datas/output/gangwon_dialect_data_age_sampled.csv", index_col=0)
df

df = df.dropna(how='any')
with open('../datas/output/gangwon_dialect_data_text.txt', 'w', encoding='utf8') as f:
    f.write('\n'.join(df['방언'] + " " + df["표준어"]))

In [21]:
from tokenizers import ByteLevelBPETokenizer

bpe_tokenizer = ByteLevelBPETokenizer()
bpe_tokenizer.train(
    files="../datas/output/gangwon_dialect_data_text.txt",
)

['./vocab.json', './merges.txt']

In [14]:
code_result_dialect = bpe_tokenizer.encode_batch(df["방언"])
code_result_standard = bpe_tokenizer.encode_batch(df["표준어"])

In [15]:
df.loc[df.index, "방언"] = [' '.join(res.tokens) for res in code_result_dialect]
df["방언"]

6            ë°Ķëĭ¤ ìĿĺ Ġë¬´ìĬ¨ Ġìĸĺê¸°ë¥¼ Ġíķ´ìķ¼ ĠëĲĺëĤĺ ?
10         ìĺĽëĤłìĹĲ Ġë°Ķëĭ¤ìĹĲìĦľ ĠëĤ´ê°Ģ Ġìŀĺíķľëĭ¤ê³ł ...
13                 êµ¬ìĭŃ Ġëªĩ ĠëħĦ ĠëıĦ ìĹĲ ĠíĸĪëĬĶëį°ìļĶ ?
14         ê·¸ëķĮê°Ģ ĠìĹłë¹Ħ ìĶ¨ Ġë°©ìĨ¡ êµŃìĹĲìĦľ Ġíķ´ëħ...
15         ê·¸ëŀĺìĦľ ĠìĹłë¹Ħ ìĶ¨ Ġë°©ìĨ¡ Ġê³µê°ľ ĠíĻĢ Ġê°...
                                 ...                        
1573194         ê°ĲìŀĲ Ġìį©ìĿĢ Ġëį°ëĬĶ ĠìĤĲ ìłľ ĠëĤ´ìĦ¸ìļĶ .
1573195    ë©Ģì©¡íķľ Ġê²ĥë§Į Ġê³¨ ëŀĺ Ġë°¥ìĹĲ ĠëĦ£ìĸ´ ì£¼...
1573196    ìłĢ ĠëħĢìĦĿìĿĢ ĠëıĦëĮĢì²´ ĠìĻľ ĠìŀĲì£¼ ĠìĤĲ ì§...
1573197    ìĤ¬ ìĨĮíķľ Ġê²ĥìĹĲëıĦ ĠìĤĲ ì§ĢëĭĪ Ġë§ĲìĿĦ ĠìĦŀ...
1573198    ìłĢ ĠìķĦìĿ´ëĬĶ ĠìĤĲ ì§Ģê¸° Ġìī¬ ìļ°ëĭĪ Ġìŀĺ Ġë...
Name: 방언, Length: 841540, dtype: object

In [16]:
df.loc[df.index, "표준어"] = [' '.join(res.tokens) for res in code_result_standard]
df["표준어"]

6            ë°Ķëĭ¤ ìĿĺ Ġë¬´ìĬ¨ Ġìĸĺê¸°ë¥¼ Ġíķ´ìķ¼ ĠëĲĺëĤĺ ?
10         ìĺĽëĤłìĹĲ Ġë°Ķëĭ¤ìĹĲìĦľ ĠëĤ´ê°Ģ Ġìŀĺíķľëĭ¤ê³ł ...
13                 êµ¬ìĭŃ Ġëªĩ ĠëħĦ ĠëıĦ ìĹĲ ĠíĸĪëĬĶëį°ìļĶ ?
14         ê·¸ëķĮê°Ģ ĠìĹłë¹Ħ ìĶ¨ Ġë°©ìĨ¡ êµŃìĹĲìĦľ Ġíķ´ëħ...
15         ê·¸ëŀĺìĦľ ĠìĹłë¹Ħ ìĶ¨ Ġë°©ìĨ¡ Ġê³µê°ľ ĠíĻĢ Ġê°...
                                 ...                        
1573194         ê°ĲìŀĲ Ġìį©ìĿĢ Ġëį°ëĬĶ ĠëıĦ ëł¤ ĠëĤ´ìĦ¸ìļĶ .
1573195    ë©Ģì©¡íķľ Ġê²ĥë§Į Ġê³¨ëĿ¼ Ġë°¥ìĹĲ ĠëĦ£ìĸ´ ì£¼ì...
1573196    ìłĢ ĠëħĢìĦĿìĿĢ ĠëıĦëĮĢì²´ ĠìĻľ ĠìŀĲì£¼ ĠíĨłëĿ¼...
1573197    ìĤ¬ ìĨĮíķľ Ġê²ĥìĹĲëıĦ ĠíĨłëĿ¼ì§Ģ ëĭĪ Ġë§ĲìĿĦ Ġ...
1573198    ìłĢ ĠìķĦìĿ´ëĬĶ ĠíĨłëĿ¼ ì§Ģê¸° Ġìī¬ ìļ°ëĭĪ Ġìŀĺ...
Name: 표준어, Length: 841540, dtype: object

In [17]:
df.to_csv("../datas/output/gangwon_dialect_data_bpe.csv")

### 테스트셋 BPE

In [19]:
df = pd.read_csv("../datas/output/gangwon_dialect_test_age_processed.csv", index_col=0)
df

,방언,표준어,연령대,출처 파일,길이
0,나는 사실 내 인생 자체가 증말 다이어트라고 해도 과언이 아닐 정도인데,나는 사실 내 인생 자체가 정말 다이어트라고 해도 과언이 아닐 정도인데,20대,DGIN21500168.json,11
1,일단 다이어트를 하면은 사람이 예민이 해지고 짜증을 많이 내잖아.,일단 다이어트를 하면은 사람이 예민이 해지고 짜증을 많이 내잖아.,20대,DGIN21500168.json,9
2,그런데 이제 나는 무조근 기분 나쁜 다이어트가 아니라 쪼끔 내 몸이 건강해지는,그런데 이제 나는 무조건 기분 나쁜 다이어트가 아니라 쪼끔 내 몸이 건강해지는,20대,DGIN21500168.json,12
3,기분 좋은 다이어트를 하고 싶은데,기분 좋은 다이어트를 하고 싶은데,20대,DGIN21500168.json,5
6,뭐~ 어떤 다이어트가 너한테 도움이 됐는지,뭐~ 어떤 다이어트가 너한테 도움이 됐는지,20대,DGIN21500168.json,6
...,...,...,...,...,...
276793,사실 뭐~ 어~ 기억은 안 나고 그때는 난 돼지고기를 먹기 위해서 많이 갔었지.,사실 뭐~ 어~ 기억은 안 나고 그때는 난 돼지고기를 먹기 위해서 많이 갔었지.,30대,DGIN21801133.json,13
276794,어렸을 때 삼겹살이라던가 거기 항상 냉동이긴 했는데 어~ 그거를 많이 먹었었던 것 같애.,어렸을 때 삼겹살이라던가 거기 항상 냉동이긴 했는데 어~ 그거를 많이 먹었었던 것 같아.,30대,DGIN21801133.json,13
276795,소괴기는 사실 어~ 나이가 들고 어~,소고기는 사실 어~ 나이가 들고 어~,30대,DGIN21801133.json,6
276796,먹었었던 것 같애 사실 어~ 젊었을 때나 어렸을 때라던가 이럴 때는 워낙 소괴기가 ...,먹었었던 것 같아 사실 어~ 젊었을 때나 어렸을 때라던가 이럴 때는 워낙 소고기가 ...,30대,DGIN21801133.json,15


In [22]:
bpe_tokenizer = ByteLevelBPETokenizer()
bpe_tokenizer.train(
    files="../datas/output/gangwon_dialect_test_text.txt",
)

bpe_tokenizer.save_model("./")

['./vocab.json', './merges.txt']

In [23]:
code_result_dialect = bpe_tokenizer.encode_batch(df["방언"])
code_result_standard = bpe_tokenizer.encode_batch(df["표준어"])

In [24]:
df.loc[df.index, "방언"] = [' '.join(res.tokens) for res in code_result_dialect]
df["방언"]

0         ëĤĺëĬĶ ĠìĤ¬ìĭ¤ ĠëĤ´ ĠìĿ¸ìĥĿ ĠìŀĲì²´ê°Ģ Ġì¦Ŀë§Ĳ...
1         ìĿ¼ëĭ¨ Ġëĭ¤ìĿ´ìĸ´íĬ¸ë¥¼ Ġíķĺë©´ìĿĢ ĠìĤ¬ëŀĮìĿ´ ...
2         ê·¸ëŁ°ëį° ĠìĿ´ìłľ ĠëĤĺëĬĶ Ġë¬´ì¡°ê·¼ Ġê¸°ë¶Ħ Ġ...
3         ê¸°ë¶Ħ Ġì¢ĭìĿĢ Ġëĭ¤ìĿ´ìĸ´íĬ¸ë¥¼ Ġíķĺê³ł Ġìĭ¶ìĿ...
6         ëŃĲ ~ Ġìĸ´ëĸ¤ Ġëĭ¤ìĿ´ìĸ´íĬ¸ê°Ģ ĠëĦĪíķľíħĮ ĠëıĦ...
                                ...                        
276793    ìĤ¬ìĭ¤ ĠëŃĲ ~ Ġìĸ´ ~ Ġê¸°ìĸµìĿĢ ĠìķĪ ĠëĤĺê³ł Ġ...
276794    ìĸ´ëł¸ìĿĦ ĠëķĮ ĠìĤ¼ê²¹ìĤ´ ìĿ´ëĿ¼ëįĺê°Ģ Ġê±°ê¸°...
276795    ìĨĮ ê´´ ê¸°ëĬĶ ĠìĤ¬ìĭ¤ Ġìĸ´ ~ ĠëĤĺìĿ´ê°Ģ Ġëĵ¤ê...
276796    ë¨¹ ìĹĪìĹĪëįĺ Ġê²ĥ Ġê°Ļìķł ĠìĤ¬ìĭ¤ Ġìĸ´ ~ ĠìłĬ...
276797    ëĤ´ê°Ģ ĠìĿ¼ìĿĦ Ġíķĺë©´ìĦľ ĠëŃĶê°Ģ Ġìī½ ìĤ¬ë¦¬ ...
Name: 방언, Length: 182691, dtype: object

In [25]:
df.loc[df.index, "표준어"] = [' '.join(res.tokens) for res in code_result_standard]
df["표준어"]

0         ëĤĺëĬĶ ĠìĤ¬ìĭ¤ ĠëĤ´ ĠìĿ¸ìĥĿ ĠìŀĲì²´ê°Ģ Ġìłķë§Ĳ...
1         ìĿ¼ëĭ¨ Ġëĭ¤ìĿ´ìĸ´íĬ¸ë¥¼ Ġíķĺë©´ìĿĢ ĠìĤ¬ëŀĮìĿ´ ...
2         ê·¸ëŁ°ëį° ĠìĿ´ìłľ ĠëĤĺëĬĶ Ġë¬´ì¡°ê±´ Ġê¸°ë¶Ħ Ġ...
3         ê¸°ë¶Ħ Ġì¢ĭìĿĢ Ġëĭ¤ìĿ´ìĸ´íĬ¸ë¥¼ Ġíķĺê³ł Ġìĭ¶ìĿ...
6         ëŃĲ ~ Ġìĸ´ëĸ¤ Ġëĭ¤ìĿ´ìĸ´íĬ¸ê°Ģ ĠëĦĪíķľíħĮ ĠëıĦ...
                                ...                        
276793    ìĤ¬ìĭ¤ ĠëŃĲ ~ Ġìĸ´ ~ Ġê¸°ìĸµìĿĢ ĠìķĪ ĠëĤĺê³ł Ġ...
276794    ìĸ´ëł¸ìĿĦ ĠëķĮ ĠìĤ¼ê²¹ìĤ´ ìĿ´ëĿ¼ëįĺê°Ģ Ġê±°ê¸°...
276795    ìĨĮ ê³łê¸°ëĬĶ ĠìĤ¬ìĭ¤ Ġìĸ´ ~ ĠëĤĺìĿ´ê°Ģ Ġëĵ¤ê³...
276796    ë¨¹ ìĹĪìĹĪëįĺ Ġê²ĥ Ġê°ĻìķĦ ĠìĤ¬ìĭ¤ Ġìĸ´ ~ ĠìłĬ...
276797    ëĤ´ê°Ģ ĠìĿ¼ìĿĦ Ġíķĺë©´ìĦľ ĠëŃĶê°Ģ Ġìī½ ìĤ¬ë¦¬ ...
Name: 표준어, Length: 182691, dtype: object

In [26]:
df.to_csv("../datas/output/gangwon_dialect_test_bpe.csv")

## 경상 방언

In [2]:
df = pd.read_csv("../datas/output/gyeongsang_dialect_train_age.csv", index_col=0)
df

,방언,표준어,연령대,출처 파일
0,자 음식을 멀 좋아하느냐 하면은,자 음식을 멀 좋아하느냐 하면은,50대,DKCI20000001.json
1,나는 사실은 다 좋아합니다 다 좋아하는데 특히 육류 돼지고기,나는 사실은 다 좋아합니다 다 좋아하는데 특히 육류 돼지고기,50대,DKCI20000001.json
2,아 자기 아 오리를 싫어했었구나,아 자기 아 오리를 싫어했었구나,50대,DKCI20000001.json
3,머 못 묵 그라고 모든걸 다 좋아하는 데 즐기진 않는다.,머 못 먹 그리고 모든걸 다 좋아하는 데 즐기진 않는다.,50대,DKCI20000001.json
4,즐기진 않는다 자기는 나하고 반대네 나는,즐기진 않는다 자기는 나하고 반대네 나는,50대,DKCI20000001.json
...,...,...,...,...
2088712,뭐~ 라비 막 예쁘다고 예쁘게 생긴 거 같다 그러면서 얘기하고,뭐~ 라비 막 예쁘다고 예쁘게 생긴 거 같다 그러면서 얘기하고,30대,DKSR20007343.json
2088713,그다음에 뭐~ 김선호 맞나 김선호,그다음에 뭐~ 김선호 맞나 김선호,30대,DKSR20007343.json
2088714,그다음에 딘딘,그다음에 딘딘,30대,DKSR20007343.json
2088715,그~ 다 연정훈 이렇게 나오는 거는 엄마가 그래도 그~ 쪼꼼은 챙겨서 보는 거 같더...,그~ 다 연정훈 이렇게 나오는 거는 엄마가 그래도 그~ 조금은 챙겨서 보는 거 같더...,30대,DKSR20007343.json


In [3]:
# 테스트 데이터로 동일한 과정을 진행하고 싶을 때
df = pd.read_csv("../datas/output/gyeongsang_dialect_test_age.csv", index_col=0)
df

,방언,표준어,연령대,출처 파일
0,어~ 안녕하세요 그~ 혹시,어~ 안녕하세요 그~ 혹시,20대,DKSR20000890.json
1,그~ 남자 볼 때 가장 먼저 생각하는 이상형이 있으신가요?,그~ 남자 볼 때 가장 먼저 생각하는 이상형이 있으신가요?,20대,DKSR20000890.json
2,전 딱히 이상형이라고,전 딱히 이상형이라고,20대,DKSR20000890.json
3,정해놓은 건 없는데 일단 제가 키가 되게 큰 편이여가지구,정해놓은 건 없는데 일단 제가 키가 되게 큰 편이여가지구,20대,DKSR20000890.json
4,최소한의 마지노선이 나보다만 크면 된다 그치만 그래도 이왕이면은,최소한의 마지노선이 나보다만 크면 된다 그치만 그래도 이왕이면은,20대,DKSR20000890.json
...,...,...,...,...
229492,(()) 자전거 타고,(()) 자전거 타고,20대,DKSR20001736.json
229493,뒤에 내가 타는 거 그런 거 뭐라 하는지 모르겠는데,뒤에 내가 타는 거 그런 거 뭐라 하는지 모르겠는데,20대,DKSR20001736.json
229494,어쨌든 그런 약간 이동 수단을 탔는데,어쨌든 그런 약간 이동 수단을 탔는데,20대,DKSR20001736.json
229495,그거 이제,그거 이제,20대,DKSR20001736.json


In [4]:
df["방언"] = df["방언"].str.replace(r"\(\(\)\)", '', regex=True)
df["표준어"] = df["표준어"].str.replace(r"\(\(\)\)", '', regex=True)

df[df["방언"].str.contains("\(\(\)\)")|df["표준어"].str.contains("\(\(\)\)")]

,방언,표준어,연령대,출처 파일


In [5]:
phrase_words = df["방언"].str.extract(r"\(\((.*)\)\)", expand=False).dropna()
for i in phrase_words.index:
    sentence = df.loc[i, "방언"]
    sentence = re.sub(r"\(\(.*\)\)", phrase_words[i], sentence)
    df.loc[i, "방언"] = sentence

phrase_words = df["표준어"].str.extract(r"\(\((.*)\)\)", expand=False).dropna()
for i in phrase_words.index:
    sentence = df.loc[i, "표준어"]
    sentence = re.sub(r"\(\(.*\)\)", phrase_words[i], sentence)
    df.loc[i, "표준어"] = sentence

df.loc[phrase_words.index]

,방언,표준어,연령대,출처 파일
4126,할아버지 선생님이셨는데 한 키가 백팔십몇인가 하튼 하여튼 그 나이 때 선생님치고는 ...,할아버지 선생님이셨는데 한 키가 백팔십몇인가 하튼 하여튼 그 나이 때 선생님치고는 ...,20대,DKSR20000903.json
4285,그렇게 막 쓰는 거를 잘하지도 못 해가지고 양치기도 못 해 그냥 한 장에서 반 장 ...,그렇게 막 쓰는 거를 잘하지도 못 해가지고 양치기도 못 해 그냥 한 장에서 반 장 ...,20대,DKSR20000903.json
4422,근데 뭔 나이 들어서 인터넷에 후기 보면은 막 지하철이 엄청 더럽고 막 냄새나고 그...,근데 뭔 나이 들어서 인터넷에 후기 보면은 막 지하철이 엄청 더럽고 막 냄새나고 그...,20대,DKSR20000904.json
4452,알 야 우리 여기 가자 하면은 그냥 짐 챙겨서 잘 따라 나가고 그러고,알 야 우리 여기 가자 하면은 그냥 짐 챙겨서 잘 따라 나가고 그러고,20대,DKSR20000904.json
4769,처음에 일이 년째에 전 저도 계속 봤었으니까 그리고 그 남자애랑 제 친구가 전부 다...,처음에 일이 년째에 전 저도 계속 봤었으니까 그리고 그 남자애랑 제 친구가 전부 다...,30대,DKSR20000905.json
...,...,...,...,...
183791,그래가지고 막 서로 목똑기도 하구,그래가지고 막 서로 목똑기도 하구,10대,DKSR20001569.json
185454,따른 대체를 xx해 봤는데,다른 대체를 xx해 봤는데,10대,DKSR20001574.json
199325,모르겠으 모르겠고 좀 무서울 거 같으니까 그냥 시도도 하지 말아야겠다 xxx,모르겠으 모르겠고 좀 무서울 거 같으니까 그냥 시도도 하지 말아야겠다 xxx,20대,DKSR20001621.json
201771,친구들한테 많이 뭐~ -이- 이런 게 좋을 것 같다 저런 게 좋을 것 같다 xx 해...,친구들한테 많이 뭐~ -이- 이런 게 좋을 것 같다 저런 게 좋을 것 같다 xx 해...,20대,DKSR20001631.json


In [6]:
phrase_words = df["방언"].str.extract(r"\(([^\)]+)\)\/\([^\)]+\)", expand=False).dropna()
for i in phrase_words.index:
    sentence = df.loc[i, "방언"]
    group_matches = re.findall(r"\(([^\)]+)\)\/\([^\)]+\)", sentence)
    rep_matches = re.findall(r"\([^\)]+\)\/\([^\)]+\)", sentence)
    for group, rep in zip(group_matches, rep_matches):
        sentence = sentence.replace(rep, group)
    df.loc[i, "방언"] = sentence

phrase_words = df["표준어"].str.extract(r"\([^\)]+\)\/\(([^\)]+)\)", expand=False).dropna()
for i in phrase_words.index:
    sentence = df.loc[i, "표준어"]
    group_matches = re.findall(r"\([^\)]+\)\/\(([^\)]+)\)", sentence)
    rep_matches = re.findall(r"\([^\)]+\)\/\([^\)]+\)", sentence)
    for group, rep in zip(group_matches, rep_matches):
        if group[0] == '#':
            sentence = sentence.replace(rep, '')
        else:
            sentence = sentence.replace(rep, group)
    df.loc[i, "표준어"] = sentence

df.loc[phrase_words.index]

,방언,표준어,연령대,출처 파일


In [7]:
laugh_index = df["방언"].str.contains("\{laughing\}")

df["방언"] = df["방언"].str.replace(r"\{laughing\}", "", regex=True)
df["표준어"] = df["표준어"].str.replace(r"\{laughing\}", "", regex=True)
df[laugh_index]

,방언,표준어,연령대,출처 파일
185,아들 놓으면 너는 평생 후회한다.,아들 낳으면 너는 평생 후회한다.,20대,DKSR20000890.json
274,아~ 신의 탑 이게 기억이 안 나네.,아~ 신의 탑 이게 기억이 안 나네.,30대,DKSR20000891.json
291,중간에 도망갔지 중간에 중간에,중간에 도망갔지 중간에 중간에,30대,DKSR20000891.json
561,저는 백종원이 백종원 티비를 보고 하는 편이라서,저는 백종원이 백종원 티비를 보고 하는 편이라서,30대,DKSR20000892.json
595,안 먹는 걸로 알아주세요.,안 먹는 걸로 알아주세요.,30대,DKSR20000892.json
...,...,...,...,...
227181,그리고 막 일하면서도 뭐~ 쇼핑하고 그냥 뭐~ 누르기만 하면 되니까,그리고 막 일하면서도 뭐~ 쇼핑하고 그냥 뭐~ 누르기만 하면 되니까,40대,DKSR20001729.json
227237,저는 그래서 그날 이후로 이제 엄마한테 쇼핑몰에선 사지 말라고,저는 그래서 그날 이후로 이제 엄마한테 쇼핑몰에선 사지 말라고,20대,DKSR20001729.json
227340,별의별 거를 다 진짜 주문하시드라구요.,별의별 거를 다 진짜 주문하시드라구요.,40대,DKSR20001729.json
227388,네 아무래도 가격이 많이 싸니까,네 아무래도 가격이 많이 싸니까,20대,DKSR20001729.json


In [8]:
stutter_sentence = df["방언"].str.contains(r"\-.*\-", regex=True)
df["방언"] = df["방언"].str.replace(r"\-.*\-", "", regex=True)
df["표준어"] = df["표준어"].str.replace(r"\-.*\-", "", regex=True)

df.loc[stutter_sentence]

,방언,표준어,연령대,출처 파일
57,술 마시고 돈 쓴 거 그래도 한 몇 개월이나 몇 개월이든 일 년이든,술 마시고 돈 쓴 거 그래도 한 몇 개월이나 몇 개월이든 일 년이든,20대,DKSR20000890.json
80,그걸로 인해서 가게 되는 여행지에 대한,그걸로 인해서 가게 되는 여행지에 대한,20대,DKSR20000890.json
108,나중에 점점 아~ 이건 안 했으면 좋겠다라고 이상형이 바뀌셨다고 하셨잖아요.,나중에 점점 아~ 이건 안 했으면 좋겠다라고 이상형이 바뀌셨다고 하셨잖아요.,20대,DKSR20000890.json
122,고런 거 쫌 배려를 해줬으면 좋겠습니다.,고런 거 조금 배려를 해줬으면 좋겠습니다.,20대,DKSR20000890.json
222,영향을 주는 거 같더라고요.,영향을 주는 거 같더라고요.,20대,DKSR20000890.json
...,...,...,...,...
229099,전자사전 혹시 사용해 보신 나이대일까요?,전자사전 혹시 사용해 보신 나이대일까요?,20대,DKSR20001735.json
229278,그래서 제가 더 돈을 많이 쓰나요 요리하고 이런 것도 돈이 많이 드나요?,그래서 제가 더 돈을 많이 쓰나요 요리하고 이런 것도 돈이 많이 드나요?,20대,DKSR20001736.json
229365,말씀 드린 것처럼,말씀 드린 것처럼,20대,DKSR20001736.json
229432,그쵸 아까 여행을 안 가셔가지고 돈이 돈을 모으셨다고 돈이 모아졌다고,그쵸 아까 여행을 안 가셔가지고 돈이 돈을 모으셨다고 돈이 모아졌다고,20대,DKSR20001736.json


### 대체어 처리

In [9]:
대체어_위치_방언 = df["방언"].str.contains(r"\&\S+[0-9]\&", regex=True)
대체어_위치_표준어 = df["표준어"].str.contains(r"\&\S+[0-9]\&", regex=True)

In [10]:
df["방언"] = df["방언"].str.replace(r"\&name[0-9]+\&", "name", regex=True)
df["방언"] = df["방언"].str.replace(r"\&address[0-9]+\&", "address", regex=True)
df["방언"] = df["방언"].str.replace(r"\&company-name[0-9]+\&", "companyname", regex=True)
df["방언"] = df["방언"].str.replace(r"\&companyname[0-9]+\&", "companyname", regex=True)

df[대체어_위치_방언]

,방언,표준어,연령대,출처 파일
15,name 님은,&name1& 님은,20대,DKSR20000890.json
25,name 님은 혹시,&name2& 님은 혹시,20대,DKSR20000890.json
41,혹시 name 님한테 가장 좋았던 연애랑,혹시 &name1& 님한테 가장 좋았던 연애랑,20대,DKSR20000890.json
61,그럼 name 님은 연애관이 어떻게 되시는 가요 뭐~ 네.,그럼 &name2& 님은 연애관이 어떻게 되시는 가요 뭐~ 네.,20대,DKSR20000890.json
63,연애관이 어떤 걸 말하는지 name 님이 먼저 말씀해 주셨으면 좋겠어요.,연애관이 어떤 걸 말하는지 &name1& 님이 먼저 말씀해 주셨으면 좋겠어요.,20대,DKSR20000890.json
...,...,...,...,...
228514,아까 name 님이 말했듯이 약간 사람이 저도 그렇고 받는 사람도 그렇고,아까 &name1& 님이 말했듯이 약간 사람이 저도 그렇고 받는 사람도 그렇고,20대,DKSR20001733.json
228555,어~ name 님은,어~ &name1& 님은,30대,DKSR20001733.json
229395,name 님처럼 옷을 좀 이렇게,&name1& 님처럼 옷을 좀 이렇게,20대,DKSR20001736.json
229405,그래서 아마 name 님도,그래서 아마 &name1& 님도,20대,DKSR20001736.json


In [11]:
df["표준어"] = df["표준어"].str.replace(r"\&name[0-9]\&", "name", regex=True)
df["표준어"] = df["표준어"].str.replace(r"\&address[0-9]\&", "address", regex=True)
df["표준어"] = df["표준어"].str.replace(r"\&company-name[0-9]\&", "companyname", regex=True)
df["표준어"] = df["표준어"].str.replace(r"\&companyname[0-9]\&", "companyname", regex=True)

df[대체어_위치_표준어]

,방언,표준어,연령대,출처 파일
15,name 님은,name 님은,20대,DKSR20000890.json
25,name 님은 혹시,name 님은 혹시,20대,DKSR20000890.json
41,혹시 name 님한테 가장 좋았던 연애랑,혹시 name 님한테 가장 좋았던 연애랑,20대,DKSR20000890.json
61,그럼 name 님은 연애관이 어떻게 되시는 가요 뭐~ 네.,그럼 name 님은 연애관이 어떻게 되시는 가요 뭐~ 네.,20대,DKSR20000890.json
63,연애관이 어떤 걸 말하는지 name 님이 먼저 말씀해 주셨으면 좋겠어요.,연애관이 어떤 걸 말하는지 name 님이 먼저 말씀해 주셨으면 좋겠어요.,20대,DKSR20000890.json
...,...,...,...,...
228514,아까 name 님이 말했듯이 약간 사람이 저도 그렇고 받는 사람도 그렇고,아까 name 님이 말했듯이 약간 사람이 저도 그렇고 받는 사람도 그렇고,20대,DKSR20001733.json
228555,어~ name 님은,어~ name 님은,30대,DKSR20001733.json
229395,name 님처럼 옷을 좀 이렇게,name 님처럼 옷을 좀 이렇게,20대,DKSR20001736.json
229405,그래서 아마 name 님도,그래서 아마 name 님도,20대,DKSR20001736.json


### 데이터 정제

In [12]:
df["길이"] = [min(len(dia.split()), len(stan.split())) for dia, stan in zip(df["방언"], df["표준어"])]
df["길이"]

0         4
1         9
2         3
3         9
4         8
         ..
229492    2
229493    9
229494    6
229495    2
229496    8
Name: 길이, Length: 229497, dtype: int64

In [13]:
suf_len_df = df[df["길이"] > 4]
len(suf_len_df)

179023

In [14]:
dia_sur_len = len(suf_len_df[suf_len_df["방언"] != suf_len_df["표준어"]])
print("길이로 제외한 후 남은 방언 샘플 수: {0}".format(dia_sur_len))

길이로 제외한 후 남은 방언 샘플 수: 28391


In [15]:
suf_len_df.to_csv("../datas/output/gyeongsang_dialect_data_age_processed.csv")

In [48]:
suf_len_df.to_csv("../datas/output/gyeongsang_dialect_test_age_processed.csv")

### 1:1 샘플링 처리

표준어 샘플이 더 많을 때만 수행

In [18]:
dial_df = suf_len_df[suf_len_df["방언"] != suf_len_df["표준어"]]
stan_df = suf_len_df[suf_len_df["방언"] == suf_len_df["표준어"]]

In [19]:
print(len(dial_df))
print(len(stan_df))

305395
1360881


In [20]:
stan_sample = stan_df.sample(len(dial_df))
stan_sample

sampled_df = pd.concat([dial_df, stan_sample])
sampled_df

,방언,표준어,연령대,출처 파일,길이
3,머 못 묵 그라고 모든걸 다 좋아하는 데 즐기진 않는다.,머 못 먹 그리고 모든걸 다 좋아하는 데 즐기진 않는다.,50대,DKCI20000001.json,10
16,근데 먹이가 있어야지 얘네가 먹고 자랄꺼 아이가.,근데 먹이가 있어야지 얘네가 먹고 자랄꺼 아니야.,50대,DKCI20000001.json,7
18,먹이가 될수 없데 근데 머가 얘들의 먹이는 머냐면 식이섬유,먹이가 될수 없데 근데 머가 얘들의 먹이는 무엇이냐면 식이섬유,50대,DKCI20000001.json,9
19,그러니깐 인자 야채 과일 머 식이섬유 머 자기는 그런거 좋아하니깐,그러니깐 인제 야채 과일 머 식이섬유 머 자기는 그런거 좋아하니깐,50대,DKCI20000001.json,10
22,아니 어쩔수 없이 먹는거 빼고 음식점 선 음식점 선택하기도 그렇고 같이 그 머 식사...,아니 어쩔수 없이 먹는거 빼고 음식점 선 음식점 선택하기도 그렇고 같이 그 머 식사...,50대,DKCI20000001.json,15
...,...,...,...,...,...
71887,약간 그게 잘 통하고 이러면 그다음부터,약간 그게 잘 통하고 이러면 그다음부터,20대,DKCI20000304.json,6
519825,수록곡이랑도 진짜 다 좋더라고 근데 나는 개인적으로,수록곡이랑도 진짜 다 좋더라고 근데 나는 개인적으로,10대,DKCI20002615.json,7
1714881,이거는 정말 제가 특수교육하고 있는 제 친구한테도 아이들 다 같이 봤으면 좋겠다고 ...,이거는 정말 제가 특수교육하고 있는 제 친구한테도 아이들 다 같이 봤으면 좋겠다고 ...,30대,DKSR20004924.json,13
177697,근데 또 결혼을 빨리 하면 후회한다고 하잖아 다들,근데 또 결혼을 빨리 하면 후회한다고 하잖아 다들,20대,DKCI20000652.json,8


In [21]:
sampled_df = sampled_df.sample(frac=1).reset_index(drop=True) # 새로 샘플한 dataframe을 셔플
sampled_df

,방언,표준어,연령대,출처 파일,길이
0,아~ 이 노래는 노래는 좋잖아 이런 느낌이에요.,아~ 이 노래는 노래는 좋잖아 이런 느낌이에요.,20대,DKSR20002079.json,7
1,땅이나 이런 나무들이 인자 막 옴을 틔워 갖고,땅이나 이런 나무들이 이제 막 옴을 틔워 갖고,50대,DKCI20002541.json,8
2,어쨌건 아침에 매일 일찍 일어나가주고 새벽 다섯 시 되면 일어나가주고 인라인장에 내...,어쨌건 아침에 매일 일찍 일어나가지고 새벽 다섯 시 되면 일어나가지고 인라인장에 내...,50대,DKCI20003110.json,18
3,생활 보면 다 드릅다 카대 그래가지고,생활 보면 다 더럽다 하대 그래가지고,20대,DKSR20000643.json,6
4,그냥 바르자니 또 각질은 되고,그냥 바르자니 또 각질은 되고,10대,DKSR20003850.json,5
...,...,...,...,...,...
610785,그니까 사람들에게 친구 추천 그~ 중심점은 정직하게 행동하고,그러니까 사람들에게 친구 추천 그~ 중심점은 정직하게 행동하고,20대,DKSR20004969.json,8
610786,암뿐만 아니라 여러 가지 다양한 질병이나 재해 같은 거를 이케 보장받을 수 있는 보...,암뿐만 아니라 여러 가지 다양한 질병이나 재해 같은 거를 이렇게 보장받을 수 있는 ...,30대,DKSR20005257.json,17
610787,말을 엄마를 말린 다음에 다음 날이 됐는데도 엄마가 나한테 사과나 뭐~ 말을 걸거나...,말을 엄마를 말린 다음에 다음 날이 됐는데도 엄마가 나한테 사과나 뭐~ 말을 걸거나...,20대,DKSR20003342.json,17
610788,그~ 사실 이제 학력 쪽으로 컴플렉스를 안 가질려고 했는데,그~ 사실 이제 학력 쪽으로 컴플렉스를 안 가질려고 했는데,30대,DKSR20003768.json,9


In [22]:
sampled_df.to_csv("../datas/output/gyeongsang_dialect_data_age_sampled.csv")

* 경상 방언 필터링 후 훈련 샘플 수: 610790
* 경상 방언 필터링 후 테스트 샘플 수: 134312

### 형태소 토큰화

In [49]:
import MeCab

def pos(sentence: str):
    """한국어 토큰을 분리합니다. 토큰과 품사를 튜플 리스트로 반환합니다.
        
        매개변수:
            sentence (str): 토큰화할 문장 
        반환값:
            token list (list[tuple]): 토큰과 품사 리스트
    """
    t = MeCab.Tagger()
    tag_result = t.parse(sentence)
    tag_result = tag_result.replace("\t", ".@!").replace("\n", ".@!").split(".@!")
    tag_word = tag_result[::2][:-1] # 마지막 EOS는 자른다
    tag_info = tag_result[1::2][:-1] # 마지막 EOS는 자른다
    return [(word, info.split(',')[0]) for word, info in zip(tag_word, tag_info)]

def morphs(sentence: str):
    """한국어 토큰을 분리합니다. 토큰의 리스트를 반환합니다.
        
        매개변수:
            sentence (str): 토큰화할 문장 
        반환값:
            token list (list): 토큰 리스트
    """
    t = MeCab.Tagger()
    tag_result = t.parse(sentence)
    tag_result = tag_result.replace("\t", ".@!").replace("\n", ".@!").split(".@!")
    return tag_result[::2][:-1]

def space_tokenize(original_sentence, token_list):
    """
    문장을 decode할 때 한국어의 띄어쓰기도 되살릴 수 있도록 token에 이를 반영하도록 처리하는 함수
    """
    
    space_token = original_sentence.split()

    if len(space_token) == 0:
        return token_list

    token_idx = 0
    cur_token = space_token[token_idx]
    cum_word = ""
    for i, word in enumerate(token_list):
        cum_word += word

        if len(cum_word) > len(word):
            token_list[i] = "##" + token_list[i]
            
        if cum_word == cur_token:
            token_idx += 1
            cum_word = ""
            cur_token = space_token[min(token_idx, len(space_token) - 1)]
    
    return token_list

def morph_and_preprocess(sentence: str):
    """한국어 토큰을 분리하고 전처리합니다. 토큰의 리스트를 반환합니다.
        
        매개변수:
            sentence (str): 토큰화할 문장 
        반환값:
            token list (list): 토큰 리스트
    """
    
    pos_result = pos(sentence)
    word_list = [word for word, _ in pos_result]
    word_list = space_tokenize(sentence, word_list) # 공백 복원 토큰화
    
    return word_list

In [50]:
df = pd.read_csv("../datas/output/gyeongsang_dialect_data_age_sampled.csv", index_col=0)
df

,방언,표준어,연령대,출처 파일,길이
0,아~ 이 노래는 노래는 좋잖아 이런 느낌이에요.,아~ 이 노래는 노래는 좋잖아 이런 느낌이에요.,20대,DKSR20002079.json,7
1,땅이나 이런 나무들이 인자 막 옴을 틔워 갖고,땅이나 이런 나무들이 이제 막 옴을 틔워 갖고,50대,DKCI20002541.json,8
2,어쨌건 아침에 매일 일찍 일어나가주고 새벽 다섯 시 되면 일어나가주고 인라인장에 내...,어쨌건 아침에 매일 일찍 일어나가지고 새벽 다섯 시 되면 일어나가지고 인라인장에 내...,50대,DKCI20003110.json,18
3,생활 보면 다 드릅다 카대 그래가지고,생활 보면 다 더럽다 하대 그래가지고,20대,DKSR20000643.json,6
4,그냥 바르자니 또 각질은 되고,그냥 바르자니 또 각질은 되고,10대,DKSR20003850.json,5
...,...,...,...,...,...
610785,그니까 사람들에게 친구 추천 그~ 중심점은 정직하게 행동하고,그러니까 사람들에게 친구 추천 그~ 중심점은 정직하게 행동하고,20대,DKSR20004969.json,8
610786,암뿐만 아니라 여러 가지 다양한 질병이나 재해 같은 거를 이케 보장받을 수 있는 보...,암뿐만 아니라 여러 가지 다양한 질병이나 재해 같은 거를 이렇게 보장받을 수 있는 ...,30대,DKSR20005257.json,17
610787,말을 엄마를 말린 다음에 다음 날이 됐는데도 엄마가 나한테 사과나 뭐~ 말을 걸거나...,말을 엄마를 말린 다음에 다음 날이 됐는데도 엄마가 나한테 사과나 뭐~ 말을 걸거나...,20대,DKSR20003342.json,17
610788,그~ 사실 이제 학력 쪽으로 컴플렉스를 안 가질려고 했는데,그~ 사실 이제 학력 쪽으로 컴플렉스를 안 가질려고 했는데,30대,DKSR20003768.json,9


In [51]:
dialect_list = df["방언"].to_list()
dialect_list = list(map(morph_and_preprocess, dialect_list))

standard_list = df["표준어"].to_list()
standard_list = list(map(morph_and_preprocess, standard_list))

In [52]:
list_to_spaced_sentence = lambda li : " ".join(li)
df["방언"] = list(map(list_to_spaced_sentence, dialect_list))
df["표준어"] = list(map(list_to_spaced_sentence, standard_list))

df.head()

,방언,표준어,연령대,출처 파일,길이
0,아 ##~ 이 노래 ##는 노래 ##는 좋 ##잖아 이런 느낌 ##이 ##에요 ##.,아 ##~ 이 노래 ##는 노래 ##는 좋 ##잖아 이런 느낌 ##이 ##에요 ##.,20대,DKSR20002079.json,7
1,땅 ##이나 이런 나무 ##들 ##이 인자 막 옴 ##을 틔워 갖 ##고,땅 ##이나 이런 나무 ##들 ##이 이제 막 옴 ##을 틔워 갖 ##고,50대,DKCI20002541.json,8
2,어쨌건 아침 ##에 매일 일찍 일어나 ##가주 ##고 새벽 다섯 시 되 ##면 일어...,어쨌건 아침 ##에 매일 일찍 일어나 ##가지 ##고 새벽 다섯 시 되 ##면 일어...,50대,DKCI20003110.json,18
3,생활 보 ##면 다 드릅 ##다 카 ##대 그래 ##가지 ##고,생활 보 ##면 다 더럽 ##다 하 ##대 그래 ##가지 ##고,20대,DKSR20000643.json,6
4,그냥 바르 ##자니 또 각질 ##은 되 ##고,그냥 바르 ##자니 또 각질 ##은 되 ##고,10대,DKSR20003850.json,5


In [53]:
df.to_csv("../datas/output/gyeongsang_dialect_data_형태소.csv")

### 테스트셋 형태소

In [54]:
df = pd.read_csv("../datas/output/gyeongsang_dialect_test_age_processed.csv", index_col=0)
df

,방언,표준어,연령대,출처 파일,길이
1,그~ 남자 볼 때 가장 먼저 생각하는 이상형이 있으신가요?,그~ 남자 볼 때 가장 먼저 생각하는 이상형이 있으신가요?,20대,DKSR20000890.json,9
3,정해놓은 건 없는데 일단 제가 키가 되게 큰 편이여가지구,정해놓은 건 없는데 일단 제가 키가 되게 큰 편이여가지구,20대,DKSR20000890.json,9
4,최소한의 마지노선이 나보다만 크면 된다 그치만 그래도 이왕이면은,최소한의 마지노선이 나보다만 크면 된다 그치만 그래도 이왕이면은,20대,DKSR20000890.json,8
5,큰 게 좋잖아요 약간 남들이 볼 때도,큰 게 좋잖아요 약간 남들이 볼 때도,20대,DKSR20000890.json,7
6,이상적인 키라는 게 있으니까 그래서,이상적인 키라는 게 있으니까 그래서,20대,DKSR20000890.json,5
...,...,...,...,...,...
229490,제일 별로였던 거는 그~ 동남아 갔을 때,제일 별로였던 거는 그~ 동남아 갔을 때,20대,DKSR20001736.json,7
229491,그~ 자전거 같은 거 인력거 같은 거?,그~ 자전거 같은 거 인력거 같은 거?,20대,DKSR20001736.json,7
229493,뒤에 내가 타는 거 그런 거 뭐라 하는지 모르겠는데,뒤에 내가 타는 거 그런 거 뭐라 하는지 모르겠는데,20대,DKSR20001736.json,9
229494,어쨌든 그런 약간 이동 수단을 탔는데,어쨌든 그런 약간 이동 수단을 탔는데,20대,DKSR20001736.json,6


In [55]:
dialect_list = df["방언"].to_list()
dialect_list = list(map(morph_and_preprocess, dialect_list))

standard_list = df["표준어"].to_list()
standard_list = list(map(morph_and_preprocess, standard_list))

In [56]:
list_to_spaced_sentence = lambda li : " ".join(li)
df["방언"] = list(map(list_to_spaced_sentence, dialect_list))
df["표준어"] = list(map(list_to_spaced_sentence, standard_list))

df.head()

,방언,표준어,연령대,출처 파일,길이
1,그 ##~ 남자 볼 때 가장 먼저 생각 ##하 ##는 이상형 ##이 있 ##으신 #...,그 ##~ 남자 볼 때 가장 먼저 생각 ##하 ##는 이상형 ##이 있 ##으신 #...,20대,DKSR20000890.json,9
3,정해 ##놓 ##은 건 없 ##는데 일단 제 ##가 키 ##가 되게 큰 편 ##이 ...,정해 ##놓 ##은 건 없 ##는데 일단 제 ##가 키 ##가 되게 큰 편 ##이 ...,20대,DKSR20000890.json,9
4,최소한 ##의 마지노선 ##이 나 ##보다 ##만 크 ##면 된다 그치만 그래도 이...,최소한 ##의 마지노선 ##이 나 ##보다 ##만 크 ##면 된다 그치만 그래도 이...,20대,DKSR20000890.json,8
5,큰 게 좋 ##잖아요 약간 남 ##들 ##이 볼 때 ##도,큰 게 좋 ##잖아요 약간 남 ##들 ##이 볼 때 ##도,20대,DKSR20000890.json,7
6,이상 ##적 ##인 키 ##라는 게 있 ##으니까 그래서,이상 ##적 ##인 키 ##라는 게 있 ##으니까 그래서,20대,DKSR20000890.json,5


In [57]:
df.to_csv("../datas/output/gyeongsang_dialect_test_형태소.csv")

### SentencePiece 토큰화

In [2]:
df = pd.read_csv("../datas/output/gyeongsang_dialect_data_age_sampled.csv", index_col=0)
df

,방언,표준어,연령대,출처 파일,길이
0,아~ 이 노래는 노래는 좋잖아 이런 느낌이에요.,아~ 이 노래는 노래는 좋잖아 이런 느낌이에요.,20대,DKSR20002079.json,7
1,땅이나 이런 나무들이 인자 막 옴을 틔워 갖고,땅이나 이런 나무들이 이제 막 옴을 틔워 갖고,50대,DKCI20002541.json,8
2,어쨌건 아침에 매일 일찍 일어나가주고 새벽 다섯 시 되면 일어나가주고 인라인장에 내...,어쨌건 아침에 매일 일찍 일어나가지고 새벽 다섯 시 되면 일어나가지고 인라인장에 내...,50대,DKCI20003110.json,18
3,생활 보면 다 드릅다 카대 그래가지고,생활 보면 다 더럽다 하대 그래가지고,20대,DKSR20000643.json,6
4,그냥 바르자니 또 각질은 되고,그냥 바르자니 또 각질은 되고,10대,DKSR20003850.json,5
...,...,...,...,...,...
610785,그니까 사람들에게 친구 추천 그~ 중심점은 정직하게 행동하고,그러니까 사람들에게 친구 추천 그~ 중심점은 정직하게 행동하고,20대,DKSR20004969.json,8
610786,암뿐만 아니라 여러 가지 다양한 질병이나 재해 같은 거를 이케 보장받을 수 있는 보...,암뿐만 아니라 여러 가지 다양한 질병이나 재해 같은 거를 이렇게 보장받을 수 있는 ...,30대,DKSR20005257.json,17
610787,말을 엄마를 말린 다음에 다음 날이 됐는데도 엄마가 나한테 사과나 뭐~ 말을 걸거나...,말을 엄마를 말린 다음에 다음 날이 됐는데도 엄마가 나한테 사과나 뭐~ 말을 걸거나...,20대,DKSR20003342.json,17
610788,그~ 사실 이제 학력 쪽으로 컴플렉스를 안 가질려고 했는데,그~ 사실 이제 학력 쪽으로 컴플렉스를 안 가질려고 했는데,30대,DKSR20003768.json,9


In [3]:
df = df.dropna(how='any')
with open('../datas/output/gyeongsang_dialect_data_text.txt', 'w', encoding='utf8') as f:
    f.write('\n'.join(df['방언'] + " " + df["표준어"]))

In [4]:
import sentencepiece as spm

spm.SentencePieceTrainer.train(input='../datas/output/gyeongsang_dialect_data_text.txt', model_prefix='gyeongsang_sp', vocab_size=30000)

In [5]:
sp_tokenizer = spm.SentencePieceProcessor(model_file='gyeongsang_sp.model')

code_result_dialect = sp_tokenizer.encode(df["방언"].to_list(), out_type=str)
code_result_standard = sp_tokenizer.encode(df["표준어"].to_list(), out_type=str)

In [6]:
df["방언"] = [' '.join(res) for res in code_result_dialect]
df["방언"]

0                       ▁아 ~ ▁이 ▁노래는 ▁노래는 ▁좋잖아 ▁이런 ▁느낌이에요 .
1                   ▁땅 이나 ▁이런 ▁나무 들이 ▁인자 ▁막 ▁ 옴 을 ▁ 틔 워 ▁갖고
2         ▁어 쨌 건 ▁아침에 ▁매일 ▁일찍 ▁일어나 가주고 ▁새벽 ▁다섯 ▁시 ▁되면 ▁일...
3                              ▁생활 ▁보면 ▁다 ▁드 릅 다 ▁카대 ▁그래가지고
4                                   ▁그냥 ▁바르 자니 ▁또 ▁각질 은 ▁되고
                                ...                        
610785       ▁그니까 ▁사람들에게 ▁친구 ▁추천 ▁그 ~ ▁중심 점 은 ▁정직하 게 ▁행동 하고
610786    ▁암 뿐만 ▁아니라 ▁여러 ▁가지 ▁다양한 ▁질병 이나 ▁재 해 ▁같은 ▁거를 ▁이...
610787    ▁말을 ▁엄마를 ▁말린 ▁다음에 ▁다음 ▁날이 ▁됐는데 도 ▁엄마가 ▁나한테 ▁사과...
610788           ▁그 ~ ▁사실 ▁이제 ▁학력 ▁쪽으로 ▁컴플렉스를 ▁안 ▁가질려고 ▁했는데
610789    ▁다 ▁다 ▁이제 ▁스트레스 ▁같은 ▁거를 ▁풀 ▁수 ▁있고 ▁기분이 ▁좋아질 ▁거...
Name: 방언, Length: 610790, dtype: object

In [7]:
df["표준어"] = [' '.join(res) for res in code_result_standard]
df["표준어"]

0                       ▁아 ~ ▁이 ▁노래는 ▁노래는 ▁좋잖아 ▁이런 ▁느낌이에요 .
1                   ▁땅 이나 ▁이런 ▁나무 들이 ▁이제 ▁막 ▁ 옴 을 ▁ 틔 워 ▁갖고
2         ▁어 쨌 건 ▁아침에 ▁매일 ▁일찍 ▁일어나가지고 ▁새벽 ▁다섯 ▁시 ▁되면 ▁일어...
3                                ▁생활 ▁보면 ▁다 ▁더럽다 ▁하대 ▁그래가지고
4                                   ▁그냥 ▁바르 자니 ▁또 ▁각질 은 ▁되고
                                ...                        
610785      ▁그러니까 ▁사람들에게 ▁친구 ▁추천 ▁그 ~ ▁중심 점 은 ▁정직하 게 ▁행동 하고
610786    ▁암 뿐만 ▁아니라 ▁여러 ▁가지 ▁다양한 ▁질병 이나 ▁재 해 ▁같은 ▁거를 ▁이...
610787    ▁말을 ▁엄마를 ▁말린 ▁다음에 ▁다음 ▁날이 ▁됐는데 도 ▁엄마가 ▁나한테 ▁사과...
610788           ▁그 ~ ▁사실 ▁이제 ▁학력 ▁쪽으로 ▁컴플렉스를 ▁안 ▁가질려고 ▁했는데
610789    ▁다 ▁다 ▁이제 ▁스트레스 ▁같은 ▁거를 ▁풀 ▁수 ▁있고 ▁기분이 ▁좋아질 ▁거...
Name: 표준어, Length: 610790, dtype: object

In [8]:
df.to_csv("../datas/output/gyeongsang_dialect_data_SentencePiece.csv")

### 테스트셋 SentencePiece

In [9]:
df = pd.read_csv("../datas/output/gyeongsang_dialect_test_age_processed.csv", index_col=0)
df

,방언,표준어,연령대,출처 파일,길이
1,그~ 남자 볼 때 가장 먼저 생각하는 이상형이 있으신가요?,그~ 남자 볼 때 가장 먼저 생각하는 이상형이 있으신가요?,20대,DKSR20000890.json,9
3,정해놓은 건 없는데 일단 제가 키가 되게 큰 편이여가지구,정해놓은 건 없는데 일단 제가 키가 되게 큰 편이여가지구,20대,DKSR20000890.json,9
4,최소한의 마지노선이 나보다만 크면 된다 그치만 그래도 이왕이면은,최소한의 마지노선이 나보다만 크면 된다 그치만 그래도 이왕이면은,20대,DKSR20000890.json,8
5,큰 게 좋잖아요 약간 남들이 볼 때도,큰 게 좋잖아요 약간 남들이 볼 때도,20대,DKSR20000890.json,7
6,이상적인 키라는 게 있으니까 그래서,이상적인 키라는 게 있으니까 그래서,20대,DKSR20000890.json,5
...,...,...,...,...,...
229490,제일 별로였던 거는 그~ 동남아 갔을 때,제일 별로였던 거는 그~ 동남아 갔을 때,20대,DKSR20001736.json,7
229491,그~ 자전거 같은 거 인력거 같은 거?,그~ 자전거 같은 거 인력거 같은 거?,20대,DKSR20001736.json,7
229493,뒤에 내가 타는 거 그런 거 뭐라 하는지 모르겠는데,뒤에 내가 타는 거 그런 거 뭐라 하는지 모르겠는데,20대,DKSR20001736.json,9
229494,어쨌든 그런 약간 이동 수단을 탔는데,어쨌든 그런 약간 이동 수단을 탔는데,20대,DKSR20001736.json,6


In [10]:
df = df.dropna(how='any')
with open('../datas/output/gyeongsang_dialect_test_text.txt', 'w', encoding='utf8') as f:
    f.write('\n'.join(df['방언'] + " " + df["표준어"]))

In [11]:
spm.SentencePieceTrainer.train(input='../datas/output/gyeongsang_dialect_test_text.txt', model_prefix='gyeongsang_sp', vocab_size=30000)

sp_tokenizer = spm.SentencePieceProcessor(model_file='gyeongsang_sp.model')

code_result_dialect = sp_tokenizer.encode(df["방언"].to_list(), out_type=str)
code_result_standard = sp_tokenizer.encode(df["표준어"].to_list(), out_type=str)

In [12]:
df["방언"] = [' '.join(res) for res in code_result_dialect]
df["방언"]

1           ▁그 ~ ▁남자 ▁볼 ▁때 ▁가장 ▁먼저 ▁생각하는 ▁이상형이 ▁있으신가요 ?
3             ▁정해놓 은 ▁건 ▁없는데 ▁일단 ▁제가 ▁키가 ▁되게 ▁큰 ▁편이여가지구
4         ▁최소한 의 ▁마지노선이 ▁나보다 만 ▁크면 ▁된다 ▁그치만 ▁그래도 ▁이왕이면은
5                           ▁큰 ▁게 ▁좋잖아요 ▁약간 ▁남들이 ▁볼 ▁때도
6                            ▁이상 적인 ▁키 라는 ▁게 ▁있으니까 ▁그래서
                              ...                      
229490                  ▁제일 ▁별로 였던 ▁거는 ▁그 ~ ▁동남아 ▁갔을 ▁때
229491                  ▁그 ~ ▁자전거 ▁같은 ▁거 ▁인력 거 ▁같은 ▁거 ?
229493            ▁뒤에 ▁내가 ▁타는 ▁거 ▁그런 ▁거 ▁뭐라 ▁하는지 ▁모르겠는데
229494                     ▁어쨌든 ▁그런 ▁약간 ▁이동 ▁수 단 을 ▁탔는데
229496      ▁그거 ▁타가지고 ▁바가지 를 ▁써가지고 ▁그 ▁싸웠던 ▁게 ▁생각 납니다 .
Name: 방언, Length: 179023, dtype: object

In [13]:
df["표준어"] = [' '.join(res) for res in code_result_standard]
df["표준어"]

1           ▁그 ~ ▁남자 ▁볼 ▁때 ▁가장 ▁먼저 ▁생각하는 ▁이상형이 ▁있으신가요 ?
3             ▁정해놓 은 ▁건 ▁없는데 ▁일단 ▁제가 ▁키가 ▁되게 ▁큰 ▁편이여가지구
4         ▁최소한 의 ▁마지노선이 ▁나보다 만 ▁크면 ▁된다 ▁그치만 ▁그래도 ▁이왕이면은
5                           ▁큰 ▁게 ▁좋잖아요 ▁약간 ▁남들이 ▁볼 ▁때도
6                            ▁이상 적인 ▁키 라는 ▁게 ▁있으니까 ▁그래서
                              ...                      
229490                  ▁제일 ▁별로 였던 ▁거는 ▁그 ~ ▁동남아 ▁갔을 ▁때
229491                  ▁그 ~ ▁자전거 ▁같은 ▁거 ▁인력 거 ▁같은 ▁거 ?
229493            ▁뒤에 ▁내가 ▁타는 ▁거 ▁그런 ▁거 ▁뭐라 ▁하는지 ▁모르겠는데
229494                     ▁어쨌든 ▁그런 ▁약간 ▁이동 ▁수 단 을 ▁탔는데
229496      ▁그거 ▁타가지고 ▁바가지 를 ▁써가지고 ▁그 ▁싸웠던 ▁게 ▁생각 납니다 .
Name: 표준어, Length: 179023, dtype: object

In [14]:
df.to_csv("../datas/output/gyeongsang_dialect_test_SentencePiece.csv")

### 자모 토큰화

In [15]:
df = pd.read_csv("../datas/output/gyeongsang_dialect_data_age_sampled.csv", index_col=0)
df

,방언,표준어,연령대,출처 파일,길이
0,아~ 이 노래는 노래는 좋잖아 이런 느낌이에요.,아~ 이 노래는 노래는 좋잖아 이런 느낌이에요.,20대,DKSR20002079.json,7
1,땅이나 이런 나무들이 인자 막 옴을 틔워 갖고,땅이나 이런 나무들이 이제 막 옴을 틔워 갖고,50대,DKCI20002541.json,8
2,어쨌건 아침에 매일 일찍 일어나가주고 새벽 다섯 시 되면 일어나가주고 인라인장에 내...,어쨌건 아침에 매일 일찍 일어나가지고 새벽 다섯 시 되면 일어나가지고 인라인장에 내...,50대,DKCI20003110.json,18
3,생활 보면 다 드릅다 카대 그래가지고,생활 보면 다 더럽다 하대 그래가지고,20대,DKSR20000643.json,6
4,그냥 바르자니 또 각질은 되고,그냥 바르자니 또 각질은 되고,10대,DKSR20003850.json,5
...,...,...,...,...,...
610785,그니까 사람들에게 친구 추천 그~ 중심점은 정직하게 행동하고,그러니까 사람들에게 친구 추천 그~ 중심점은 정직하게 행동하고,20대,DKSR20004969.json,8
610786,암뿐만 아니라 여러 가지 다양한 질병이나 재해 같은 거를 이케 보장받을 수 있는 보...,암뿐만 아니라 여러 가지 다양한 질병이나 재해 같은 거를 이렇게 보장받을 수 있는 ...,30대,DKSR20005257.json,17
610787,말을 엄마를 말린 다음에 다음 날이 됐는데도 엄마가 나한테 사과나 뭐~ 말을 걸거나...,말을 엄마를 말린 다음에 다음 날이 됐는데도 엄마가 나한테 사과나 뭐~ 말을 걸거나...,20대,DKSR20003342.json,17
610788,그~ 사실 이제 학력 쪽으로 컴플렉스를 안 가질려고 했는데,그~ 사실 이제 학력 쪽으로 컴플렉스를 안 가질려고 했는데,30대,DKSR20003768.json,9


In [16]:
from jamo import h2j, j2hcj # 자모화

def jamo_tokenization(sentence):
    token_list = []

    for c in sentence:
        if c == ' ':
            token_list.append("<SPC>")
        else:
            jamo_str = j2hcj(h2j(c))
            token_list.extend(jamo_str)
            token_list.append("<SEP>")
    
    return token_list

In [17]:
dialect_list = df["방언"].to_list()
dialect_list = list(map(jamo_tokenization, dialect_list))

standard_list = df["표준어"].to_list()
standard_list = list(map(jamo_tokenization, standard_list))

In [18]:
list_to_spaced_sentence = lambda li : " ".join(li)
df.loc[df.index, "방언"] = list(map(list_to_spaced_sentence, dialect_list))
df.loc[df.index, "표준어"] = list(map(list_to_spaced_sentence, standard_list))

df

,방언,표준어,연령대,출처 파일,길이
0,ㅇ ㅏ <SEP> ~ <SEP> <SPC> ㅇ ㅣ <SEP> <SPC> ㄴ ㅗ <S...,ㅇ ㅏ <SEP> ~ <SEP> <SPC> ㅇ ㅣ <SEP> <SPC> ㄴ ㅗ <S...,20대,DKSR20002079.json,7
1,ㄸ ㅏ ㅇ <SEP> ㅇ ㅣ <SEP> ㄴ ㅏ <SEP> <SPC> ㅇ ㅣ <SEP...,ㄸ ㅏ ㅇ <SEP> ㅇ ㅣ <SEP> ㄴ ㅏ <SEP> <SPC> ㅇ ㅣ <SEP...,50대,DKCI20002541.json,8
2,ㅇ ㅓ <SEP> ㅉ ㅐ ㅆ <SEP> ㄱ ㅓ ㄴ <SEP> <SPC> ㅇ ㅏ <S...,ㅇ ㅓ <SEP> ㅉ ㅐ ㅆ <SEP> ㄱ ㅓ ㄴ <SEP> <SPC> ㅇ ㅏ <S...,50대,DKCI20003110.json,18
3,ㅅ ㅐ ㅇ <SEP> ㅎ ㅘ ㄹ <SEP> <SPC> ㅂ ㅗ <SEP> ㅁ ㅕ ㄴ ...,ㅅ ㅐ ㅇ <SEP> ㅎ ㅘ ㄹ <SEP> <SPC> ㅂ ㅗ <SEP> ㅁ ㅕ ㄴ ...,20대,DKSR20000643.json,6
4,ㄱ ㅡ <SEP> ㄴ ㅑ ㅇ <SEP> <SPC> ㅂ ㅏ <SEP> ㄹ ㅡ <SEP...,ㄱ ㅡ <SEP> ㄴ ㅑ ㅇ <SEP> <SPC> ㅂ ㅏ <SEP> ㄹ ㅡ <SEP...,10대,DKSR20003850.json,5
...,...,...,...,...,...
610785,<SPC> ㄱ ㅡ <SEP> ㄴ ㅣ <SEP> ㄲ ㅏ <SEP> <SPC> ㅅ ㅏ ...,<SPC> ㄱ ㅡ <SEP> ㄹ ㅓ <SEP> ㄴ ㅣ <SEP> ㄲ ㅏ <SEP> ...,20대,DKSR20004969.json,8
610786,ㅇ ㅏ ㅁ <SEP> ㅃ ㅜ ㄴ <SEP> ㅁ ㅏ ㄴ <SEP> <SPC> ㅇ ㅏ ...,ㅇ ㅏ ㅁ <SEP> ㅃ ㅜ ㄴ <SEP> ㅁ ㅏ ㄴ <SEP> <SPC> ㅇ ㅏ ...,30대,DKSR20005257.json,17
610787,ㅁ ㅏ ㄹ <SEP> ㅇ ㅡ ㄹ <SEP> <SPC> ㅇ ㅓ ㅁ <SEP> ㅁ ㅏ ...,ㅁ ㅏ ㄹ <SEP> ㅇ ㅡ ㄹ <SEP> <SPC> ㅇ ㅓ ㅁ <SEP> ㅁ ㅏ ...,20대,DKSR20003342.json,17
610788,ㄱ ㅡ <SEP> ~ <SEP> <SPC> ㅅ ㅏ <SEP> ㅅ ㅣ ㄹ <SEP> ...,ㄱ ㅡ <SEP> ~ <SEP> <SPC> ㅅ ㅏ <SEP> ㅅ ㅣ ㄹ <SEP> ...,30대,DKSR20003768.json,9


In [19]:
df.to_csv("../datas/output/gyeongsang_dialect_data_jamo.csv")

### 테스트셋 자모

In [20]:
df = pd.read_csv("../datas/output/gyeongsang_dialect_test_age_processed.csv", index_col=0)
df

,방언,표준어,연령대,출처 파일,길이
1,그~ 남자 볼 때 가장 먼저 생각하는 이상형이 있으신가요?,그~ 남자 볼 때 가장 먼저 생각하는 이상형이 있으신가요?,20대,DKSR20000890.json,9
3,정해놓은 건 없는데 일단 제가 키가 되게 큰 편이여가지구,정해놓은 건 없는데 일단 제가 키가 되게 큰 편이여가지구,20대,DKSR20000890.json,9
4,최소한의 마지노선이 나보다만 크면 된다 그치만 그래도 이왕이면은,최소한의 마지노선이 나보다만 크면 된다 그치만 그래도 이왕이면은,20대,DKSR20000890.json,8
5,큰 게 좋잖아요 약간 남들이 볼 때도,큰 게 좋잖아요 약간 남들이 볼 때도,20대,DKSR20000890.json,7
6,이상적인 키라는 게 있으니까 그래서,이상적인 키라는 게 있으니까 그래서,20대,DKSR20000890.json,5
...,...,...,...,...,...
229490,제일 별로였던 거는 그~ 동남아 갔을 때,제일 별로였던 거는 그~ 동남아 갔을 때,20대,DKSR20001736.json,7
229491,그~ 자전거 같은 거 인력거 같은 거?,그~ 자전거 같은 거 인력거 같은 거?,20대,DKSR20001736.json,7
229493,뒤에 내가 타는 거 그런 거 뭐라 하는지 모르겠는데,뒤에 내가 타는 거 그런 거 뭐라 하는지 모르겠는데,20대,DKSR20001736.json,9
229494,어쨌든 그런 약간 이동 수단을 탔는데,어쨌든 그런 약간 이동 수단을 탔는데,20대,DKSR20001736.json,6


In [21]:
dialect_list = df["방언"].to_list()
dialect_list = list(map(jamo_tokenization, dialect_list))

standard_list = df["표준어"].to_list()
standard_list = list(map(jamo_tokenization, standard_list))

In [22]:
list_to_spaced_sentence = lambda li : " ".join(li)
df.loc[df.index, "방언"] = list(map(list_to_spaced_sentence, dialect_list))
df.loc[df.index, "표준어"] = list(map(list_to_spaced_sentence, standard_list))

df

,방언,표준어,연령대,출처 파일,길이
1,ㄱ ㅡ <SEP> ~ <SEP> <SPC> ㄴ ㅏ ㅁ <SEP> ㅈ ㅏ <SEP> ...,ㄱ ㅡ <SEP> ~ <SEP> <SPC> ㄴ ㅏ ㅁ <SEP> ㅈ ㅏ <SEP> ...,20대,DKSR20000890.json,9
3,ㅈ ㅓ ㅇ <SEP> ㅎ ㅐ <SEP> ㄴ ㅗ ㅎ <SEP> ㅇ ㅡ ㄴ <SEP> ...,ㅈ ㅓ ㅇ <SEP> ㅎ ㅐ <SEP> ㄴ ㅗ ㅎ <SEP> ㅇ ㅡ ㄴ <SEP> ...,20대,DKSR20000890.json,9
4,ㅊ ㅚ <SEP> ㅅ ㅗ <SEP> ㅎ ㅏ ㄴ <SEP> ㅇ ㅢ <SEP> <SPC...,ㅊ ㅚ <SEP> ㅅ ㅗ <SEP> ㅎ ㅏ ㄴ <SEP> ㅇ ㅢ <SEP> <SPC...,20대,DKSR20000890.json,8
5,ㅋ ㅡ ㄴ <SEP> <SPC> ㄱ ㅔ <SEP> <SPC> ㅈ ㅗ ㅎ <SEP> ...,ㅋ ㅡ ㄴ <SEP> <SPC> ㄱ ㅔ <SEP> <SPC> ㅈ ㅗ ㅎ <SEP> ...,20대,DKSR20000890.json,7
6,ㅇ ㅣ <SEP> ㅅ ㅏ ㅇ <SEP> ㅈ ㅓ ㄱ <SEP> ㅇ ㅣ ㄴ <SEP> ...,ㅇ ㅣ <SEP> ㅅ ㅏ ㅇ <SEP> ㅈ ㅓ ㄱ <SEP> ㅇ ㅣ ㄴ <SEP> ...,20대,DKSR20000890.json,5
...,...,...,...,...,...
229490,ㅈ ㅔ <SEP> ㅇ ㅣ ㄹ <SEP> <SPC> ㅂ ㅕ ㄹ <SEP> ㄹ ㅗ <S...,ㅈ ㅔ <SEP> ㅇ ㅣ ㄹ <SEP> <SPC> ㅂ ㅕ ㄹ <SEP> ㄹ ㅗ <S...,20대,DKSR20001736.json,7
229491,ㄱ ㅡ <SEP> ~ <SEP> <SPC> ㅈ ㅏ <SEP> ㅈ ㅓ ㄴ <SEP> ...,ㄱ ㅡ <SEP> ~ <SEP> <SPC> ㅈ ㅏ <SEP> ㅈ ㅓ ㄴ <SEP> ...,20대,DKSR20001736.json,7
229493,ㄷ ㅟ <SEP> ㅇ ㅔ <SEP> <SPC> ㄴ ㅐ <SEP> ㄱ ㅏ <SEP> ...,ㄷ ㅟ <SEP> ㅇ ㅔ <SEP> <SPC> ㄴ ㅐ <SEP> ㄱ ㅏ <SEP> ...,20대,DKSR20001736.json,9
229494,ㅇ ㅓ <SEP> ㅉ ㅐ ㅆ <SEP> ㄷ ㅡ ㄴ <SEP> <SPC> ㄱ ㅡ <S...,ㅇ ㅓ <SEP> ㅉ ㅐ ㅆ <SEP> ㄷ ㅡ ㄴ <SEP> <SPC> ㄱ ㅡ <S...,20대,DKSR20001736.json,6


In [23]:
df.to_csv("../datas/output/gyeongsang_dialect_test_jamo.csv")

### Byte 수준 BPE

In [2]:
df = pd.read_csv("../datas/output/gyeongsang_dialect_data_age_sampled.csv", index_col=0)
df

,방언,표준어,연령대,출처 파일,길이
0,아~ 이 노래는 노래는 좋잖아 이런 느낌이에요.,아~ 이 노래는 노래는 좋잖아 이런 느낌이에요.,20대,DKSR20002079.json,7
1,땅이나 이런 나무들이 인자 막 옴을 틔워 갖고,땅이나 이런 나무들이 이제 막 옴을 틔워 갖고,50대,DKCI20002541.json,8
2,어쨌건 아침에 매일 일찍 일어나가주고 새벽 다섯 시 되면 일어나가주고 인라인장에 내...,어쨌건 아침에 매일 일찍 일어나가지고 새벽 다섯 시 되면 일어나가지고 인라인장에 내...,50대,DKCI20003110.json,18
3,생활 보면 다 드릅다 카대 그래가지고,생활 보면 다 더럽다 하대 그래가지고,20대,DKSR20000643.json,6
4,그냥 바르자니 또 각질은 되고,그냥 바르자니 또 각질은 되고,10대,DKSR20003850.json,5
...,...,...,...,...,...
610785,그니까 사람들에게 친구 추천 그~ 중심점은 정직하게 행동하고,그러니까 사람들에게 친구 추천 그~ 중심점은 정직하게 행동하고,20대,DKSR20004969.json,8
610786,암뿐만 아니라 여러 가지 다양한 질병이나 재해 같은 거를 이케 보장받을 수 있는 보...,암뿐만 아니라 여러 가지 다양한 질병이나 재해 같은 거를 이렇게 보장받을 수 있는 ...,30대,DKSR20005257.json,17
610787,말을 엄마를 말린 다음에 다음 날이 됐는데도 엄마가 나한테 사과나 뭐~ 말을 걸거나...,말을 엄마를 말린 다음에 다음 날이 됐는데도 엄마가 나한테 사과나 뭐~ 말을 걸거나...,20대,DKSR20003342.json,17
610788,그~ 사실 이제 학력 쪽으로 컴플렉스를 안 가질려고 했는데,그~ 사실 이제 학력 쪽으로 컴플렉스를 안 가질려고 했는데,30대,DKSR20003768.json,9


In [3]:
from tokenizers import ByteLevelBPETokenizer

bpe_tokenizer = ByteLevelBPETokenizer()
bpe_tokenizer.train(
    files="../datas/output/gyeongsang_dialect_data_text.txt",
)

In [4]:
code_result_dialect = bpe_tokenizer.encode_batch(df["방언"])
code_result_standard = bpe_tokenizer.encode_batch(df["표준어"])

In [5]:
df.loc[df.index, "방언"] = [' '.join(res.tokens) for res in code_result_dialect]
df["방언"]

0         ìķĦ ~ ĠìĿ´ Ġëħ¸ëŀĺëĬĶ Ġëħ¸ëŀĺëĬĶ Ġì¢ĭìŀĸìķĦ Ġì...
1         ëķħ ìĿ´ëĤĺ ĠìĿ´ëŁ° ĠëĤĺë¬´ë ĵ¤ìĿ´ ĠìĿ¸ìŀĲ Ġë§ī...
2         ìĸ´ì ¨Į ê±´ ĠìķĦì¹¨ìĹĲ Ġë§¤ìĿ¼ ĠìĿ¼ì°į ĠìĿ¼ìĸ´...
3         ìĥĿíĻľ Ġë³´ë©´ Ġëĭ¤ Ġëĵľ ë¦ ħ ëĭ¤ Ġì¹´ëĮĢ Ġê·¸...
4           ê·¸ëĥ¥ Ġë°Ķë¥´ì ŀĲ ëĭĪ ĠëĺĲ Ġê°ģ ì§ĪìĿĢ ĠëĲĺê³ł
                                ...                        
610785    Ġê·¸ëĭĪê¹Į ĠìĤ¬ëŀĮëĵ¤ìĹĲê²Į Ġì¹ľêµ¬ Ġì¶Ķì²ľ Ġê...
610786    ìķĶ ë¿Ĳë§Į ĠìķĦëĭĪëĿ¼ ĠìĹ¬ëŁ¬ Ġê°Ģì§Ģ Ġëĭ¤ìĸĳí...
610787    ë§ĲìĿĦ ĠìĹĦë§Īë¥¼ Ġë§Ĳë¦° Ġëĭ¤ìĿĮìĹĲ Ġëĭ¤ìĿĮ Ġ...
610788    ê·¸ ~ ĠìĤ¬ìĭ¤ ĠìĿ´ìłľ ĠíķĻ ëł¥ Ġìª½ìľ¼ë¡ľ Ġì»´...
610789    ëĭ¤ Ġëĭ¤ ĠìĿ´ìłľ ĠìĬ¤íĬ¸ëłĪìĬ¤ Ġê°ĻìĿĢ Ġê±°ë¥¼...
Name: 방언, Length: 610790, dtype: object

In [6]:
df.loc[df.index, "표준어"] = [' '.join(res.tokens) for res in code_result_standard]
df["표준어"]

0         ìķĦ ~ ĠìĿ´ Ġëħ¸ëŀĺëĬĶ Ġëħ¸ëŀĺëĬĶ Ġì¢ĭìŀĸìķĦ Ġì...
1         ëķħ ìĿ´ëĤĺ ĠìĿ´ëŁ° ĠëĤĺë¬´ë ĵ¤ìĿ´ ĠìĿ´ìłľ Ġë§ī...
2         ìĸ´ì ¨Į ê±´ ĠìķĦì¹¨ìĹĲ Ġë§¤ìĿ¼ ĠìĿ¼ì°į ĠìĿ¼ìĸ´...
3         ìĥĿíĻľ Ġë³´ë©´ Ġëĭ¤ ĠëįĶëŁ½ ëĭ¤ ĠíķĺëĮĢ Ġê·¸ëŀ...
4           ê·¸ëĥ¥ Ġë°Ķë¥´ì ŀĲ ëĭĪ ĠëĺĲ Ġê°ģ ì§ĪìĿĢ ĠëĲĺê³ł
                                ...                        
610785    Ġê·¸ëŁ¬ëĭĪê¹Į ĠìĤ¬ëŀĮëĵ¤ìĹĲê²Į Ġì¹ľêµ¬ Ġì¶Ķì²ľ...
610786    ìķĶ ë¿Ĳë§Į ĠìķĦëĭĪëĿ¼ ĠìĹ¬ëŁ¬ Ġê°Ģì§Ģ Ġëĭ¤ìĸĳí...
610787    ë§ĲìĿĦ ĠìĹĦë§Īë¥¼ Ġë§Ĳë¦° Ġëĭ¤ìĿĮìĹĲ Ġëĭ¤ìĿĮ Ġ...
610788    ê·¸ ~ ĠìĤ¬ìĭ¤ ĠìĿ´ìłľ ĠíķĻ ëł¥ Ġìª½ìľ¼ë¡ľ Ġì»´...
610789    ëĭ¤ Ġëĭ¤ ĠìĿ´ìłľ ĠìĬ¤íĬ¸ëłĪìĬ¤ Ġê°ĻìĿĢ Ġê±°ë¥¼...
Name: 표준어, Length: 610790, dtype: object

In [7]:
df.to_csv("../datas/output/gyeongsang_dialect_data_bpe.csv")

### 테스트셋 BPE

In [8]:
df = pd.read_csv("../datas/output/gyeongsang_dialect_test_age_processed.csv", index_col=0)
df

,방언,표준어,연령대,출처 파일,길이
1,그~ 남자 볼 때 가장 먼저 생각하는 이상형이 있으신가요?,그~ 남자 볼 때 가장 먼저 생각하는 이상형이 있으신가요?,20대,DKSR20000890.json,9
3,정해놓은 건 없는데 일단 제가 키가 되게 큰 편이여가지구,정해놓은 건 없는데 일단 제가 키가 되게 큰 편이여가지구,20대,DKSR20000890.json,9
4,최소한의 마지노선이 나보다만 크면 된다 그치만 그래도 이왕이면은,최소한의 마지노선이 나보다만 크면 된다 그치만 그래도 이왕이면은,20대,DKSR20000890.json,8
5,큰 게 좋잖아요 약간 남들이 볼 때도,큰 게 좋잖아요 약간 남들이 볼 때도,20대,DKSR20000890.json,7
6,이상적인 키라는 게 있으니까 그래서,이상적인 키라는 게 있으니까 그래서,20대,DKSR20000890.json,5
...,...,...,...,...,...
229490,제일 별로였던 거는 그~ 동남아 갔을 때,제일 별로였던 거는 그~ 동남아 갔을 때,20대,DKSR20001736.json,7
229491,그~ 자전거 같은 거 인력거 같은 거?,그~ 자전거 같은 거 인력거 같은 거?,20대,DKSR20001736.json,7
229493,뒤에 내가 타는 거 그런 거 뭐라 하는지 모르겠는데,뒤에 내가 타는 거 그런 거 뭐라 하는지 모르겠는데,20대,DKSR20001736.json,9
229494,어쨌든 그런 약간 이동 수단을 탔는데,어쨌든 그런 약간 이동 수단을 탔는데,20대,DKSR20001736.json,6


In [9]:
from tokenizers import ByteLevelBPETokenizer

bpe_tokenizer = ByteLevelBPETokenizer()
bpe_tokenizer.train(
    files="../datas/output/gyeongsang_dialect_test_text.txt",
)

bpe_tokenizer.save_model("./")

['./vocab.json', './merges.txt']

In [10]:
code_result_dialect = bpe_tokenizer.encode_batch(df["방언"])
code_result_standard = bpe_tokenizer.encode_batch(df["표준어"])

In [11]:
df.loc[df.index, "방언"] = [' '.join(res.tokens) for res in code_result_dialect]
df["방언"]

1         ê·¸ ~ ĠëĤ¨ìŀĲ Ġë³¼ ĠëķĮ Ġê°Ģìŀ¥ Ġë¨¼ìłĢ ĠìĥĿê°...
3         ìłķ íķ´ëĨĵìĿĢ Ġê±´ ĠìĹĨëĬĶëį° ĠìĿ¼ëĭ¨ Ġìłľê°Ģ ...
4         ìµľ ìĨĮíķľ ìĿĺ Ġë§Īì§Ģëħ¸ ìĦłìĿ´ ĠëĤĺë³´ëĭ¤ ë§...
5         íģ° Ġê²Į Ġì¢ĭìŀĸìķĦìļĶ Ġìķ½ê°Ħ ĠëĤ¨ëĵ¤ìĿ´ Ġë³¼...
6         ìĿ´ìĥģ ìłģìĿ¸ ĠíĤ¤ ëĿ¼ëĬĶ Ġê²Į ĠìŀĪìľ¼ëĭĪê¹Į Ġ...
                                ...                        
229490    ìłľìĿ¼ Ġë³Ħë¡ľìĺĢëįĺ Ġê±°ëĬĶ Ġê·¸ ~ ĠëıĻëĤ¨ìķĦ...
229491    ê·¸ ~ ĠìŀĲìłĦê±° Ġê°ĻìĿĢ Ġê±° ĠìĿ¸ ëł¥ ê±° Ġê°...
229493    ëĴ¤ìĹĲ ĠëĤ´ê°Ģ ĠíĥĢëĬĶ Ġê±° Ġê·¸ëŁ° Ġê±° ĠëŃĲë...
229494    ìĸ´ì¨Įëĵł Ġê·¸ëŁ° Ġìķ½ê°Ħ ĠìĿ´ëıĻ ĠìĪĺ ëĭ¨ìĿĦ ...
229496    ê·¸ê±° ĠíĥĢê°Ģì§Ģê³ł Ġë°Ķê°Ģì§Ģë¥¼ Ġìį¨ê°Ģì§Ģê...
Name: 방언, Length: 179023, dtype: object

In [12]:
df.loc[df.index, "표준어"] = [' '.join(res.tokens) for res in code_result_standard]
df["표준어"]

1         ê·¸ ~ ĠëĤ¨ìŀĲ Ġë³¼ ĠëķĮ Ġê°Ģìŀ¥ Ġë¨¼ìłĢ ĠìĥĿê°...
3         ìłķ íķ´ëĨĵìĿĢ Ġê±´ ĠìĹĨëĬĶëį° ĠìĿ¼ëĭ¨ Ġìłľê°Ģ ...
4         ìµľ ìĨĮíķľ ìĿĺ Ġë§Īì§Ģëħ¸ ìĦłìĿ´ ĠëĤĺë³´ëĭ¤ ë§...
5         íģ° Ġê²Į Ġì¢ĭìŀĸìķĦìļĶ Ġìķ½ê°Ħ ĠëĤ¨ëĵ¤ìĿ´ Ġë³¼...
6         ìĿ´ìĥģ ìłģìĿ¸ ĠíĤ¤ ëĿ¼ëĬĶ Ġê²Į ĠìŀĪìľ¼ëĭĪê¹Į Ġ...
                                ...                        
229490    ìłľìĿ¼ Ġë³Ħë¡ľìĺĢëįĺ Ġê±°ëĬĶ Ġê·¸ ~ ĠëıĻëĤ¨ìķĦ...
229491    ê·¸ ~ ĠìŀĲìłĦê±° Ġê°ĻìĿĢ Ġê±° ĠìĿ¸ ëł¥ ê±° Ġê°...
229493    ëĴ¤ìĹĲ ĠëĤ´ê°Ģ ĠíĥĢëĬĶ Ġê±° Ġê·¸ëŁ° Ġê±° ĠëŃĲë...
229494    ìĸ´ì¨Įëĵł Ġê·¸ëŁ° Ġìķ½ê°Ħ ĠìĿ´ëıĻ ĠìĪĺ ëĭ¨ìĿĦ ...
229496    ê·¸ê±° ĠíĥĢê°Ģì§Ģê³ł Ġë°Ķê°Ģì§Ģë¥¼ Ġìį¨ê°Ģì§Ģê...
Name: 표준어, Length: 179023, dtype: object

In [13]:
df.to_csv("../datas/output/gyeongsang_dialect_test_bpe.csv")

## 충청 방언

In [14]:
df = pd.read_csv("../datas/output/chungcheong_dialect_train_age.csv", index_col=0)
df

,방언,표준어,연령대,출처 파일
0,네 요즘 프로 레슬링 유튜브 관련 채무너,네 요즘 프로 레슬링 유튜브 관련 채무너,20대,DCDG20000001.json
1,채널을 많이 보고 있는데 응 거기에서 많이 나오는 선수 자체가,채널을 많이 보고 있는데 응 거기에서 많이 나오는 선수 자체가,20대,DCDG20000001.json
2,그 드류 맥킨타이어 그 선수에 대해서 많이 나오더라고,그 드류 맥킨타이어 그 선수에 대해서 많이 나오더라고,20대,DCDG20000001.json
3,그 사람이 알고 보니까 십 년 전에는,그 사람이 알고 보니까 십 년 전에는,20대,DCDG20000001.json
4,그 자버라고 대개 선수들 막 승리 많이 하게 해 주는,그 자버라고 대개 선수들 막 승리 많이 하게 해 주는,20대,DCDG20000001.json
...,...,...,...,...
1848450,부동산도 그리쿠 앞으로 인제 코로나가 어트게 될진 모르겠지만 앞으로 방향이,부동산도 그렇구 앞으로 인제 코로나가 어트게 될진 모르겠지만 앞으로 방향이,50대,DCNA20002450.json
1848451,코로나로 인해서 확 바껴 가지고 앞으로 우리 인제,코로나로 인해서 확 바뀌어 가지고 앞으로 우리 인제,50대,DCNA20002450.json
1848452,문화가 틀려져 문화가 문화도 틀려지고 이제 지금까지 살았던 패턴이 완전히 틀려져 가...,문화가 틀려져 문화가 문화도 틀려지고 이제 지금까지 살았던 패턴이 완전히 틀려져 가...,50대,DCNA20002450.json
1848453,그 변화에도 우리는 지금 이제 따라가야 돼 앞으로,그 변화에도 우리는 지금 이제 따라가야 돼 앞으로,50대,DCNA20002450.json


In [27]:
# 테스트 데이터로 동일한 과정을 진행하고 싶을 때
df = pd.read_csv("../datas/output/chungcheong_dialect_test_age.csv", index_col=0)
df

,방언,표준어,연령대,출처 파일
0,우리집 거북이 용봉이가 지금 십팔 년에서 십구 년차 되잖아.,우리집 거북이 용봉이가 지금 십팔 년에서 십구 년차 되잖아.,50대,DCDG20000009.json
1,근데 용봉이가 중간에,근데 용봉이가 중간에,50대,DCDG20000009.json
2,어~ 아파서 고비가 있었는데 지금은 그걸 잘 극복해 가지고 건강하게 우리 식구랑 같...,어~ 아파서 고비가 있었는데 지금은 그걸 잘 극복해 가지고 건강하게 우리 식구랑 같...,50대,DCDG20000009.json
3,근데 엄마가 너무 용봉이를 예뻐한다고 니가 엄청 질투가 많은데,근데 엄마가 너무 용봉이를 예뻐한다고 니가 엄청 질투가 많은데,50대,DCDG20000009.json
4,그래도 용봉이가 가족 일원으로서 예쁘지?,그래도 용봉이가 가족 일원으로서 예쁘지?,50대,DCDG20000009.json
...,...,...,...,...
227998,그~ 선조들이 했던 그 말씀들이 새록새록 와닿아요.,그~ 선조들이 했던 그 말씀들이 새록새록 와닿아요.,50대,DCNA20000036.json
227999,인제 철들자 잉~ 저기 한다고 쪼꼼 철이 드는지 어쩐지,인제 철들자 잉~ 저기 한다고 조금 철이 드는지 어쩐지,50대,DCNA20000036.json
228000,그런 감이 좀 있어요.,그런 감이 좀 있어요.,50대,DCNA20000036.json
228001,어떻게 보면 선생님 지금 그게 -오- 오히려 인제 마음이 여유로워 졌기 때문에,어떻게 보면 선생님 지금 그게 -오- 오히려 인제 마음이 여유로워 졌기 때문에,50대,DCNA20000036.json


In [28]:
df["방언"] = df["방언"].str.replace(r"\(\(\)\)", '', regex=True)
df["표준어"] = df["표준어"].str.replace(r"\(\(\)\)", '', regex=True)

In [29]:
phrase_words = df["방언"].str.extract(r"\(\((.*)\)\)", expand=False).dropna()
for i in phrase_words.index:
    sentence = df.loc[i, "방언"]
    sentence = re.sub(r"\(\(.*\)\)", phrase_words[i], sentence)
    df.loc[i, "방언"] = sentence

phrase_words = df["표준어"].str.extract(r"\(\((.*)\)\)", expand=False).dropna()
for i in phrase_words.index:
    sentence = df.loc[i, "표준어"]
    sentence = re.sub(r"\(\(.*\)\)", phrase_words[i], sentence)
    df.loc[i, "표준어"] = sentence

df.loc[phrase_words.index]

,방언,표준어,연령대,출처 파일
421,까 한 편 집중해서 보기 시작하면 한 두 시간쯤에는 결론이 나져 있는 거지.,까 한 편 집중해서 보기 시작하면 한 두 시간쯤에는 결론이 나져 있는 거지.,50대,DCDG20000010.json
436,떤 기억력 면에서도 그렇고,떤 기억력 면에서도 그렇고,50대,DCDG20000010.json
447,데 내가 좋아하는 배우도 있을 수 있으니까,데 내가 좋아하는 배우도 있을 수 있으니까,50대,DCDG20000010.json
861,이까 뇌의 그냥 시냅스적인 신호라는 거지 뇌와 뇌세포와의 간에 연결에서,이까 뇌의 그냥 시냅스적인 신호라는 거지 뇌와 뇌세포와의 간에 연결에서,20대,DCDG20000012.json
863,예를 들어서 그 공각기동대에서도 나오잖아 아무리 피를 흘리거도,예를 들어서 그 공각기동대에서도 나오잖아 아무리 피를 흘리거도,20대,DCDG20000012.json
...,...,...,...,...
195333,이끼구 있는데 제가 그러다 보니까 로망이,이끼구 있는데 제가 그러다 보니까 로망이,40대,DCEX20020150.json
209997,그니깐 더 이중적으로 느껴지는 게 근데 그때 기억나?,그러니깐 더 이중적으로 느껴지는 게 근데 그때 기억나?,20대,DCEX20030110.json
215322,제가 먹었던 국밥 중에서 제일 맛있어요 XXX 먹는 게.,제가 먹었던 국밥 중에서 제일 맛있어요 XXX 먹는 게.,30대,DCNA20000009.json
224531,거기 이름이 진짜 저도 지금 생각이 안 나는데,거기 이름이 진짜 저도 지금 생각이 안 나는데,30대,DCNA20000030.json


In [30]:
phrase_words = df["방언"].str.extract(r"\(([^\)]+)\)\/\([^\)]+\)", expand=False).dropna()
for i in phrase_words.index:
    sentence = df.loc[i, "방언"]
    group_matches = re.findall(r"\(([^\)]+)\)\/\([^\)]+\)", sentence)
    rep_matches = re.findall(r"\([^\)]+\)\/\([^\)]+\)", sentence)
    for group, rep in zip(group_matches, rep_matches):
        sentence = sentence.replace(rep, group)
    df.loc[i, "방언"] = sentence

phrase_words = df["표준어"].str.extract(r"\([^\)]+\)\/\(([^\)]+)\)", expand=False).dropna()
for i in phrase_words.index:
    sentence = df.loc[i, "표준어"]
    group_matches = re.findall(r"\([^\)]+\)\/\(([^\)]+)\)", sentence)
    rep_matches = re.findall(r"\([^\)]+\)\/\([^\)]+\)", sentence)
    for group, rep in zip(group_matches, rep_matches):
        if group[0] == '#':
            sentence = sentence.replace(rep, '')
        else:
            sentence = sentence.replace(rep, group)
    df.loc[i, "표준어"] = sentence

df.loc[phrase_words.index]

,방언,표준어,연령대,출처 파일
117312,거긴 동해안 쪽여 그래서 나는 그런 것도 있고 그런데 서해안은 절대 안마 안 좋아.,거긴 동해안 쪽이야 그래서 나는 그런 것도 있고 그런데 서해안은 절대 안마 안 좋아.,60대 이상,DCEX20002087.json
117316,이짝 서해안 쪽으로는 잘 모르고 그른데 이 짝 으로는 내 아는 놈 후배가 있어갖고 ...,이쪽 서해안 쪽으로는 잘 모르고 그른데 이 쪽 으로는 내 아는 놈 후배가 있어갖고 ...,60대 이상,DCEX20002087.json
117410,베트남은 갔드니 먹는 거는 아주 헛일여 먹는 건 물도 안 먹고 반찬도 하나도 없고,베트남은 갔더니 먹는 거는 아주 헛일야 먹는 건 물도 안 먹고 반찬도 하나도 없고,60대 이상,DCEX20002087.json
117465,근데 왜 댕긴는가 몰라 집에 있는기 편해긴 편했는데 아들때메 댕깃지 아들때민에 댕기고,근데 왜 다녔는가는 몰라 집에 있는게 편하긴 편했는데 아들때문에 때문에 아들때문에 다니고,60대 이상,DCEX20002087.json
117853,착화감도 괜찮고 가죽질도 나쁘지 않고 그리고 밑창도 이렇게 통으로 고무로 되어 있어...,착화감도 괜찮고 가죽질도 나쁘지 않고 그리고 밑창도 이렇게 통으로 고무로 되어 있어...,30대,DCEX20002089.json
118119,아직 큰 언니가 뭐~ 우리 뭐~ 우리집으로 뭐~ 몇 시까정 와라 뭐~ 뭐~ 한다고 ...,아직 큰 언니가 뭐~ 우리 뭐~ 우리집으로 뭐~ 몇 시까지 와라 뭐~ 뭐~ 한다고 ...,50대,DCEX20002091.json
118253,부모님 생일도 자기인저 혼자 해줘야 되고 어버이의 날도 혼자 해야 되니까 부담이 있...,부모님 생일도 자기인제 혼자 해줘야 되고 어버이의 날도 혼자 해야 되니까 부담이 있...,50대,DCEX20002091.json
119032,우리나라는 왜냐하면 어~국지적으로 지금 굉장히 그 뭐라 해야 댜~쫌 섬처럼 이렇게 ...,우리나라는 왜냐하면 어~국지적으로 지금 굉장히 그 뭐라 해야 댜~조금 섬처럼 이렇게...,20대,DCEX20002117.json
119123,안맞지 유럽인데 못살고 유럽인디 이슬람 국가다 그래서 갔는데 정말 음 정말 못 살더라고,안맞지 유럽인데 못살고 유럽인데 이슬람 국가다 그래서 갔는데 정말 음 정말 못 살더라고,20대,DCEX20002117.json
119536,쪼끔 잘 에~마치야 되지 않을-긍까 조금 잘 맞는 편이지 않을까 라는 생각을 좀 많...,조금 잘 에~맞춰야 되지 않을-그러니까 조금 잘 맞는 편이지 않을까 라는 생각을 좀...,20대,DCEX20002121.json


In [31]:
laugh_index = df["방언"].str.contains("\{laughing\}")

df["방언"] = df["방언"].str.replace(r"\{laughing\}", "", regex=True)
df["표준어"] = df["표준어"].str.replace(r"\{laughing\}", "", regex=True)
df[laugh_index]

,방언,표준어,연령대,출처 파일
1201,운전면허는 뭐 있지만 이거는 없다고 봐야 되는 거고,운전면허는 뭐 있지만 이거는 없다고 봐야 되는 거고,20대,DCDG20000013.json
1202,내가 너보다 운전을 못할 거야 아마,내가 너보다 운전을 못할 거야 아마,20대,DCDG20000013.json
1315,그리고 뭐 되게 강 당장 중요했던 그런 게 아니니까 그냥 생각만 하고 말았지 이번에...,그리고 뭐 되게 강 당장 중요했던 그런 게 아니니까 그냥 생각만 하고 말았지 이번에...,20대,DCDG20000013.json
1553,어~ 용인 딱히 용인은 그게 없는 거 같애.,어~ 용인 딱히 용인은 그게 없는 거 같애.,30대,DCDG20000014.json
1915,까먹고 있었네,까먹고 있었네,10대,DCDG20000016.json
...,...,...,...,...
224490,또 어디 국내로도 가고 싶은데 생각해 봤어요?,또 어디 국내로도 가고 싶은데 생각해 봤어요?,30대,DCNA20000030.json
226315,난 그쪽엔 포기했잖아 그래서,난 그쪽엔 포기했잖아 그래서,50대,DCNA20000033.json
227684,그냥 쌩쇼 같은 그런 응 그렇게 했어요 그래서 좀 재밌더라고.,그냥 쌩쇼 같은 그런 응 그렇게 했어요 그래서 좀 재밌더라고.,50대,DCNA20000036.json
227711,아들만 둘이어서 선생님,아들만 둘이어서 선생님,50대,DCNA20000036.json


In [32]:
stutter_sentence = df["방언"].str.contains(r"\-.*\-", regex=True)
df["방언"] = df["방언"].str.replace(r"\-.*\-", "", regex=True)
df["표준어"] = df["표준어"].str.replace(r"\-.*\-", "", regex=True)

df.loc[stutter_sentence]

,방언,표준어,연령대,출처 파일
21,그다음부터는 용봉이도 자기 힘들 때 돌봐준 엄마를 기억를 하는지,그다음부터는 용봉이도 자기 힘들 때 돌봐준 엄마를 기억를 하는지,50대,DCDG20000009.json
58,어디가 불편해 보여서 그때 그때 응급처치를 하면,어디가 불편해 보여서 그때 그때 응급처치를 하면,50대,DCDG20000009.json
175,어떤 행동 꼼지의 어떤 행동?,어떤 행동 꼼지의 어떤 행동?,50대,DCDG20000009.json
211,그런 애한테 집을 넓히는 거는 그렇게 효율적이지 않은 거 같애.,그런 애한테 집을 넓히는 거는 그렇게 효율적이지 않은 거 같애.,50대,DCDG20000009.json
362,너는 재미없었 다 던 작품도 기억을 다 하고 있더라고,너는 재미없었 다 던 작품도 기억을 다 하고 있더라고,50대,DCDG20000010.json
...,...,...,...,...
227792,어~ 그~ 사계절 그 특색이 있는 것 같애요.,어~ 그~ 사계절 그 특색이 있는 것 같애요.,50대,DCNA20000036.json
227821,더욱 어~ 그러한 또 추억도 있어요 어~ 그러한 추억,더욱 어~ 그러한 또 추억도 있어요 어~ 그러한 추억,50대,DCNA20000036.json
227882,저희는 선생님 아직 이불 못 버리고 있는 상황이 왜 오냐면은,저희는 선생님 아직 이불 못 버리고 있는 상황이 왜 오냐면은,50대,DCNA20000036.json
227949,아침 일찍 이~ 저~ 일요일날 일곱 시 새벽에 일어나서 캤어요.,아침 일찍 이~ 저~ 일요일날 일곱 시 새벽에 일어나서 캤어요.,50대,DCNA20000036.json


### 대체어 처리

In [33]:
대체어_위치_방언 = df["방언"].str.contains(r"\&\S+[0-9]\&", regex=True)
대체어_위치_표준어 = df["표준어"].str.contains(r"\&\S+[0-9]\&", regex=True)

In [34]:
df["방언"] = df["방언"].str.replace(r"\&name[0-9]+\&", "name", regex=True)
df["방언"] = df["방언"].str.replace(r"\&address[0-9]+\&", "address", regex=True)
df["방언"] = df["방언"].str.replace(r"\&company-name[0-9]+\&", "companyname", regex=True)
df["방언"] = df["방언"].str.replace(r"\&companyname[0-9]+\&", "companyname", regex=True)

df[대체어_위치_방언]

,방언,표준어,연령대,출처 파일
7,그렇지 name는 자식이니까 당연히 신경을 써 줘야 되는데,그렇지 &name1&는 자식이니까 당연히 신경을 써 줘야 되는데,50대,DCDG20000009.json
25,그러니까 말투도 어 name 부르는 거 보다도 훨씬 더 애정을 듬뿍 담아 가지고,그러니까 말투도 어 &name1& 부르는 거 보다도 훨씬 더 애정을 듬뿍 담아 가지고,50대,DCDG20000009.json
36,name가 가끔씩 그~ 불만스럽게 투정부리는 날 좀 예뻐해달라는 그거는 어~,&name1&가 가끔씩 그~ 불만스럽게 투정부리는 날 좀 예뻐해달라는 그거는 어~,50대,DCDG20000009.json
38,쪼금 노력을 더 해보고 그냥 십팔 년 됐지만 그래도 name 동생이니까 name 혼...,쪼금 노력을 더 해보고 그냥 십팔 년 됐지만 그래도 &name1& 동생이니까 &na...,50대,DCDG20000009.json
39,동생 역할을 하고 있으니까 name도 엄마 못지않게 용봉이한테 더 애정을 담아서 더...,동생 역할을 하고 있으니까 &name1&도 엄마 못지않게 용봉이한테 더 애정을 담아...,50대,DCDG20000009.json
...,...,...,...,...
227404,name씨도 아빠 없어도 간다고 가도 또 옆에서 붙어서 하는 거랑 틀리더라구,&name5&씨도 아빠 없어도 간다고 가도 또 옆에서 붙어서 하는 거랑 틀리더라구,50대,DCNA20000035.json
227479,아저씨가 언니 아저씨만 잘하면 시댁 얘기할 거 없어요 우리도 name씨도,아저씨가 언니 아저씨만 잘하면 시댁 얘기할 거 없어요 우리도 &name5&씨도,50대,DCNA20000035.json
227480,name씨만 잘하면 시댁이야 다 할 거 얘 얘기 할 거 없는데,&name5&씨만 잘하면 시댁이야 다 할 거 얘 얘기 할 거 없는데,50대,DCNA20000035.json
227481,name씨가 막 열 받게 하면은 시댁도 다 미워서 그렇지 똑같지 뭐 언니네도,&name5&씨가 막 열 받게 하면은 시댁도 다 미워서 그렇지 똑같지 뭐 언니네도,50대,DCNA20000035.json


In [35]:
df["표준어"] = df["표준어"].str.replace(r"\&name[0-9]\&", "name", regex=True)
df["표준어"] = df["표준어"].str.replace(r"\&address[0-9]\&", "address", regex=True)
df["표준어"] = df["표준어"].str.replace(r"\&company-name[0-9]\&", "companyname", regex=True)
df["표준어"] = df["표준어"].str.replace(r"\&companyname[0-9]\&", "companyname", regex=True)

df[대체어_위치_표준어]

,방언,표준어,연령대,출처 파일
7,그렇지 name는 자식이니까 당연히 신경을 써 줘야 되는데,그렇지 name는 자식이니까 당연히 신경을 써 줘야 되는데,50대,DCDG20000009.json
25,그러니까 말투도 어 name 부르는 거 보다도 훨씬 더 애정을 듬뿍 담아 가지고,그러니까 말투도 어 name 부르는 거 보다도 훨씬 더 애정을 듬뿍 담아 가지고,50대,DCDG20000009.json
36,name가 가끔씩 그~ 불만스럽게 투정부리는 날 좀 예뻐해달라는 그거는 어~,name가 가끔씩 그~ 불만스럽게 투정부리는 날 좀 예뻐해달라는 그거는 어~,50대,DCDG20000009.json
38,쪼금 노력을 더 해보고 그냥 십팔 년 됐지만 그래도 name 동생이니까 name 혼...,쪼금 노력을 더 해보고 그냥 십팔 년 됐지만 그래도 name 동생이니까 name 혼...,50대,DCDG20000009.json
39,동생 역할을 하고 있으니까 name도 엄마 못지않게 용봉이한테 더 애정을 담아서 더...,동생 역할을 하고 있으니까 name도 엄마 못지않게 용봉이한테 더 애정을 담아서 더...,50대,DCDG20000009.json
...,...,...,...,...
227404,name씨도 아빠 없어도 간다고 가도 또 옆에서 붙어서 하는 거랑 틀리더라구,name씨도 아빠 없어도 간다고 가도 또 옆에서 붙어서 하는 거랑 틀리더라구,50대,DCNA20000035.json
227479,아저씨가 언니 아저씨만 잘하면 시댁 얘기할 거 없어요 우리도 name씨도,아저씨가 언니 아저씨만 잘하면 시댁 얘기할 거 없어요 우리도 name씨도,50대,DCNA20000035.json
227480,name씨만 잘하면 시댁이야 다 할 거 얘 얘기 할 거 없는데,name씨만 잘하면 시댁이야 다 할 거 얘 얘기 할 거 없는데,50대,DCNA20000035.json
227481,name씨가 막 열 받게 하면은 시댁도 다 미워서 그렇지 똑같지 뭐 언니네도,name씨가 막 열 받게 하면은 시댁도 다 미워서 그렇지 똑같지 뭐 언니네도,50대,DCNA20000035.json


### 데이터 길이

In [36]:
df["길이"] = [min(len(dia.split()), len(stan.split())) for dia, stan in zip(df["방언"], df["표준어"])]
df["길이"]

0          9
1          3
2         17
3          9
4          5
          ..
227998     7
227999     9
228000     4
228001    11
228002    10
Name: 길이, Length: 228003, dtype: int64

In [37]:
suf_len_df = df[df["길이"] > 4]
len(suf_len_df)

190001

In [38]:
dia_sur_len = len(suf_len_df[suf_len_df["방언"] != suf_len_df["표준어"]])
print("길이로 제외한 후 남은 방언 샘플 수: {0}".format(dia_sur_len))

길이로 제외한 후 남은 방언 샘플 수: 31055


In [39]:
suf_len_df.to_csv("../datas/output/chungcheong_dialect_test_age_processed.csv")

In [26]:
suf_len_df.to_csv("../datas/output/chungcheong_dialect_data_age_processed.csv")

### 1:1 샘플링 처리

In [2]:
suf_len_df = pd.read_csv("../datas/output/chungcheong_dialect_data_age_processed.csv", index_col=0)
suf_len_df

,방언,표준어,연령대,출처 파일,길이
0,네 요즘 프로 레슬링 유튜브 관련 채무너,네 요즘 프로 레슬링 유튜브 관련 채무너,20대,DCDG20000001.json,7
1,채널을 많이 보고 있는데 응 거기에서 많이 나오는 선수 자체가,채널을 많이 보고 있는데 응 거기에서 많이 나오는 선수 자체가,20대,DCDG20000001.json,10
2,그 드류 맥킨타이어 그 선수에 대해서 많이 나오더라고,그 드류 맥킨타이어 그 선수에 대해서 많이 나오더라고,20대,DCDG20000001.json,8
3,그 사람이 알고 보니까 십 년 전에는,그 사람이 알고 보니까 십 년 전에는,20대,DCDG20000001.json,7
4,그 자버라고 대개 선수들 막 승리 많이 하게 해 주는,그 자버라고 대개 선수들 막 승리 많이 하게 해 주는,20대,DCDG20000001.json,10
...,...,...,...,...,...
1848449,안 그러면 몸을 사리고 얌전히 있고 인제 쳐다보고 이~ 좀 들고 이케 봐야 되겠네 ...,안 그러면 몸을 사리고 얌전히 있고 인제 쳐다보고 이~ 좀 들고 이렇게 봐야 되겠네...,50대,DCNA20002450.json,17
1848450,부동산도 그리쿠 앞으로 인제 코로나가 어트게 될진 모르겠지만 앞으로 방향이,부동산도 그렇구 앞으로 인제 코로나가 어트게 될진 모르겠지만 앞으로 방향이,50대,DCNA20002450.json,10
1848451,코로나로 인해서 확 바껴 가지고 앞으로 우리 인제,코로나로 인해서 확 바뀌어 가지고 앞으로 우리 인제,50대,DCNA20002450.json,8
1848452,문화가 틀려져 문화가 문화도 틀려지고 이제 지금까지 살았던 패턴이 완전히 틀려져 가...,문화가 틀려져 문화가 문화도 틀려지고 이제 지금까지 살았던 패턴이 완전히 틀려져 가...,50대,DCNA20002450.json,13


In [3]:
dial_df = suf_len_df[suf_len_df["방언"] != suf_len_df["표준어"]]
stan_df = suf_len_df[suf_len_df["방언"] == suf_len_df["표준어"]]

In [4]:
print(len(dial_df))
print(len(stan_df))

260494
1308912


In [5]:
stan_sample = stan_df.sample(len(dial_df))
stan_sample

sampled_df = pd.concat([dial_df, stan_sample])
sampled_df

,방언,표준어,연령대,출처 파일,길이
111,이러한 대중들과 스타 그까 대중들의 스타성과,이러한 대중들과 스타 그러니까 대중들의 스타성과,20대,DCDG20000001.json,6
129,근데 프로모 그니까 또 스포츠 같은 경우에는 프로모터들이 엄청 중요하잖아.,근데 프로모 그러니까 또 스포츠 같은 경우에는 프로모터들이 엄청 중요하잖아.,20대,DCDG20000001.json,10
187,또 몇 프로는 생활비로 빠져나가면 이제 수중에 있는 돈이 얼마 없는 거야 그~ 미국...,또 몇 프로는 생활비로 빠져나가면 이제 수중에 있는 돈이 얼마 없는 거야 그~ 미국...,20대,DCDG20000001.json,16
230,그 유소년을 뗘 본 적이 없대.,그 유소년을 뛰어 본 적이 없대.,20대,DCDG20000001.json,6
296,이렇게 설명을 보는 거 보면서 되 나는 되게 놀랬거든.,이렇게 설명을 보는 거 보면서 되 나는 되게 놀랐거든.,20대,DCDG20000002.json,9
...,...,...,...,...,...
129853,전국 단위로 나가게 되니까 이제 사람들이 나보다 잘하는 사람들이 많은 거야.,전국 단위로 나가게 되니까 이제 사람들이 나보다 잘하는 사람들이 많은 거야.,10대,DCDG20000967.json,11
299079,바로 너한테 줬어 그 목도리를 핸드 메이드 라벨을 붙여서,바로 너한테 줬어 그 목도리를 핸드 메이드 라벨을 붙여서,20대,DCDG20001703.json,9
273572,뭐라 그럴까 그런 데 하다 보믄 투자도 해야 된대는데,뭐라 그럴까 그런 데 하다 보믄 투자도 해야 된대는데,50대,DCDG20001593.json,9
534493,그런 것처럼 쪼금 제 마음이 막 이렇게 막 불편하더라구요 그~,그런 것처럼 쪼금 제 마음이 막 이렇게 막 불편하더라구요 그~,20대,DCDG20002745.json,10


In [6]:
sampled_df = sampled_df.sample(frac=1).reset_index(drop=True) # 새로 샘플한 dataframe을 셔플
sampled_df

,방언,표준어,연령대,출처 파일,길이
0,영화보다는 그런 쪽으로 쫌 많이 다녀서,영화보다는 그런 쪽으로 조금 많이 다녀서,50대,DCNA20001462.json,6
1,에~ 저는 원래 고등학생 때까지,에~ 저는 원래 고등학생 때까지,20대,DCDG20000795.json,5
2,그 companyname 프로그램 이걸 많이 보거든.,그 companyname 프로그램 이걸 많이 보거든.,20대,DCDG20001601.json,6
3,고위험 그~ 군이 아니야 근데 만약에 그게 임신성 당뇨로 판정이 되면은 그 엄마는 ...,고위험 그~ 군이 아니야 근데 만약에 그게 임신성 당뇨로 판정이 되면은 그 엄마는 ...,40대,DCEX20001058.json,21
4,코로나 때문에 사람들이 많이 집에 있고 이제 이릏게 했으니까,코로나 때문에 사람들이 많이 집에 있고 이제 이렇게 했으니까,40대,DCNA20001390.json,9
...,...,...,...,...,...
520983,거기서 밥을 해면 뭐 누룽지를 해야 되네 뭐 하네 막 이러는데 밥 할 것이 없어,거기서 밥을 해면 뭐 누룽지를 해야 되네 뭐 하네 막 이러는데 밥 할 것이 없어,60대 이상,DCNA20001746.json,15
520984,그~ 욕심을 부려서 그려 욕심을 부려 갖고,그~ 욕심을 부려서 그래 욕심을 부려 갖고,60대 이상,DCNA20001459.json,7
520985,그~ 낮아졌다는 게 쫌 그렇게 어~ 한 게 아니라,그~ 낮아졌다는 게 조금 그렇게 어~ 한 게 아니라,50대,DCDG20000849.json,9
520986,사람들 퀸와사비 처음 나왔을 때도 완전 문화충격이어가지구,사람들 퀸와사비 처음 나왔을 때도 완전 문화충격이어가지구,20대,DCNA20000770.json,7


In [7]:
sampled_df.to_csv("../datas/output/chungcheong_dialect_data_age_sampled.csv")

* 충청 방언 필터링 후 훈련 샘플 수: 520988
* 충청 방언 필터링 후 테스트 샘플 수: 190001

### 형태소 토큰화

In [2]:
import MeCab

def pos(sentence: str):
    """한국어 토큰을 분리합니다. 토큰과 품사를 튜플 리스트로 반환합니다.
        
        매개변수:
            sentence (str): 토큰화할 문장 
        반환값:
            token list (list[tuple]): 토큰과 품사 리스트
    """
    t = MeCab.Tagger()
    tag_result = t.parse(sentence)
    tag_result = tag_result.replace("\t", ".@!").replace("\n", ".@!").split(".@!")
    tag_word = tag_result[::2][:-1] # 마지막 EOS는 자른다
    tag_info = tag_result[1::2][:-1] # 마지막 EOS는 자른다
    return [(word, info.split(',')[0]) for word, info in zip(tag_word, tag_info)]

def morphs(sentence: str):
    """한국어 토큰을 분리합니다. 토큰의 리스트를 반환합니다.
        
        매개변수:
            sentence (str): 토큰화할 문장 
        반환값:
            token list (list): 토큰 리스트
    """
    t = MeCab.Tagger()
    tag_result = t.parse(sentence)
    tag_result = tag_result.replace("\t", ".@!").replace("\n", ".@!").split(".@!")
    return tag_result[::2][:-1]

def space_tokenize(original_sentence, token_list):
    """
    문장을 decode할 때 한국어의 띄어쓰기도 되살릴 수 있도록 token에 이를 반영하도록 처리하는 함수
    """
    
    space_token = original_sentence.split()

    if len(space_token) == 0:
        return token_list

    token_idx = 0
    cur_token = space_token[token_idx]
    cum_word = ""
    for i, word in enumerate(token_list):
        cum_word += word

        if len(cum_word) > len(word):
            token_list[i] = "##" + token_list[i]
            
        if cum_word == cur_token:
            token_idx += 1
            cum_word = ""
            cur_token = space_token[min(token_idx, len(space_token) - 1)]
    
    return token_list

def morph_and_preprocess(sentence: str):
    """한국어 토큰을 분리하고 전처리합니다. 토큰의 리스트를 반환합니다.
        
        매개변수:
            sentence (str): 토큰화할 문장 
        반환값:
            token list (list): 토큰 리스트
    """
    
    pos_result = pos(sentence)
    word_list = [word for word, _ in pos_result]
    word_list = space_tokenize(sentence, word_list) # 공백 복원 토큰화
    
    return word_list

In [3]:
df = pd.read_csv("../datas/output/chungcheong_dialect_data_age_sampled.csv", index_col=0)
df

,방언,표준어,연령대,출처 파일,길이
0,영화보다는 그런 쪽으로 쫌 많이 다녀서,영화보다는 그런 쪽으로 조금 많이 다녀서,50대,DCNA20001462.json,6
1,에~ 저는 원래 고등학생 때까지,에~ 저는 원래 고등학생 때까지,20대,DCDG20000795.json,5
2,그 companyname 프로그램 이걸 많이 보거든.,그 companyname 프로그램 이걸 많이 보거든.,20대,DCDG20001601.json,6
3,고위험 그~ 군이 아니야 근데 만약에 그게 임신성 당뇨로 판정이 되면은 그 엄마는 ...,고위험 그~ 군이 아니야 근데 만약에 그게 임신성 당뇨로 판정이 되면은 그 엄마는 ...,40대,DCEX20001058.json,21
4,코로나 때문에 사람들이 많이 집에 있고 이제 이릏게 했으니까,코로나 때문에 사람들이 많이 집에 있고 이제 이렇게 했으니까,40대,DCNA20001390.json,9
...,...,...,...,...,...
520983,거기서 밥을 해면 뭐 누룽지를 해야 되네 뭐 하네 막 이러는데 밥 할 것이 없어,거기서 밥을 해면 뭐 누룽지를 해야 되네 뭐 하네 막 이러는데 밥 할 것이 없어,60대 이상,DCNA20001746.json,15
520984,그~ 욕심을 부려서 그려 욕심을 부려 갖고,그~ 욕심을 부려서 그래 욕심을 부려 갖고,60대 이상,DCNA20001459.json,7
520985,그~ 낮아졌다는 게 쫌 그렇게 어~ 한 게 아니라,그~ 낮아졌다는 게 조금 그렇게 어~ 한 게 아니라,50대,DCDG20000849.json,9
520986,사람들 퀸와사비 처음 나왔을 때도 완전 문화충격이어가지구,사람들 퀸와사비 처음 나왔을 때도 완전 문화충격이어가지구,20대,DCNA20000770.json,7


In [4]:
dialect_list = df["방언"].to_list()
dialect_list = list(map(morph_and_preprocess, dialect_list))

standard_list = df["표준어"].to_list()
standard_list = list(map(morph_and_preprocess, standard_list))

In [5]:
list_to_spaced_sentence = lambda li : " ".join(li)
df.loc[df.index, "방언"] = list(map(list_to_spaced_sentence, dialect_list))
df.loc[df.index, "표준어"] = list(map(list_to_spaced_sentence, standard_list))

df.head()

,방언,표준어,연령대,출처 파일,길이
0,영화 ##보다 ##는 그런 쪽 ##으로 쫌 많이 다녀서,영화 ##보다 ##는 그런 쪽 ##으로 조금 많이 다녀서,50대,DCNA20001462.json,6
1,에 ##~ 저 ##는 원래 고등학생 때 ##까지,에 ##~ 저 ##는 원래 고등학생 때 ##까지,20대,DCDG20000795.json,5
2,그 companyname 프로그램 이걸 많이 보 ##거든 ##.,그 companyname 프로그램 이걸 많이 보 ##거든 ##.,20대,DCDG20001601.json,6
3,고위험 그 ##~ 군 ##이 아니 ##야 근데 만약 ##에 그게 임신 ##성 당뇨 ...,고위험 그 ##~ 군 ##이 아니 ##야 근데 만약 ##에 그게 임신 ##성 당뇨 ...,40대,DCEX20001058.json,21
4,코로나 때문 ##에 사람 ##들 ##이 많이 집 ##에 있 ##고 이제 이릏 ##게...,코로나 때문 ##에 사람 ##들 ##이 많이 집 ##에 있 ##고 이제 이렇게 했 ...,40대,DCNA20001390.json,9


In [6]:
df.to_csv("../datas/output/chungcheong_dialect_data_형태소.csv")

### 테스트셋 형태소

In [7]:
df = pd.read_csv("../datas/output/chungcheong_dialect_test_age_processed.csv", index_col=0)
df

,방언,표준어,연령대,출처 파일,길이
0,우리집 거북이 용봉이가 지금 십팔 년에서 십구 년차 되잖아.,우리집 거북이 용봉이가 지금 십팔 년에서 십구 년차 되잖아.,50대,DCDG20000009.json,9
2,어~ 아파서 고비가 있었는데 지금은 그걸 잘 극복해 가지고 건강하게 우리 식구랑 같...,어~ 아파서 고비가 있었는데 지금은 그걸 잘 극복해 가지고 건강하게 우리 식구랑 같...,50대,DCDG20000009.json,17
3,근데 엄마가 너무 용봉이를 예뻐한다고 니가 엄청 질투가 많은데,근데 엄마가 너무 용봉이를 예뻐한다고 니가 엄청 질투가 많은데,50대,DCDG20000009.json,9
4,그래도 용봉이가 가족 일원으로서 예쁘지?,그래도 용봉이가 가족 일원으로서 예쁘지?,50대,DCDG20000009.json,5
5,아~ 예쁘긴 한데 엄마는 너무 나보다는 용봉이를 좀 더 많이 챙기지 않나 하는 생각...,아~ 예쁘긴 한데 엄마는 너무 나보다는 용봉이를 좀 더 많이 챙기지 않나 하는 생각...,20대,DCDG20000009.json,15
...,...,...,...,...,...
227997,이제 그런 소리가 예전에는 아후 그런게 무슨 와닿지 않았는데 지금은 인제,이제 그런 소리가 예전에는 아후 그런게 무슨 와닿지 않았는데 지금은 인제,50대,DCNA20000036.json,11
227998,그~ 선조들이 했던 그 말씀들이 새록새록 와닿아요.,그~ 선조들이 했던 그 말씀들이 새록새록 와닿아요.,50대,DCNA20000036.json,7
227999,인제 철들자 잉~ 저기 한다고 쪼꼼 철이 드는지 어쩐지,인제 철들자 잉~ 저기 한다고 조금 철이 드는지 어쩐지,50대,DCNA20000036.json,9
228001,어떻게 보면 선생님 지금 그게 오히려 인제 마음이 여유로워 졌기 때문에,어떻게 보면 선생님 지금 그게 오히려 인제 마음이 여유로워 졌기 때문에,50대,DCNA20000036.json,11


In [8]:
dialect_list = df["방언"].to_list()
dialect_list = list(map(morph_and_preprocess, dialect_list))

standard_list = df["표준어"].to_list()
standard_list = list(map(morph_and_preprocess, standard_list))

In [9]:
list_to_spaced_sentence = lambda li : " ".join(li)
df["방언"] = list(map(list_to_spaced_sentence, dialect_list))
df["표준어"] = list(map(list_to_spaced_sentence, standard_list))

df.head()

,방언,표준어,연령대,출처 파일,길이
0,우리 ##집 거북 ##이 용봉 ##이 ##가 지금 십 ##팔 년 ##에서 십 ##구...,우리 ##집 거북 ##이 용봉 ##이 ##가 지금 십 ##팔 년 ##에서 십 ##구...,50대,DCDG20000009.json,9
2,어 ##~ 아파서 고비 ##가 있 ##었 ##는데 지금 ##은 그걸 잘 극복 ##해...,어 ##~ 아파서 고비 ##가 있 ##었 ##는데 지금 ##은 그걸 잘 극복 ##해...,50대,DCDG20000009.json,17
3,근데 엄마 ##가 너무 용봉 ##이 ##를 예뻐 ##한다고 니 ##가 엄청 질투 #...,근데 엄마 ##가 너무 용봉 ##이 ##를 예뻐 ##한다고 니 ##가 엄청 질투 #...,50대,DCDG20000009.json,9
4,그래도 용봉 ##이 ##가 가족 일원 ##으로서 예쁘 ##지 ##?,그래도 용봉 ##이 ##가 가족 일원 ##으로서 예쁘 ##지 ##?,50대,DCDG20000009.json,5
5,아 ##~ 예쁘 ##긴 한데 엄마 ##는 너무 나 ##보다 ##는 용봉 ##이 ##...,아 ##~ 예쁘 ##긴 한데 엄마 ##는 너무 나 ##보다 ##는 용봉 ##이 ##...,20대,DCDG20000009.json,15


In [10]:
df.to_csv("../datas/output/chungcheong_dialect_test_형태소.csv")

### SentencePiece 토큰화

In [11]:
df = pd.read_csv("../datas/output/chungcheong_dialect_data_age_sampled.csv", index_col=0)
df

,방언,표준어,연령대,출처 파일,길이
0,영화보다는 그런 쪽으로 쫌 많이 다녀서,영화보다는 그런 쪽으로 조금 많이 다녀서,50대,DCNA20001462.json,6
1,에~ 저는 원래 고등학생 때까지,에~ 저는 원래 고등학생 때까지,20대,DCDG20000795.json,5
2,그 companyname 프로그램 이걸 많이 보거든.,그 companyname 프로그램 이걸 많이 보거든.,20대,DCDG20001601.json,6
3,고위험 그~ 군이 아니야 근데 만약에 그게 임신성 당뇨로 판정이 되면은 그 엄마는 ...,고위험 그~ 군이 아니야 근데 만약에 그게 임신성 당뇨로 판정이 되면은 그 엄마는 ...,40대,DCEX20001058.json,21
4,코로나 때문에 사람들이 많이 집에 있고 이제 이릏게 했으니까,코로나 때문에 사람들이 많이 집에 있고 이제 이렇게 했으니까,40대,DCNA20001390.json,9
...,...,...,...,...,...
520983,거기서 밥을 해면 뭐 누룽지를 해야 되네 뭐 하네 막 이러는데 밥 할 것이 없어,거기서 밥을 해면 뭐 누룽지를 해야 되네 뭐 하네 막 이러는데 밥 할 것이 없어,60대 이상,DCNA20001746.json,15
520984,그~ 욕심을 부려서 그려 욕심을 부려 갖고,그~ 욕심을 부려서 그래 욕심을 부려 갖고,60대 이상,DCNA20001459.json,7
520985,그~ 낮아졌다는 게 쫌 그렇게 어~ 한 게 아니라,그~ 낮아졌다는 게 조금 그렇게 어~ 한 게 아니라,50대,DCDG20000849.json,9
520986,사람들 퀸와사비 처음 나왔을 때도 완전 문화충격이어가지구,사람들 퀸와사비 처음 나왔을 때도 완전 문화충격이어가지구,20대,DCNA20000770.json,7


In [12]:
df = df.dropna(how='any')
with open('../datas/output/chungcheong_dialect_data_text.txt', 'w', encoding='utf8') as f:
    f.write('\n'.join(df['방언'] + " " + df["표준어"]))

In [13]:
import sentencepiece as spm

spm.SentencePieceTrainer.train(input='../datas/output/chungcheong_dialect_data_text.txt', model_prefix='chungcheong_data_sp', vocab_size=30000)

In [14]:
sp_tokenizer = spm.SentencePieceProcessor(model_file='chungcheong_data_sp.model')

code_result_dialect = sp_tokenizer.encode(df["방언"].to_list(), out_type=str)
code_result_standard = sp_tokenizer.encode(df["표준어"].to_list(), out_type=str)

In [15]:
df.loc[df.index, "방언"] = [' '.join(res) for res in code_result_dialect]
df["방언"]

0                              ▁영화 보다는 ▁그런 ▁쪽으로 ▁쫌 ▁많이 ▁다녀서
1                                   ▁에 ~ ▁저는 ▁원래 ▁고등학생 ▁때까지
2                      ▁그 ▁companyname ▁프로그램 ▁이걸 ▁많이 ▁보거든 .
3         ▁고 위 험 ▁그 ~ ▁군 이 ▁아니야 ▁근데 ▁만약에 ▁그게 ▁임신 성 ▁당뇨 로...
4                ▁코로나 ▁때문에 ▁사람들이 ▁많이 ▁집에 ▁있고 ▁이제 ▁이릏게 ▁했으니까
                                ...                        
520983    ▁거기서 ▁밥을 ▁해 면 ▁뭐 ▁누룽지 를 ▁해야 ▁되네 ▁뭐 ▁하네 ▁막 ▁이러는...
520984                     ▁그 ~ ▁욕심을 ▁부려 서 ▁그려 ▁욕심을 ▁부려 ▁갖고
520985             ▁그 ~ ▁낮아 졌다 는 ▁게 ▁쫌 ▁그렇게 ▁어 ~ ▁한 ▁게 ▁아니라
520986        ▁사람들 ▁ 퀸 와 사 비 ▁처음 ▁나왔을 ▁때도 ▁완전 ▁문화 충 격 이어가지구
520987    ▁그나마 ▁발라드 ▁ 중엔 ▁제가 ▁옛날 ▁발라드를 ▁쫌 ▁그래도 ▁좋아했었던 ▁편...
Name: 방언, Length: 520967, dtype: object

In [16]:
df.loc[df.index, "표준어"] = [' '.join(res) for res in code_result_standard]
df["표준어"]

0                             ▁영화 보다는 ▁그런 ▁쪽으로 ▁조금 ▁많이 ▁다녀서
1                                   ▁에 ~ ▁저는 ▁원래 ▁고등학생 ▁때까지
2                      ▁그 ▁companyname ▁프로그램 ▁이걸 ▁많이 ▁보거든 .
3         ▁고 위 험 ▁그 ~ ▁군 이 ▁아니야 ▁근데 ▁만약에 ▁그게 ▁임신 성 ▁당뇨 로...
4                ▁코로나 ▁때문에 ▁사람들이 ▁많이 ▁집에 ▁있고 ▁이제 ▁이렇게 ▁했으니까
                                ...                        
520983    ▁거기서 ▁밥을 ▁해 면 ▁뭐 ▁누룽지 를 ▁해야 ▁되네 ▁뭐 ▁하네 ▁막 ▁이러는...
520984                     ▁그 ~ ▁욕심을 ▁부려 서 ▁그래 ▁욕심을 ▁부려 ▁갖고
520985            ▁그 ~ ▁낮아 졌다 는 ▁게 ▁조금 ▁그렇게 ▁어 ~ ▁한 ▁게 ▁아니라
520986        ▁사람들 ▁ 퀸 와 사 비 ▁처음 ▁나왔을 ▁때도 ▁완전 ▁문화 충 격 이어가지구
520987    ▁그나마 ▁발라드 ▁ 중엔 ▁제가 ▁옛날 ▁발라드를 ▁조금 ▁그래도 ▁좋아했었던 ▁...
Name: 표준어, Length: 520967, dtype: object

In [17]:
df.to_csv("../datas/output/chungcheong_dialect_data_SentencePiece.csv")

### 테스트셋 SentencePiece

In [18]:
df = pd.read_csv("../datas/output/chungcheong_dialect_test_age_processed.csv", index_col=0)
df

,방언,표준어,연령대,출처 파일,길이
0,우리집 거북이 용봉이가 지금 십팔 년에서 십구 년차 되잖아.,우리집 거북이 용봉이가 지금 십팔 년에서 십구 년차 되잖아.,50대,DCDG20000009.json,9
2,어~ 아파서 고비가 있었는데 지금은 그걸 잘 극복해 가지고 건강하게 우리 식구랑 같...,어~ 아파서 고비가 있었는데 지금은 그걸 잘 극복해 가지고 건강하게 우리 식구랑 같...,50대,DCDG20000009.json,17
3,근데 엄마가 너무 용봉이를 예뻐한다고 니가 엄청 질투가 많은데,근데 엄마가 너무 용봉이를 예뻐한다고 니가 엄청 질투가 많은데,50대,DCDG20000009.json,9
4,그래도 용봉이가 가족 일원으로서 예쁘지?,그래도 용봉이가 가족 일원으로서 예쁘지?,50대,DCDG20000009.json,5
5,아~ 예쁘긴 한데 엄마는 너무 나보다는 용봉이를 좀 더 많이 챙기지 않나 하는 생각...,아~ 예쁘긴 한데 엄마는 너무 나보다는 용봉이를 좀 더 많이 챙기지 않나 하는 생각...,20대,DCDG20000009.json,15
...,...,...,...,...,...
227997,이제 그런 소리가 예전에는 아후 그런게 무슨 와닿지 않았는데 지금은 인제,이제 그런 소리가 예전에는 아후 그런게 무슨 와닿지 않았는데 지금은 인제,50대,DCNA20000036.json,11
227998,그~ 선조들이 했던 그 말씀들이 새록새록 와닿아요.,그~ 선조들이 했던 그 말씀들이 새록새록 와닿아요.,50대,DCNA20000036.json,7
227999,인제 철들자 잉~ 저기 한다고 쪼꼼 철이 드는지 어쩐지,인제 철들자 잉~ 저기 한다고 조금 철이 드는지 어쩐지,50대,DCNA20000036.json,9
228001,어떻게 보면 선생님 지금 그게 오히려 인제 마음이 여유로워 졌기 때문에,어떻게 보면 선생님 지금 그게 오히려 인제 마음이 여유로워 졌기 때문에,50대,DCNA20000036.json,11


In [19]:
df = df.dropna(how='any')
with open('../datas/output/chungcheong_dialect_test_text.txt', 'w', encoding='utf8') as f:
    f.write('\n'.join(df['방언'] + " " + df["표준어"]))

In [20]:
spm.SentencePieceTrainer.train(input='../datas/output/chungcheong_dialect_test_text.txt', model_prefix='chungcheong_sp', vocab_size=30000)

sp_tokenizer = spm.SentencePieceProcessor(model_file='chungcheong_sp.model')

code_result_dialect = sp_tokenizer.encode(df["방언"].to_list(), out_type=str)
code_result_standard = sp_tokenizer.encode(df["표준어"].to_list(), out_type=str)

In [21]:
df.loc[df.index, "방언"] = [' '.join(res) for res in code_result_dialect]
df["방언"]

0              ▁우리집 ▁거북이 ▁용봉이가 ▁지금 ▁십팔 ▁년 에서 ▁십구 ▁년차 ▁되잖아 .
2         ▁어 ~ ▁아파서 ▁고 비가 ▁있었는데 ▁지금은 ▁그걸 ▁잘 ▁극복해 ▁가지고 ▁건...
3            ▁근데 ▁엄마가 ▁너무 ▁용봉이 를 ▁예뻐 한다고 ▁니가 ▁엄청 ▁질투 가 ▁많은데
4                            ▁그래도 ▁용봉이가 ▁가족 ▁일 원 으로서 ▁예쁘지 ?
5         ▁아 ~ ▁예쁘 긴 ▁한데 ▁엄마는 ▁너무 ▁나 보다는 ▁용봉이 를 ▁좀 ▁더 ▁많...
                                ...                        
227997    ▁이제 ▁그런 ▁소리가 ▁예전에는 ▁아후 ▁그런게 ▁무슨 ▁와닿지 ▁않았는데 ▁지금...
227998           ▁그 ~ ▁선 조 들이 ▁했던 ▁그 ▁말씀 들이 ▁새록새록 ▁와 닿아 요 .
227999          ▁인제 ▁철 들 자 ▁잉 ~ ▁저기 ▁한다고 ▁쪼꼼 ▁철이 ▁드는 지 ▁어쩐지
228001    ▁어떻게 ▁보면 ▁선생님 ▁지금 ▁그게 ▁오히려 ▁인제 ▁마음이 ▁여유 로워 ▁졌 ...
228002          ▁이게 ▁좀 ▁넓게 ▁볼 ▁수 ▁있는 게 ▁아닐까 ▁그런 ▁생각도 ▁들어요 .
Name: 방언, Length: 190000, dtype: object

In [22]:
df.loc[df.index, "표준어"] = [' '.join(res) for res in code_result_standard]
df["표준어"]

0              ▁우리집 ▁거북이 ▁용봉이가 ▁지금 ▁십팔 ▁년 에서 ▁십구 ▁년차 ▁되잖아 .
2         ▁어 ~ ▁아파서 ▁고 비가 ▁있었는데 ▁지금은 ▁그걸 ▁잘 ▁극복해 ▁가지고 ▁건...
3            ▁근데 ▁엄마가 ▁너무 ▁용봉이 를 ▁예뻐 한다고 ▁니가 ▁엄청 ▁질투 가 ▁많은데
4                            ▁그래도 ▁용봉이가 ▁가족 ▁일 원 으로서 ▁예쁘지 ?
5         ▁아 ~ ▁예쁘 긴 ▁한데 ▁엄마는 ▁너무 ▁나 보다는 ▁용봉이 를 ▁좀 ▁더 ▁많...
                                ...                        
227997    ▁이제 ▁그런 ▁소리가 ▁예전에는 ▁아후 ▁그런게 ▁무슨 ▁와닿지 ▁않았는데 ▁지금...
227998           ▁그 ~ ▁선 조 들이 ▁했던 ▁그 ▁말씀 들이 ▁새록새록 ▁와 닿아 요 .
227999          ▁인제 ▁철 들 자 ▁잉 ~ ▁저기 ▁한다고 ▁조금 ▁철이 ▁드는 지 ▁어쩐지
228001    ▁어떻게 ▁보면 ▁선생님 ▁지금 ▁그게 ▁오히려 ▁인제 ▁마음이 ▁여유 로워 ▁졌 ...
228002          ▁이게 ▁좀 ▁넓게 ▁볼 ▁수 ▁있는 게 ▁아닐까 ▁그런 ▁생각도 ▁들어요 .
Name: 표준어, Length: 190000, dtype: object

In [23]:
df.to_csv("../datas/output/chungcheong_dialect_test_SentencePiece.csv")

### 자모 토큰화

In [2]:
df = pd.read_csv("../datas/output/chungcheong_dialect_data_age_sampled.csv", index_col=0)
df

,방언,표준어,연령대,출처 파일,길이
0,영화보다는 그런 쪽으로 쫌 많이 다녀서,영화보다는 그런 쪽으로 조금 많이 다녀서,50대,DCNA20001462.json,6
1,에~ 저는 원래 고등학생 때까지,에~ 저는 원래 고등학생 때까지,20대,DCDG20000795.json,5
2,그 companyname 프로그램 이걸 많이 보거든.,그 companyname 프로그램 이걸 많이 보거든.,20대,DCDG20001601.json,6
3,고위험 그~ 군이 아니야 근데 만약에 그게 임신성 당뇨로 판정이 되면은 그 엄마는 ...,고위험 그~ 군이 아니야 근데 만약에 그게 임신성 당뇨로 판정이 되면은 그 엄마는 ...,40대,DCEX20001058.json,21
4,코로나 때문에 사람들이 많이 집에 있고 이제 이릏게 했으니까,코로나 때문에 사람들이 많이 집에 있고 이제 이렇게 했으니까,40대,DCNA20001390.json,9
...,...,...,...,...,...
520983,거기서 밥을 해면 뭐 누룽지를 해야 되네 뭐 하네 막 이러는데 밥 할 것이 없어,거기서 밥을 해면 뭐 누룽지를 해야 되네 뭐 하네 막 이러는데 밥 할 것이 없어,60대 이상,DCNA20001746.json,15
520984,그~ 욕심을 부려서 그려 욕심을 부려 갖고,그~ 욕심을 부려서 그래 욕심을 부려 갖고,60대 이상,DCNA20001459.json,7
520985,그~ 낮아졌다는 게 쫌 그렇게 어~ 한 게 아니라,그~ 낮아졌다는 게 조금 그렇게 어~ 한 게 아니라,50대,DCDG20000849.json,9
520986,사람들 퀸와사비 처음 나왔을 때도 완전 문화충격이어가지구,사람들 퀸와사비 처음 나왔을 때도 완전 문화충격이어가지구,20대,DCNA20000770.json,7


In [3]:
from jamo import h2j, j2hcj # 자모화

def jamo_tokenization(sentence):
    token_list = []

    for c in sentence:
        if c == ' ':
            token_list.append("<SPC>")
        else:
            jamo_str = j2hcj(h2j(c))
            token_list.extend(jamo_str)
            token_list.append("<SEP>")
    
    return token_list

In [4]:
dialect_list = df["방언"].to_list()
dialect_list = list(map(jamo_tokenization, dialect_list))

standard_list = df["표준어"].to_list()
standard_list = list(map(jamo_tokenization, standard_list))

In [5]:
list_to_spaced_sentence = lambda li : " ".join(li)
df.loc[df.index, "방언"] = list(map(list_to_spaced_sentence, dialect_list))
df.loc[df.index, "표준어"] = list(map(list_to_spaced_sentence, standard_list))

df

,방언,표준어,연령대,출처 파일,길이
0,ㅇ ㅕ ㅇ <SEP> ㅎ ㅘ <SEP> ㅂ ㅗ <SEP> ㄷ ㅏ <SEP> ㄴ ㅡ ...,ㅇ ㅕ ㅇ <SEP> ㅎ ㅘ <SEP> ㅂ ㅗ <SEP> ㄷ ㅏ <SEP> ㄴ ㅡ ...,50대,DCNA20001462.json,6
1,ㅇ ㅔ <SEP> ~ <SEP> <SPC> ㅈ ㅓ <SEP> ㄴ ㅡ ㄴ <SEP> ...,ㅇ ㅔ <SEP> ~ <SEP> <SPC> ㅈ ㅓ <SEP> ㄴ ㅡ ㄴ <SEP> ...,20대,DCDG20000795.json,5
2,ㄱ ㅡ <SEP> <SPC> c <SEP> o <SEP> m <SEP> p <SEP...,ㄱ ㅡ <SEP> <SPC> c <SEP> o <SEP> m <SEP> p <SEP...,20대,DCDG20001601.json,6
3,ㄱ ㅗ <SEP> ㅇ ㅟ <SEP> ㅎ ㅓ ㅁ <SEP> <SPC> ㄱ ㅡ <SEP...,ㄱ ㅗ <SEP> ㅇ ㅟ <SEP> ㅎ ㅓ ㅁ <SEP> <SPC> ㄱ ㅡ <SEP...,40대,DCEX20001058.json,21
4,ㅋ ㅗ <SEP> ㄹ ㅗ <SEP> ㄴ ㅏ <SEP> <SPC> ㄸ ㅐ <SEP> ...,ㅋ ㅗ <SEP> ㄹ ㅗ <SEP> ㄴ ㅏ <SEP> <SPC> ㄸ ㅐ <SEP> ...,40대,DCNA20001390.json,9
...,...,...,...,...,...
520983,ㄱ ㅓ <SEP> ㄱ ㅣ <SEP> ㅅ ㅓ <SEP> <SPC> ㅂ ㅏ ㅂ <SEP...,ㄱ ㅓ <SEP> ㄱ ㅣ <SEP> ㅅ ㅓ <SEP> <SPC> ㅂ ㅏ ㅂ <SEP...,60대 이상,DCNA20001746.json,15
520984,ㄱ ㅡ <SEP> ~ <SEP> <SPC> ㅇ ㅛ ㄱ <SEP> ㅅ ㅣ ㅁ <SEP...,ㄱ ㅡ <SEP> ~ <SEP> <SPC> ㅇ ㅛ ㄱ <SEP> ㅅ ㅣ ㅁ <SEP...,60대 이상,DCNA20001459.json,7
520985,ㄱ ㅡ <SEP> ~ <SEP> <SPC> ㄴ ㅏ ㅈ <SEP> ㅇ ㅏ <SEP> ...,ㄱ ㅡ <SEP> ~ <SEP> <SPC> ㄴ ㅏ ㅈ <SEP> ㅇ ㅏ <SEP> ...,50대,DCDG20000849.json,9
520986,ㅅ ㅏ <SEP> ㄹ ㅏ ㅁ <SEP> ㄷ ㅡ ㄹ <SEP> <SPC> ㅋ ㅟ ㄴ ...,ㅅ ㅏ <SEP> ㄹ ㅏ ㅁ <SEP> ㄷ ㅡ ㄹ <SEP> <SPC> ㅋ ㅟ ㄴ ...,20대,DCNA20000770.json,7


In [6]:
df.to_csv("../datas/output/chungcheong_dialect_data_jamo.csv")

### 테스트셋 자모

In [7]:
df = pd.read_csv("../datas/output/chungcheong_dialect_test_age_processed.csv", index_col=0)
df

,방언,표준어,연령대,출처 파일,길이
0,우리집 거북이 용봉이가 지금 십팔 년에서 십구 년차 되잖아.,우리집 거북이 용봉이가 지금 십팔 년에서 십구 년차 되잖아.,50대,DCDG20000009.json,9
2,어~ 아파서 고비가 있었는데 지금은 그걸 잘 극복해 가지고 건강하게 우리 식구랑 같...,어~ 아파서 고비가 있었는데 지금은 그걸 잘 극복해 가지고 건강하게 우리 식구랑 같...,50대,DCDG20000009.json,17
3,근데 엄마가 너무 용봉이를 예뻐한다고 니가 엄청 질투가 많은데,근데 엄마가 너무 용봉이를 예뻐한다고 니가 엄청 질투가 많은데,50대,DCDG20000009.json,9
4,그래도 용봉이가 가족 일원으로서 예쁘지?,그래도 용봉이가 가족 일원으로서 예쁘지?,50대,DCDG20000009.json,5
5,아~ 예쁘긴 한데 엄마는 너무 나보다는 용봉이를 좀 더 많이 챙기지 않나 하는 생각...,아~ 예쁘긴 한데 엄마는 너무 나보다는 용봉이를 좀 더 많이 챙기지 않나 하는 생각...,20대,DCDG20000009.json,15
...,...,...,...,...,...
227997,이제 그런 소리가 예전에는 아후 그런게 무슨 와닿지 않았는데 지금은 인제,이제 그런 소리가 예전에는 아후 그런게 무슨 와닿지 않았는데 지금은 인제,50대,DCNA20000036.json,11
227998,그~ 선조들이 했던 그 말씀들이 새록새록 와닿아요.,그~ 선조들이 했던 그 말씀들이 새록새록 와닿아요.,50대,DCNA20000036.json,7
227999,인제 철들자 잉~ 저기 한다고 쪼꼼 철이 드는지 어쩐지,인제 철들자 잉~ 저기 한다고 조금 철이 드는지 어쩐지,50대,DCNA20000036.json,9
228001,어떻게 보면 선생님 지금 그게 오히려 인제 마음이 여유로워 졌기 때문에,어떻게 보면 선생님 지금 그게 오히려 인제 마음이 여유로워 졌기 때문에,50대,DCNA20000036.json,11


In [8]:
dialect_list = df["방언"].to_list()
dialect_list = list(map(jamo_tokenization, dialect_list))

standard_list = df["표준어"].to_list()
standard_list = list(map(jamo_tokenization, standard_list))

In [9]:
list_to_spaced_sentence = lambda li : " ".join(li)
df.loc[df.index, "방언"] = list(map(list_to_spaced_sentence, dialect_list))
df.loc[df.index, "표준어"] = list(map(list_to_spaced_sentence, standard_list))

df

,방언,표준어,연령대,출처 파일,길이
0,ㅇ ㅜ <SEP> ㄹ ㅣ <SEP> ㅈ ㅣ ㅂ <SEP> <SPC> ㄱ ㅓ <SEP...,ㅇ ㅜ <SEP> ㄹ ㅣ <SEP> ㅈ ㅣ ㅂ <SEP> <SPC> ㄱ ㅓ <SEP...,50대,DCDG20000009.json,9
2,ㅇ ㅓ <SEP> ~ <SEP> <SPC> ㅇ ㅏ <SEP> ㅍ ㅏ <SEP> ㅅ ...,ㅇ ㅓ <SEP> ~ <SEP> <SPC> ㅇ ㅏ <SEP> ㅍ ㅏ <SEP> ㅅ ...,50대,DCDG20000009.json,17
3,ㄱ ㅡ ㄴ <SEP> ㄷ ㅔ <SEP> <SPC> ㅇ ㅓ ㅁ <SEP> ㅁ ㅏ <S...,ㄱ ㅡ ㄴ <SEP> ㄷ ㅔ <SEP> <SPC> ㅇ ㅓ ㅁ <SEP> ㅁ ㅏ <S...,50대,DCDG20000009.json,9
4,ㄱ ㅡ <SEP> ㄹ ㅐ <SEP> ㄷ ㅗ <SEP> <SPC> ㅇ ㅛ ㅇ <SEP...,ㄱ ㅡ <SEP> ㄹ ㅐ <SEP> ㄷ ㅗ <SEP> <SPC> ㅇ ㅛ ㅇ <SEP...,50대,DCDG20000009.json,5
5,ㅇ ㅏ <SEP> ~ <SEP> <SPC> ㅇ ㅖ <SEP> ㅃ ㅡ <SEP> ㄱ ...,ㅇ ㅏ <SEP> ~ <SEP> <SPC> ㅇ ㅖ <SEP> ㅃ ㅡ <SEP> ㄱ ...,20대,DCDG20000009.json,15
...,...,...,...,...,...
227997,ㅇ ㅣ <SEP> ㅈ ㅔ <SEP> <SPC> ㄱ ㅡ <SEP> ㄹ ㅓ ㄴ <SEP...,ㅇ ㅣ <SEP> ㅈ ㅔ <SEP> <SPC> ㄱ ㅡ <SEP> ㄹ ㅓ ㄴ <SEP...,50대,DCNA20000036.json,11
227998,ㄱ ㅡ <SEP> ~ <SEP> <SPC> ㅅ ㅓ ㄴ <SEP> ㅈ ㅗ <SEP> ...,ㄱ ㅡ <SEP> ~ <SEP> <SPC> ㅅ ㅓ ㄴ <SEP> ㅈ ㅗ <SEP> ...,50대,DCNA20000036.json,7
227999,ㅇ ㅣ ㄴ <SEP> ㅈ ㅔ <SEP> <SPC> ㅊ ㅓ ㄹ <SEP> ㄷ ㅡ ㄹ ...,ㅇ ㅣ ㄴ <SEP> ㅈ ㅔ <SEP> <SPC> ㅊ ㅓ ㄹ <SEP> ㄷ ㅡ ㄹ ...,50대,DCNA20000036.json,9
228001,ㅇ ㅓ <SEP> ㄸ ㅓ ㅎ <SEP> ㄱ ㅔ <SEP> <SPC> ㅂ ㅗ <SEP...,ㅇ ㅓ <SEP> ㄸ ㅓ ㅎ <SEP> ㄱ ㅔ <SEP> <SPC> ㅂ ㅗ <SEP...,50대,DCNA20000036.json,11


In [10]:
df.to_csv("../datas/output/chungcheong_dialect_test_jamo.csv")

### Byte 수준 BPE

In [2]:
df = pd.read_csv("../datas/output/chungcheong_dialect_data_age_sampled.csv", index_col=0)
df

,방언,표준어,연령대,출처 파일,길이
0,영화보다는 그런 쪽으로 쫌 많이 다녀서,영화보다는 그런 쪽으로 조금 많이 다녀서,50대,DCNA20001462.json,6
1,에~ 저는 원래 고등학생 때까지,에~ 저는 원래 고등학생 때까지,20대,DCDG20000795.json,5
2,그 companyname 프로그램 이걸 많이 보거든.,그 companyname 프로그램 이걸 많이 보거든.,20대,DCDG20001601.json,6
3,고위험 그~ 군이 아니야 근데 만약에 그게 임신성 당뇨로 판정이 되면은 그 엄마는 ...,고위험 그~ 군이 아니야 근데 만약에 그게 임신성 당뇨로 판정이 되면은 그 엄마는 ...,40대,DCEX20001058.json,21
4,코로나 때문에 사람들이 많이 집에 있고 이제 이릏게 했으니까,코로나 때문에 사람들이 많이 집에 있고 이제 이렇게 했으니까,40대,DCNA20001390.json,9
...,...,...,...,...,...
520983,거기서 밥을 해면 뭐 누룽지를 해야 되네 뭐 하네 막 이러는데 밥 할 것이 없어,거기서 밥을 해면 뭐 누룽지를 해야 되네 뭐 하네 막 이러는데 밥 할 것이 없어,60대 이상,DCNA20001746.json,15
520984,그~ 욕심을 부려서 그려 욕심을 부려 갖고,그~ 욕심을 부려서 그래 욕심을 부려 갖고,60대 이상,DCNA20001459.json,7
520985,그~ 낮아졌다는 게 쫌 그렇게 어~ 한 게 아니라,그~ 낮아졌다는 게 조금 그렇게 어~ 한 게 아니라,50대,DCDG20000849.json,9
520986,사람들 퀸와사비 처음 나왔을 때도 완전 문화충격이어가지구,사람들 퀸와사비 처음 나왔을 때도 완전 문화충격이어가지구,20대,DCNA20000770.json,7


In [3]:
from tokenizers import ByteLevelBPETokenizer

bpe_tokenizer = ByteLevelBPETokenizer()
bpe_tokenizer.train(
    files="../datas/output/chungcheong_dialect_data_text.txt",
)

In [4]:
code_result_dialect = bpe_tokenizer.encode_batch(df["방언"])
code_result_standard = bpe_tokenizer.encode_batch(df["표준어"])

In [5]:
df.loc[df.index, "방언"] = [' '.join(res.tokens) for res in code_result_dialect]
df["방언"]

0         ìĺģíĻĶ ë³´ëĭ¤ëĬĶ Ġê·¸ëŁ° Ġìª½ìľ¼ë¡ľ Ġì«Į Ġë§İì...
1            ìĹĲ ~ ĠìłĢëĬĶ ĠìĽĲëŀĺ Ġê³łëĵ±íķĻìĥĿ ĠëķĮê¹Įì§Ģ
2         ê·¸ Ġcompanyname ĠíĶĦë¡ľê·¸ëŀ¨ ĠìĿ´ê±¸ Ġë§İìĿ´...
3         ê³ł ìľĦíĹĺ Ġê·¸ ~ Ġêµ° ìĿ´ ĠìķĦëĭĪìķ¼ Ġê·¼ëį° ...
4         ì½Ķë¡ľëĤĺ ĠëķĮë¬¸ìĹĲ ĠìĤ¬ëŀĮëĵ¤ìĿ´ Ġë§İìĿ´ Ġì§...
                                ...                        
520983    ê±°ê¸°ìĦľ Ġë°¥ìĿĦ Ġíķ´ë©´ ĠëŃĲ ĠëĪĦë£½ ì§Ģë¥¼ ...
520984    ê·¸ ~ Ġìļķìĭ¬ìĿĦ Ġë¶Ģ ëł¤ìĦľ Ġê·¸ëł¤ Ġìļķìĭ¬ìĿ...
520985    ê·¸ ~ ĠëĤ®ìķĦ ì¡Į ëĭ¤ëĬĶ Ġê²Į Ġì«Į Ġê·¸ëłĩê²Į ...
520986    ìĤ¬ëŀĮëĵ¤ ĠíĢ¸ ìĻĢ ìĤ¬ ë¹Ħ Ġì²ĺìĿĮ ĠëĤĺìĻĶìĿĦ ...
520987    ê·¸ëĤĺë§Ī Ġë°ľëĿ¼ëĵľ Ġì¤ĳ ìĹĶ Ġìłľê°Ģ ĠìĺĽëĤł ...
Name: 방언, Length: 520988, dtype: object

In [6]:
df.loc[df.index, "표준어"] = [' '.join(res.tokens) for res in code_result_standard]
df["표준어"]

0         ìĺģíĻĶ ë³´ëĭ¤ëĬĶ Ġê·¸ëŁ° Ġìª½ìľ¼ë¡ľ Ġì¡°ê¸Ī Ġë...
1            ìĹĲ ~ ĠìłĢëĬĶ ĠìĽĲëŀĺ Ġê³łëĵ±íķĻìĥĿ ĠëķĮê¹Įì§Ģ
2         ê·¸ Ġcompanyname ĠíĶĦë¡ľê·¸ëŀ¨ ĠìĿ´ê±¸ Ġë§İìĿ´...
3         ê³ł ìľĦíĹĺ Ġê·¸ ~ Ġêµ° ìĿ´ ĠìķĦëĭĪìķ¼ Ġê·¼ëį° ...
4         ì½Ķë¡ľëĤĺ ĠëķĮë¬¸ìĹĲ ĠìĤ¬ëŀĮëĵ¤ìĿ´ Ġë§İìĿ´ Ġì§...
                                ...                        
520983    ê±°ê¸°ìĦľ Ġë°¥ìĿĦ Ġíķ´ë©´ ĠëŃĲ ĠëĪĦë£½ ì§Ģë¥¼ ...
520984    ê·¸ ~ Ġìļķìĭ¬ìĿĦ Ġë¶Ģ ëł¤ìĦľ Ġê·¸ëŀĺ Ġìļķìĭ¬ìĿ...
520985    ê·¸ ~ ĠëĤ®ìķĦ ì¡Į ëĭ¤ëĬĶ Ġê²Į Ġì¡°ê¸Ī Ġê·¸ëłĩê...
520986    ìĤ¬ëŀĮëĵ¤ ĠíĢ¸ ìĻĢ ìĤ¬ ë¹Ħ Ġì²ĺìĿĮ ĠëĤĺìĻĶìĿĦ ...
520987    ê·¸ëĤĺë§Ī Ġë°ľëĿ¼ëĵľ Ġì¤ĳ ìĹĶ Ġìłľê°Ģ ĠìĺĽëĤł ...
Name: 표준어, Length: 520988, dtype: object

In [7]:
df.to_csv("../datas/output/chungcheong_dialect_data_bpe.csv")

### 테스트셋 BPE

In [8]:
df = pd.read_csv("../datas/output/chungcheong_dialect_test_age_processed.csv", index_col=0)
df

,방언,표준어,연령대,출처 파일,길이
0,우리집 거북이 용봉이가 지금 십팔 년에서 십구 년차 되잖아.,우리집 거북이 용봉이가 지금 십팔 년에서 십구 년차 되잖아.,50대,DCDG20000009.json,9
2,어~ 아파서 고비가 있었는데 지금은 그걸 잘 극복해 가지고 건강하게 우리 식구랑 같...,어~ 아파서 고비가 있었는데 지금은 그걸 잘 극복해 가지고 건강하게 우리 식구랑 같...,50대,DCDG20000009.json,17
3,근데 엄마가 너무 용봉이를 예뻐한다고 니가 엄청 질투가 많은데,근데 엄마가 너무 용봉이를 예뻐한다고 니가 엄청 질투가 많은데,50대,DCDG20000009.json,9
4,그래도 용봉이가 가족 일원으로서 예쁘지?,그래도 용봉이가 가족 일원으로서 예쁘지?,50대,DCDG20000009.json,5
5,아~ 예쁘긴 한데 엄마는 너무 나보다는 용봉이를 좀 더 많이 챙기지 않나 하는 생각...,아~ 예쁘긴 한데 엄마는 너무 나보다는 용봉이를 좀 더 많이 챙기지 않나 하는 생각...,20대,DCDG20000009.json,15
...,...,...,...,...,...
227997,이제 그런 소리가 예전에는 아후 그런게 무슨 와닿지 않았는데 지금은 인제,이제 그런 소리가 예전에는 아후 그런게 무슨 와닿지 않았는데 지금은 인제,50대,DCNA20000036.json,11
227998,그~ 선조들이 했던 그 말씀들이 새록새록 와닿아요.,그~ 선조들이 했던 그 말씀들이 새록새록 와닿아요.,50대,DCNA20000036.json,7
227999,인제 철들자 잉~ 저기 한다고 쪼꼼 철이 드는지 어쩐지,인제 철들자 잉~ 저기 한다고 조금 철이 드는지 어쩐지,50대,DCNA20000036.json,9
228001,어떻게 보면 선생님 지금 그게 오히려 인제 마음이 여유로워 졌기 때문에,어떻게 보면 선생님 지금 그게 오히려 인제 마음이 여유로워 졌기 때문에,50대,DCNA20000036.json,11


In [9]:
from tokenizers import ByteLevelBPETokenizer

bpe_tokenizer = ByteLevelBPETokenizer()
bpe_tokenizer.train(
    files="../datas/output/chungcheong_dialect_test_text.txt",
)

bpe_tokenizer.save_model("./")

['./vocab.json', './merges.txt']

In [10]:
code_result_dialect = bpe_tokenizer.encode_batch(df["방언"])
code_result_standard = bpe_tokenizer.encode_batch(df["표준어"])

In [11]:
df.loc[df.index, "방언"] = [' '.join(res.tokens) for res in code_result_dialect]
df["방언"]

0         ìļ°ë¦¬ ì§ĳ Ġê±°ë¶ģìĿ´ Ġìļ©ë´īìĿ´ê°Ģ Ġì§Ģê¸Ī Ġì...
2         ìĸ´ ~ ĠìķĦíĮĮìĦľ Ġê³ł ë¹Ħê°Ģ ĠìŀĪìĹĪëĬĶëį° Ġì§...
3         ê·¼ëį° ĠìĹĦë§Īê°Ģ ĠëĦĪë¬´ Ġìļ©ë´ī ìĿ´ë¥¼ ĠìĺĪë...
4         ê·¸ëŀĺëıĦ Ġìļ©ë´īìĿ´ê°Ģ Ġê°Ģì¡± ĠìĿ¼ ìĽĲ ìľ¼ë¡...
5         ìķĦ ~ ĠìĺĪìģĺ ê¸´ Ġíķľëį° ĠìĹĦë§ĪëĬĶ ĠëĦĪë¬´ Ġ...
                                ...                        
227997    ìĿ´ìłľ Ġê·¸ëŁ° ĠìĨĮë¦¬ê°Ģ ĠìĺĪìłĦìĹĲëĬĶ ĠìķĦíĽ...
227998    ê·¸ ~ ĠìĦł ì¡° ëĵ¤ìĿ´ ĠíĸĪëįĺ Ġê·¸ Ġë§ĲìĶĢ ëĵ¤...
227999    ìĿ¸ìłľ Ġì²ł ëĵ¤ ìŀĲ Ġìŀī ~ ĠìłĢê¸° Ġíķľëĭ¤ê³ł ...
228001    ìĸ´ëĸ»ê²Į Ġë³´ë©´ ĠìĦłìĥĿëĭĺ Ġì§Ģê¸Ī Ġê·¸ê²Į Ġ...
228002    ìĿ´ê²Į Ġì¢Ģ ĠëĦĵê²Į Ġë³¼ ĠìĪĺ ĠìŀĪëĬĶê²Į ĠìķĦë...
Name: 방언, Length: 190001, dtype: object

In [12]:
df.loc[df.index, "표준어"] = [' '.join(res.tokens) for res in code_result_standard]
df["표준어"]

0         ìļ°ë¦¬ ì§ĳ Ġê±°ë¶ģìĿ´ Ġìļ©ë´īìĿ´ê°Ģ Ġì§Ģê¸Ī Ġì...
2         ìĸ´ ~ ĠìķĦíĮĮìĦľ Ġê³ł ë¹Ħê°Ģ ĠìŀĪìĹĪëĬĶëį° Ġì§...
3         ê·¼ëį° ĠìĹĦë§Īê°Ģ ĠëĦĪë¬´ Ġìļ©ë´ī ìĿ´ë¥¼ ĠìĺĪë...
4         ê·¸ëŀĺëıĦ Ġìļ©ë´īìĿ´ê°Ģ Ġê°Ģì¡± ĠìĿ¼ ìĽĲ ìľ¼ë¡...
5         ìķĦ ~ ĠìĺĪìģĺ ê¸´ Ġíķľëį° ĠìĹĦë§ĪëĬĶ ĠëĦĪë¬´ Ġ...
                                ...                        
227997    ìĿ´ìłľ Ġê·¸ëŁ° ĠìĨĮë¦¬ê°Ģ ĠìĺĪìłĦìĹĲëĬĶ ĠìķĦíĽ...
227998    ê·¸ ~ ĠìĦł ì¡° ëĵ¤ìĿ´ ĠíĸĪëįĺ Ġê·¸ Ġë§ĲìĶĢ ëĵ¤...
227999    ìĿ¸ìłľ Ġì²ł ëĵ¤ ìŀĲ Ġìŀī ~ ĠìłĢê¸° Ġíķľëĭ¤ê³ł ...
228001    ìĸ´ëĸ»ê²Į Ġë³´ë©´ ĠìĦłìĥĿëĭĺ Ġì§Ģê¸Ī Ġê·¸ê²Į Ġ...
228002    ìĿ´ê²Į Ġì¢Ģ ĠëĦĵê²Į Ġë³¼ ĠìĪĺ ĠìŀĪëĬĶê²Į ĠìķĦë...
Name: 표준어, Length: 190001, dtype: object

In [13]:
df.to_csv("../datas/output/chungcheong_dialect_test_bpe.csv")

## 제주 방언

In [21]:
df = pd.read_csv("../datas/output/jeju_dialect_train_age.csv", index_col=0)
df

,방언,표준어,연령대,출처 파일
0,난 진짜 벌초 할때 뱀나오잖아 뱀도,난 진짜 벌초 할때 뱀나오잖아 뱀도,50대,DZES20000002.json
1,어어 그니까,어어 그니까,50대,DZES20000002.json
2,아 잘도,아 잘도,50대,DZES20000002.json
3,예초기 뱀도 짤려,예초기 뱀도 짤려,50대,DZES20000002.json
4,어 그니까 완전 끔찍해,어 그니까 완전 끔찍해,50대,DZES20000002.json
...,...,...,...,...
2774252,&name3&이 &name3&이가 그거 아니 쬐그만한 거 가지고 있던데,&name3&이 &name3&이가 그거 아니야 작은 거 가지고 있던데,30대,DZJD21002407.json
2774253,어 맞아 나 좀 쬐끄만한디,어 맞아 나 좀 작은데,30대,DZJD21002407.json
2774254,네가 그거인 거 닮.,네가 그거인 거 같아.,30대,DZJD21002407.json
2774255,아 여행용(인게)/(이네),아 여행용(인게)/(이네),30대,DZJD21002407.json


In [37]:
# 테스트 데이터로 동일한 과정을 진행하고 싶을 때
df = pd.read_csv("../datas/output/jeju_dialect_test_age.csv", index_col=0)
df

,방언,표준어,연령대,출처 파일
0,언니 만낭 반가워 아 오늘 제주 방언 에이 아이 데이터,언니 만나서 반가워 아 오늘 제주 방언 에이 아이 데이터,50대,DZES20000001.json
1,어 그거 대화 주제는 오늘 대화 주제는,어 그거 대화 주제는 오늘 대화 주제는,50대,DZES20000001.json
2,어 명절 설 명절 추석 명절 요로케 나누어서 해볼 거예,어 명절 설 명절 추석 명절 요로케 나누어서 해볼 거예요,50대,DZES20000001.json
3,자 그믄 이제부터 얘기해 보게예,자 그러면 이제부터 얘기해 봐요,50대,DZES20000001.json
4,어,어,50대,DZES20000001.json
...,...,...,...,...
333797,마무리 해그넹 저녁에 잠깐 나가크라.,마무리 해서 저녁에 잠깐 나갈게.,30대,DZJD21002409.json
333798,어 겅 하라,어 그렇게 해라,30대,DZJD21002409.json
333799,카페나 가게.,카페나 가자.,30대,DZJD21002409.json
333800,난 차 어시난 우리 집 못가,난 차 없으니까 우리 집 못가,30대,DZJD21002409.json


In [38]:
df[df["방언"].str.contains("\(\(\)\)")|df["표준어"].str.contains("\(\(\)\)")]

,방언,표준어,연령대,출처 파일
68,아 {laughing} (()),아 {laughing} (()),50대,DZES20000001.json
123,아 (()),아 (()),50대,DZES20000001.json
188,야 화장실에 가사 저기 방귀 (()),야 화장실에 가야 저기 방귀 (()),50대,DZES20000001.json
189,신문 찾아 (()) {laughing} 해그내,신문 찾아 (()) {laughing} 해서,50대,DZES20000001.json
224,맨날 그 (()) 그니까 보면 차례를 지낸 것도 우리는,맨날 그 (()) 그니까 보면 차례를 지낸 것도 우리는,50대,DZES20000001.json
...,...,...,...,...
333601,(()) 할 까 생각중이거든,(()) 할 까 생각중이거든,30대,DZJD21002409.json
333650,이게 안돼 얘가 충전이 안 돼 그러니까 즉 헬스장 가고 그때 옛날에 코로나 전에 헬...,이게 안돼 얘가 충전이 안 돼 그러니까 즉 헬스장 가고 그때 옛날에 코로나 전에 헬...,30대,DZJD21002409.json
333671,막 흘러 내릴 때도 있고 가다가 (()),막 흘러 내릴 때도 있고 가다가 (()),30대,DZJD21002409.json
333679,그 저 런닝머신에 간병이 그 맞 꼽아질거네 런닝머신에 같이 (()) 그거 터져가지고,그 저 런닝머신에 간병이 그 맞 꼽아질거네 런닝머신에 같이 (()) 그거 터져가지고,30대,DZJD21002409.json


In [39]:
df["방언"] = df["방언"].str.replace(r"\(\(\)\)", '', regex=True)
df["표준어"] = df["표준어"].str.replace(r"\(\(\)\)", '', regex=True)

In [40]:
phrase_words = df["방언"].str.extract(r"\(\((.*)\)\)", expand=False).dropna()
df.loc[phrase_words.index]

,방언,표준어,연령대,출처 파일
84,탁 잡아가고 영 영 이렇게 ((이렇게)) 서로 영,탁 잡아가고 이렇게 이렇게 이렇게 ((이렇게)) 서로 이렇게,50대,DZES20000001.json
116,뭐 음식은 뭐 그냥 그냥 주는대로 ((주슨대로)) 먹는거고,뭐 음식은 뭐 그냥 그냥 주는대로 ((주슨대로)) 먹는거고,20대,DZES20000001.json
178,고 정도 ((보였주게)),고 정도 ((보였주게)),50대,DZES20000001.json
204,((그지)) 딸*들은 참여 안 시켜주잖아,((그지)) 딸들은 참여 안 시켜주잖아,50대,DZES20000001.json
233,또 그리고 또 그런 거 ((있잖아)) 이런 거 있잖아 세배전에,또 그리고 또 그런 거 ((있잖아)) 이런 거 있잖아 세배전에,50대,DZES20000001.json
...,...,...,...,...
332502,뭘 ((바라는건데?)),뭘 ((바라는건데?)),30대,DZJD21002406.json
332507,((멀직이)) ((따라잡으면)),((멀직이)) ((따라잡으면)),30대,DZJD21002406.json
332513,너 지도자 과정 ((듣는다고?)) 자격증 딸 거라?,너 지도자 과정 ((듣는다고?)) 자격증 딸 거야?,30대,DZJD21002406.json
332523,게난이. 집중해 ((이러면서)) {laughing},그러게. 집중해 ((이러면서)) {laughing},30대,DZJD21002406.json


In [41]:
phrase_words = df["방언"].str.extract(r"\(\((.*)\)\)", expand=False).dropna()
for i in phrase_words.index:
    sentence = df.loc[i, "방언"]
    sentence = re.sub(r"\(\(.*\)\)", phrase_words[i], sentence)
    df.loc[i, "방언"] = sentence

phrase_words = df["표준어"].str.extract(r"\(\((.*)\)\)", expand=False).dropna()
for i in phrase_words.index:
    sentence = df.loc[i, "표준어"]
    sentence = re.sub(r"\(\(.*\)\)", phrase_words[i], sentence)
    df.loc[i, "표준어"] = sentence

df.loc[phrase_words.index]

,방언,표준어,연령대,출처 파일
84,탁 잡아가고 영 영 이렇게 이렇게 서로 영,탁 잡아가고 이렇게 이렇게 이렇게 이렇게 서로 이렇게,50대,DZES20000001.json
116,뭐 음식은 뭐 그냥 그냥 주는대로 주슨대로 먹는거고,뭐 음식은 뭐 그냥 그냥 주는대로 주슨대로 먹는거고,20대,DZES20000001.json
178,고 정도 보였주게,고 정도 보였주게,50대,DZES20000001.json
204,그지 딸*들은 참여 안 시켜주잖아,그지 딸들은 참여 안 시켜주잖아,50대,DZES20000001.json
233,또 그리고 또 그런 거 있잖아 이런 거 있잖아 세배전에,또 그리고 또 그런 거 있잖아 이런 거 있잖아 세배전에,50대,DZES20000001.json
...,...,...,...,...
332502,뭘 바라는건데?,뭘 바라는건데?,30대,DZJD21002406.json
332507,멀직이)) ((따라잡으면,멀직이)) ((따라잡으면,30대,DZJD21002406.json
332513,너 지도자 과정 듣는다고? 자격증 딸 거라?,너 지도자 과정 듣는다고? 자격증 딸 거야?,30대,DZJD21002406.json
332523,게난이. 집중해 이러면서 {laughing},그러게. 집중해 이러면서 {laughing},30대,DZJD21002406.json


In [42]:
phrase_words = df["방언"].str.extract(r"\((.*)\)\/\(.*\)", expand=False).dropna()
df.loc[phrase_words.index]

,방언,표준어,연령대,출처 파일
920,어려운 때도있고 기쁠때도있고 슬플때도 있는(거라)/(거야),어려운 때도있고 기쁠때도있고 슬플때도 있는(거라)/(거야),60대 이상,DZES20000004.json
929,영허믄 이거 우리(밭이여)/(밭이야),이렇게하면 이거 우리(밭이여)/(밭이야),60대 이상,DZES20000004.json
931,그저(이)/(#이) 가정에 밥,그저(이)/(#이) 가정에 밥,60대 이상,DZES20000004.json
938,여자들은 사는 건 힘들었어 애기도 낳고 밭에(강)/(가서) 검질도메고,여자들은 사는 건 힘들었어 애기도 낳고 밭에(강)/(가서) 잡초도뽑고,60대 이상,DZES20000004.json
940,돌멩이 한 몇 개를 놔서 그(우에)/(위에) 솥을 앉쳐서 밥을 했거든?,돌멩이 한 몇 개를 놓고 그(우에)/(위에) 솥을 앉쳐서 밥을 했거든?,60대 이상,DZES20000004.json
...,...,...,...,...
331891,거기에 옛날에 그 그 무슨 마트 옆에 자그맣게 해그넹 치킨도 팔고 누나 거기 치킨 ...,거기에 옛날에 그 그 무슨 마트 옆에 조그맣게 해서 치킨도 팔고 누나 거기 치킨 엄...,30대,DZJD21002405.json
332473,막 그리고 또 막 나 열심히 하고 -(이시면)/(있으면)-,막 그리고 또 막 나 열심히 하고 -(이시면)/(있으면)-,30대,DZJD21002406.json
332715,한번 올라오라(게)/(#게),한번 올라오라(게)/(#게),30대,DZJD21002408.json
332874,아(예)/(#예) 삼춘 게 뭔일마씨. 영 하멍,아(예)/(#예) 아저씨 #게 무슨일이에요. 이렇게 하면서,30대,DZJD21002408.json


'#'이 있는 경우는 표준어에서 해당 문자열 전체를 삭제하는 것이 말이 된다.

In [43]:
phrase_words = df["방언"].str.extract(r"\(([^\)]+)\)\/\([^\)]+\)", expand=False).dropna()
for i in phrase_words.index:
    sentence = df.loc[i, "방언"]
    group_matches = re.findall(r"\(([^\)]+)\)\/\([^\)]+\)", sentence)
    rep_matches = re.findall(r"\([^\)]+\)\/\([^\)]+\)", sentence)
    for group, rep in zip(group_matches, rep_matches):
        sentence = sentence.replace(rep, group)
    df.loc[i, "방언"] = sentence

phrase_words = df["표준어"].str.extract(r"\([^\)]+\)\/\(([^\)]+)\)", expand=False).dropna()
for i in phrase_words.index:
    sentence = df.loc[i, "표준어"]
    group_matches = re.findall(r"\([^\)]+\)\/\(([^\)]+)\)", sentence)
    rep_matches = re.findall(r"\([^\)]+\)\/\([^\)]+\)", sentence)
    for group, rep in zip(group_matches, rep_matches):
        if group[0] == '#':
            sentence = sentence.replace(rep, '')
        else:
            sentence = sentence.replace(rep, group)
    df.loc[i, "표준어"] = sentence

df.loc[phrase_words.index]

,방언,표준어,연령대,출처 파일
920,어려운 때도있고 기쁠때도있고 슬플때도 있는거라,어려운 때도있고 기쁠때도있고 슬플때도 있는거야,60대 이상,DZES20000004.json
929,영허믄 이거 우리밭이여,이렇게하면 이거 우리밭이야,60대 이상,DZES20000004.json
931,그저이 가정에 밥,그저 가정에 밥,60대 이상,DZES20000004.json
938,여자들은 사는 건 힘들었어 애기도 낳고 밭에강 검질도메고,여자들은 사는 건 힘들었어 애기도 낳고 밭에가서 잡초도뽑고,60대 이상,DZES20000004.json
940,돌멩이 한 몇 개를 놔서 그우에 솥을 앉쳐서 밥을 했거든?,돌멩이 한 몇 개를 놓고 그위에 솥을 앉쳐서 밥을 했거든?,60대 이상,DZES20000004.json
...,...,...,...,...
331891,거기에 옛날에 그 그 무슨 마트 옆에 자그맣게 해그넹 치킨도 팔고 누나 거기 치킨 ...,거기에 옛날에 그 그 무슨 마트 옆에 조그맣게 해서 치킨도 팔고 누나 거기 치킨 엄...,30대,DZJD21002405.json
332473,막 그리고 또 막 나 열심히 하고 -이시면-,막 그리고 또 막 나 열심히 하고 -있으면-,30대,DZJD21002406.json
332715,한번 올라오라게,한번 올라오라,30대,DZJD21002408.json
332874,아예 삼춘 게 뭔일마씨. 영 하멍,아 아저씨 #게 무슨일이에요. 이렇게 하면서,30대,DZJD21002408.json


In [44]:
laugh_index = df["방언"].str.contains("\{laughing\}")

df["방언"] = df["방언"].str.replace(r"\{laughing\}", "", regex=True)
df["표준어"] = df["표준어"].str.replace(r"\{laughing\}", "", regex=True)
df[laugh_index]

,방언,표준어,연령대,출처 파일
17,밀가룬지 쌀가룬지를 모르크라 근데 그거를 푹,밀가룬지 쌀가룬지를 모르겠어 근데 그거를 푹,50대,DZES20000001.json
68,아,아,50대,DZES20000001.json
70,안 봐봔,안 봤었어,50대,DZES20000001.json
72,어,어,50대,DZES20000001.json
109,아 세뱃돈 어어어어,아 세뱃돈 어어어어,50대,DZES20000001.json
...,...,...,...,...
332544,얘기하는 내용도 약간 좀 영적인 거 라 해야되나,얘기하는 내용도 약간 좀 영적인 거 라 해야되나,30대,DZJD21002406.json
333090,어 _,어 _,30대,DZJD21002408.json
333114,,,30대,DZJD21002408.json
333169,아,아,30대,DZJD21002408.json


In [45]:
stutter_sentence = df["방언"].str.contains(r"\-.*\-", regex=True)
df["방언"] = df["방언"].str.replace(r"\-.*\-", "", regex=True)
df["표준어"] = df["표준어"].str.replace(r"\-.*\-", "", regex=True)

df.loc[stutter_sentence]

,방언,표준어,연령대,출처 파일
1756,번개 치고 막 우르릉 허잖아.,번개 치고 막 우르릉 하잖아.,60대 이상,DZES20000010.json
1847,우산을 손에 들잖아,우산을 손에 들잖아,50대,DZES20000010.json
2249,이거 좀 고장난 거를,이거 좀 고장난 거를,50대,DZES20000010.json
7979,아 &companyname1&,아 &companyname1&,60대 이상,DZES20000025.json
8099,그런데 이런 거 진짜 다 듣는 건가,그런데 이런 거 진짜 다 듣는 건가,20대,DZES20000039.json
...,...,...,...,...
330511,&companyname4& 밖에 몰라,&companyname4& 밖에 몰라,30대,DZJD21002393.json
330526,어 &companyname3&,어 &companyname3&,30대,DZJD21002393.json
331959,사진도 보내주고 하고 하지.,사진도 보내주고 하고 하지.,30대,DZJD21002406.json
332163,수동 사믄 이제 자동도 사는 사는 거여.,수동 사면 이제 자동도 사는 사는 거야.,30대,DZJD21002406.json


In [46]:
대체어_위치_방언 = df["방언"].str.contains(r"\&\S+[0-9]\&", regex=True)
대체어_위치_표준어 = df["표준어"].str.contains(r"\&\S+[0-9]\&", regex=True)

In [47]:
df["방언"] = df["방언"].str.replace(r"\&name[0-9]+\&", "name", regex=True)
df["방언"] = df["방언"].str.replace(r"\&address[0-9]+\&", "address", regex=True)
df["방언"] = df["방언"].str.replace(r"\&company-name[0-9]+\&", "companyname", regex=True)
df["방언"] = df["방언"].str.replace(r"\&companyname[0-9]+\&", "companyname", regex=True)

df[대체어_위치_방언]

,방언,표준어,연령대,출처 파일
105,name이는 저기 너네 또래 애들은 명절때 어떤 음식 제일 좋아라 해?,&name1&이는 저기 너네 또래 애들은 명절때 어떤 음식 제일 좋아라 해?,50대,DZES20000001.json
152,name이도 그랬잖아 이,&name1&이도 그랬잖아 #이,50대,DZES20000001.json
276,name이는 들으면 생소하켜게,&name1&이는 들으면 생소하겠다,50대,DZES20000001.json
622,이 그 뭐니게 name가 끼면은,이 그 뭐니 &name2&가 끼면은,50대,DZES20000001.json
838,아이고 name이 혼자 다 하게 생겼네?,아이고 &name1&이 혼자 다 하게 생겼네?,50대,DZES20000001.json
...,...,...,...,...
332842,통역하랜 name민쌤 내려오셔야 될 것 같은데요,통역하라고 &name1&민쌤 내려오셔야 될 것 같은데요,30대,DZJD21002408.json
332949,야 그 name 이 시험 합격 해신가? 거 결과 언제 나온댄?,야 그 &name4& 이 시험 합격 했나? 거 결과 언제 나온대?,30대,DZJD21002408.json
332962,name이 떨어졌댄 전화 와서.,&name5&이 떨어졌다고 전화 왔어.,30대,DZJD21002408.json
332963,누구? name?,누구? &name5&?,30대,DZJD21002408.json


In [48]:
df["표준어"] = df["표준어"].str.replace(r"\&name[0-9]\&", "name", regex=True)
df["표준어"] = df["표준어"].str.replace(r"\&address[0-9]\&", "address", regex=True)
df["표준어"] = df["표준어"].str.replace(r"\&company-name[0-9]\&", "companyname", regex=True)
df["표준어"] = df["표준어"].str.replace(r"\&companyname[0-9]\&", "companyname", regex=True)

df[대체어_위치_표준어]

,방언,표준어,연령대,출처 파일
105,name이는 저기 너네 또래 애들은 명절때 어떤 음식 제일 좋아라 해?,name이는 저기 너네 또래 애들은 명절때 어떤 음식 제일 좋아라 해?,50대,DZES20000001.json
152,name이도 그랬잖아 이,name이도 그랬잖아 #이,50대,DZES20000001.json
276,name이는 들으면 생소하켜게,name이는 들으면 생소하겠다,50대,DZES20000001.json
622,이 그 뭐니게 name가 끼면은,이 그 뭐니 name가 끼면은,50대,DZES20000001.json
838,아이고 name이 혼자 다 하게 생겼네?,아이고 name이 혼자 다 하게 생겼네?,50대,DZES20000001.json
...,...,...,...,...
332842,통역하랜 name민쌤 내려오셔야 될 것 같은데요,통역하라고 name민쌤 내려오셔야 될 것 같은데요,30대,DZJD21002408.json
332949,야 그 name 이 시험 합격 해신가? 거 결과 언제 나온댄?,야 그 name 이 시험 합격 했나? 거 결과 언제 나온대?,30대,DZJD21002408.json
332962,name이 떨어졌댄 전화 와서.,name이 떨어졌다고 전화 왔어.,30대,DZJD21002408.json
332963,누구? name?,누구? name?,30대,DZJD21002408.json


### 데이터 길이

In [49]:
df["길이"] = [min(len(dia.split()), len(stan.split())) for dia, stan in zip(df["방언"], df["표준어"])]
df["길이"]

0         10
1          7
2         10
3          5
4          1
          ..
333797     5
333798     3
333799     2
333800     6
333801     2
Name: 길이, Length: 333802, dtype: int64

In [50]:
suf_len_df = df[df["길이"] > 4]
len(suf_len_df)

160123

In [51]:
dia_sur_len = len(suf_len_df[suf_len_df["방언"] != suf_len_df["표준어"]])
print("길이로 제외한 후 남은 방언 샘플 수: {0}".format(dia_sur_len))

길이로 제외한 후 남은 방언 샘플 수: 85685


In [52]:
suf_len_df.to_csv("../datas/output/jeju_dialect_test_age_processed.csv")

In [36]:
suf_len_df.to_csv("../datas/output/jeju_dialect_data_age_processed.csv")

### 1:1 샘플링 처리

In [53]:
suf_len_df = pd.read_csv("../datas/output/jeju_dialect_data_age_processed.csv", index_col=0)
suf_len_df

,방언,표준어,연령대,출처 파일,길이
0,난 진짜 벌초 할때 뱀나오잖아 뱀도,난 진짜 벌초 할때 뱀나오잖아 뱀도,50대,DZES20000002.json,6
9,아 예초하다가이 뱀 나온거야 뱀이 하나 도 아니고,아 예초하다가 뱀 나온거야 뱀이 하나 도 아니고,50대,DZES20000002.json,8
11,아 겅하니까이 예초 할땐 조심해야 돼,아 그러니까 예초 할땐 조심해야 돼,50대,DZES20000002.json,6
14,저기 돌멩이 같은 거 있잖아,저기 돌멩이 같은 거 있잖아,50대,DZES20000002.json,5
17,나 아는 사람이 벌 이거이 그거 벌초하다가 예초기가,나 아는 사람이 벌 이거 그거 벌초하다가 예초기가,50대,DZES20000002.json,8
...,...,...,...,...,...
2774248,어 루미큐브 재밌지 큐브가 어 것도 조커 두 개만 있는 줄 알았는데 여러 가지 있더...,어 루미큐브 재밌지 큐브가 어 그것도 조커 두 개만 있는 줄 알았는데 여러 가지 있...,30대,DZJD21002407.json,16
2774250,색깔 반전 시키는 것도 있댄 하고이,색깔 반전 시키는 것도 있다고 하고,30대,DZJD21002407.json,6
2774251,그리고 요즘 뭐냐 루미큐브도 루미큐브 트래블이라고 해서 여행용으로 다니고 다닐 수도...,그리고 요즘 뭐냐 루미큐브도 루미큐브 트래블이라고 해서 여행용으로 다니고 다닐 수도...,30대,DZJD21002407.json,16
2774252,name이 name이가 그거 아니 쬐그만한 거 가지고 있던데,name이 name이가 그거 아니야 작은 거 가지고 있던데,30대,DZJD21002407.json,8


In [54]:
dial_df = suf_len_df[suf_len_df["방언"] != suf_len_df["표준어"]]
stan_df = suf_len_df[suf_len_df["방언"] == suf_len_df["표준어"]]

In [55]:
print(len(dial_df))
print(len(stan_df))

758384
617051


방언 표본이 더 많기 때문에 전체 데이터 그대로 사용

In [56]:
sampled_df = suf_len_df
sampled_df.to_csv("../datas/output/jeju_dialect_data_age_sampled.csv")

* 제주 방언 필터링 후 훈련 샘플 수: 1375435
* 제주 방언 필터링 후 테스트 샘플 수: 160123

### 형태소 토큰화

In [2]:
import MeCab

def pos(sentence: str):
    """한국어 토큰을 분리합니다. 토큰과 품사를 튜플 리스트로 반환합니다.
        
        매개변수:
            sentence (str): 토큰화할 문장 
        반환값:
            token list (list[tuple]): 토큰과 품사 리스트
    """
    t = MeCab.Tagger()
    tag_result = t.parse(sentence)
    tag_result = tag_result.replace("\t", ".@!").replace("\n", ".@!").split(".@!")
    tag_word = tag_result[::2][:-1] # 마지막 EOS는 자른다
    tag_info = tag_result[1::2][:-1] # 마지막 EOS는 자른다
    return [(word, info.split(',')[0]) for word, info in zip(tag_word, tag_info)]

def morphs(sentence: str):
    """한국어 토큰을 분리합니다. 토큰의 리스트를 반환합니다.
        
        매개변수:
            sentence (str): 토큰화할 문장 
        반환값:
            token list (list): 토큰 리스트
    """
    t = MeCab.Tagger()
    tag_result = t.parse(sentence)
    tag_result = tag_result.replace("\t", ".@!").replace("\n", ".@!").split(".@!")
    return tag_result[::2][:-1]

def space_tokenize(original_sentence, token_list):
    """
    문장을 decode할 때 한국어의 띄어쓰기도 되살릴 수 있도록 token에 이를 반영하도록 처리하는 함수
    """
    
    space_token = original_sentence.split()

    if len(space_token) == 0:
        return token_list

    token_idx = 0
    cur_token = space_token[token_idx]
    cum_word = ""
    for i, word in enumerate(token_list):
        cum_word += word

        if len(cum_word) > len(word):
            token_list[i] = "##" + token_list[i]
            
        if cum_word == cur_token:
            token_idx += 1
            cum_word = ""
            cur_token = space_token[min(token_idx, len(space_token) - 1)]
    
    return token_list

def morph_and_preprocess(sentence: str):
    """한국어 토큰을 분리하고 전처리합니다. 토큰의 리스트를 반환합니다.
        
        매개변수:
            sentence (str): 토큰화할 문장 
        반환값:
            token list (list): 토큰 리스트
    """
    
    pos_result = pos(sentence)
    word_list = [word for word, _ in pos_result]
    word_list = space_tokenize(sentence, word_list) # 공백 복원 토큰화
    
    return word_list

In [3]:
df = pd.read_csv("../datas/output/jeju_dialect_data_age_sampled.csv", index_col=0)
df

,방언,표준어,연령대,출처 파일,길이
0,난 진짜 벌초 할때 뱀나오잖아 뱀도,난 진짜 벌초 할때 뱀나오잖아 뱀도,50대,DZES20000002.json,6
9,아 예초하다가이 뱀 나온거야 뱀이 하나 도 아니고,아 예초하다가 뱀 나온거야 뱀이 하나 도 아니고,50대,DZES20000002.json,8
11,아 겅하니까이 예초 할땐 조심해야 돼,아 그러니까 예초 할땐 조심해야 돼,50대,DZES20000002.json,6
14,저기 돌멩이 같은 거 있잖아,저기 돌멩이 같은 거 있잖아,50대,DZES20000002.json,5
17,나 아는 사람이 벌 이거이 그거 벌초하다가 예초기가,나 아는 사람이 벌 이거 그거 벌초하다가 예초기가,50대,DZES20000002.json,8
...,...,...,...,...,...
2774248,어 루미큐브 재밌지 큐브가 어 것도 조커 두 개만 있는 줄 알았는데 여러 가지 있더...,어 루미큐브 재밌지 큐브가 어 그것도 조커 두 개만 있는 줄 알았는데 여러 가지 있...,30대,DZJD21002407.json,16
2774250,색깔 반전 시키는 것도 있댄 하고이,색깔 반전 시키는 것도 있다고 하고,30대,DZJD21002407.json,6
2774251,그리고 요즘 뭐냐 루미큐브도 루미큐브 트래블이라고 해서 여행용으로 다니고 다닐 수도...,그리고 요즘 뭐냐 루미큐브도 루미큐브 트래블이라고 해서 여행용으로 다니고 다닐 수도...,30대,DZJD21002407.json,16
2774252,name이 name이가 그거 아니 쬐그만한 거 가지고 있던데,name이 name이가 그거 아니야 작은 거 가지고 있던데,30대,DZJD21002407.json,8


In [4]:
dialect_list = df["방언"].to_list()
dialect_list = list(map(morph_and_preprocess, dialect_list))

standard_list = df["표준어"].to_list()
standard_list = list(map(morph_and_preprocess, standard_list))

In [5]:
list_to_spaced_sentence = lambda li : " ".join(li)
df.loc[df.index, "방언"] = list(map(list_to_spaced_sentence, dialect_list))
df.loc[df.index, "표준어"] = list(map(list_to_spaced_sentence, standard_list))

df.head()

,방언,표준어,연령대,출처 파일,길이
0,난 진짜 벌초 할 ##때 뱀 ##나오 ##잖아 뱀 ##도,난 진짜 벌초 할 ##때 뱀 ##나오 ##잖아 뱀 ##도,50대,DZES20000002.json,6
9,아 예초 ##하 ##다가 ##이 뱀 나온 ##거 ##야 뱀 ##이 하나 도 아니 ##고,아 예초 ##하 ##다가 뱀 나온 ##거 ##야 뱀 ##이 하나 도 아니 ##고,50대,DZES20000002.json,8
11,아 겅하니까이 예초 할땐 조심 ##해야 돼,아 그러니까 예초 할땐 조심 ##해야 돼,50대,DZES20000002.json,6
14,저기 돌멩이 같 ##은 거 있 ##잖아,저기 돌멩이 같 ##은 거 있 ##잖아,50대,DZES20000002.json,5
17,나 아 ##는 사람 ##이 벌 이거 ##이 그거 벌초 ##하 ##다가 예초기 ##가,나 아 ##는 사람 ##이 벌 이거 그거 벌초 ##하 ##다가 예초기 ##가,50대,DZES20000002.json,8


In [6]:
df.to_csv("../datas/output/jeju_dialect_data_형태소.csv")

### 테스트셋 형태소

In [7]:
df = pd.read_csv("../datas/output/jeju_dialect_test_age_processed.csv", index_col=0)
df

,방언,표준어,연령대,출처 파일,길이
0,언니 만낭 반가워 아 오늘 제주 방언 에이 아이 데이터,언니 만나서 반가워 아 오늘 제주 방언 에이 아이 데이터,50대,DZES20000001.json,10
1,어 그거 대화 주제는 오늘 대화 주제는,어 그거 대화 주제는 오늘 대화 주제는,50대,DZES20000001.json,7
2,어 명절 설 명절 추석 명절 요로케 나누어서 해볼 거예,어 명절 설 명절 추석 명절 요로케 나누어서 해볼 거예요,50대,DZES20000001.json,10
3,자 그믄 이제부터 얘기해 보게예,자 그러면 이제부터 얘기해 봐요,50대,DZES20000001.json,5
7,어 그믄 언니네 설 명절 때 음식 어떵 해?,어 그러면 언니네 설 명절 때 음식 어떻게 해?,50대,DZES20000001.json,9
...,...,...,...,...,...
333782,일요일 오후 두 시 비행기,일요일 오후 두 시 비행기,30대,DZJD21002409.json,5
333784,가다가 뭐 하면 공항으로 보러 가게.,가다가 뭐 하면 공항으로 보러 가자.,30대,DZJD21002409.json,6
333790,막걸리나 한 잔 바 땡기던가,막걸리나 한 잔 바 땡기던가,30대,DZJD21002409.json,5
333797,마무리 해그넹 저녁에 잠깐 나가크라.,마무리 해서 저녁에 잠깐 나갈게.,30대,DZJD21002409.json,5


In [8]:
dialect_list = df["방언"].to_list()
dialect_list = list(map(morph_and_preprocess, dialect_list))

standard_list = df["표준어"].to_list()
standard_list = list(map(morph_and_preprocess, standard_list))

In [9]:
list_to_spaced_sentence = lambda li : " ".join(li)
df.loc[df.index, "방언"] = list(map(list_to_spaced_sentence, dialect_list))
df.loc[df.index, "표준어"] = list(map(list_to_spaced_sentence, standard_list))

df.head()

,방언,표준어,연령대,출처 파일,길이
0,언니 만 ##낭 반가워 아 오늘 제주 방언 에이 아이 데이터,언니 만나 ##서 반가워 아 오늘 제주 방언 에이 아이 데이터,50대,DZES20000001.json,10
1,어 그거 대화 주제 ##는 오늘 대화 주제 ##는,어 그거 대화 주제 ##는 오늘 대화 주제 ##는,50대,DZES20000001.json,7
2,어 명절 설 명절 추석 명절 요로 ##케 나누 ##어서 해 ##볼 거 ##예,어 명절 설 명절 추석 명절 요로 ##케 나누 ##어서 해 ##볼 거 ##예요,50대,DZES20000001.json,10
3,자 그믄 이제 ##부터 얘기 ##해 보 ##게 ##예,자 그러면 이제 ##부터 얘기 ##해 봐요,50대,DZES20000001.json,5
7,어 그믄 언니 ##네 설 명절 때 음식 어 ##떵 해 ##?,어 그러면 언니 ##네 설 명절 때 음식 어떻게 해 ##?,50대,DZES20000001.json,9


In [10]:
df.to_csv("../datas/output/jeju_dialect_test_형태소.csv")

### SentencePiece 토큰화

In [3]:
df = pd.read_csv("../datas/output/jeju_dialect_data_age_sampled.csv", index_col=0)
df

,방언,표준어,연령대,출처 파일,길이
0,난 진짜 벌초 할때 뱀나오잖아 뱀도,난 진짜 벌초 할때 뱀나오잖아 뱀도,50대,DZES20000002.json,6
9,아 예초하다가이 뱀 나온거야 뱀이 하나 도 아니고,아 예초하다가 뱀 나온거야 뱀이 하나 도 아니고,50대,DZES20000002.json,8
11,아 겅하니까이 예초 할땐 조심해야 돼,아 그러니까 예초 할땐 조심해야 돼,50대,DZES20000002.json,6
14,저기 돌멩이 같은 거 있잖아,저기 돌멩이 같은 거 있잖아,50대,DZES20000002.json,5
17,나 아는 사람이 벌 이거이 그거 벌초하다가 예초기가,나 아는 사람이 벌 이거 그거 벌초하다가 예초기가,50대,DZES20000002.json,8
...,...,...,...,...,...
2774248,어 루미큐브 재밌지 큐브가 어 것도 조커 두 개만 있는 줄 알았는데 여러 가지 있더...,어 루미큐브 재밌지 큐브가 어 그것도 조커 두 개만 있는 줄 알았는데 여러 가지 있...,30대,DZJD21002407.json,16
2774250,색깔 반전 시키는 것도 있댄 하고이,색깔 반전 시키는 것도 있다고 하고,30대,DZJD21002407.json,6
2774251,그리고 요즘 뭐냐 루미큐브도 루미큐브 트래블이라고 해서 여행용으로 다니고 다닐 수도...,그리고 요즘 뭐냐 루미큐브도 루미큐브 트래블이라고 해서 여행용으로 다니고 다닐 수도...,30대,DZJD21002407.json,16
2774252,name이 name이가 그거 아니 쬐그만한 거 가지고 있던데,name이 name이가 그거 아니야 작은 거 가지고 있던데,30대,DZJD21002407.json,8


In [4]:
df = df.dropna(how='any')
with open('../datas/output/jeju_dialect_data_text.txt', 'w', encoding='utf8') as f:
    f.write('\n'.join(df['방언'] + " " + df["표준어"]))

In [5]:
import sentencepiece as spm

spm.SentencePieceTrainer.train(input='../datas/output/jeju_dialect_data_text.txt', model_prefix='jeju_data_sp', vocab_size=30000)

In [6]:
sp_tokenizer = spm.SentencePieceProcessor(model_file='jeju_data_sp.model')

code_result_dialect = sp_tokenizer.encode(df["방언"].to_list(), out_type=str)
code_result_standard = sp_tokenizer.encode(df["표준어"].to_list(), out_type=str)

In [7]:
df.loc[df.index, "방언"] = [' '.join(res) for res in code_result_dialect]
df["방언"]

0                              ▁난 ▁진짜 ▁벌초 ▁할때 ▁뱀 나 오 잖아 ▁뱀 도
9                     ▁아 ▁예 초 하다가 이 ▁뱀 ▁나온거야 ▁뱀이 ▁하나 ▁도 ▁아니고
11                              ▁아 ▁겅하니까 이 ▁예 초 ▁할땐 ▁조심해야 ▁돼
14                                      ▁저기 ▁돌멩이 ▁같은 ▁거 ▁있잖아
17                  ▁나 ▁아는 ▁사람이 ▁벌 ▁이거 이 ▁그거 ▁벌초 하다가 ▁예 초 기가
                                 ...                        
2774248    ▁어 ▁루미큐브 ▁재밌지 ▁큐브 가 ▁어 ▁것도 ▁조커 ▁두 ▁개만 ▁있는 ▁줄 ▁...
2774250                           ▁색깔 ▁반전 ▁시키는 ▁것도 ▁있댄 ▁하고 이
2774251    ▁그리고 ▁요즘 ▁뭐냐 ▁루미큐브 도 ▁루미큐브 ▁트 래 블 이라고 ▁해서 ▁여행 ...
2774252        ▁name 이 ▁name 이가 ▁그거 ▁아니 ▁쬐 그 만한 ▁거 ▁가지고 ▁있던데
2774253                                 ▁어 ▁맞아 ▁나 ▁좀 ▁쬐끄만 한디
Name: 방언, Length: 1375247, dtype: object

In [8]:
df.loc[df.index, "표준어"] = [' '.join(res) for res in code_result_standard]
df["표준어"]

0                              ▁난 ▁진짜 ▁벌초 ▁할때 ▁뱀 나 오 잖아 ▁뱀 도
9                       ▁아 ▁예 초 하다가 ▁뱀 ▁나온거야 ▁뱀이 ▁하나 ▁도 ▁아니고
11                                ▁아 ▁그러니까 ▁예 초 ▁할땐 ▁조심해야 ▁돼
14                                      ▁저기 ▁돌멩이 ▁같은 ▁거 ▁있잖아
17                    ▁나 ▁아는 ▁사람이 ▁벌 ▁이거 ▁그거 ▁벌초 하다가 ▁예 초 기가
                                 ...                        
2774248    ▁어 ▁루미큐브 ▁재밌지 ▁큐브 가 ▁어 ▁그것도 ▁조커 ▁두 ▁개만 ▁있는 ▁줄 ...
2774250                            ▁색깔 ▁반전 ▁시키는 ▁것도 ▁있다고 ▁하고
2774251    ▁그리고 ▁요즘 ▁뭐냐 ▁루미큐브 도 ▁루미큐브 ▁트 래 블 이라고 ▁해서 ▁여행 ...
2774252           ▁name 이 ▁name 이가 ▁그거 ▁아니야 ▁작은 ▁거 ▁가지고 ▁있던데
2774253                                    ▁어 ▁맞아 ▁나 ▁좀 ▁작은데
Name: 표준어, Length: 1375247, dtype: object

In [9]:
df.to_csv("../datas/output/jeju_dialect_data_SentencePiece.csv")

### 테스트셋 SentencePiece

In [10]:
df = pd.read_csv("../datas/output/jeju_dialect_test_age_processed.csv", index_col=0)
df

,방언,표준어,연령대,출처 파일,길이
0,언니 만낭 반가워 아 오늘 제주 방언 에이 아이 데이터,언니 만나서 반가워 아 오늘 제주 방언 에이 아이 데이터,50대,DZES20000001.json,10
1,어 그거 대화 주제는 오늘 대화 주제는,어 그거 대화 주제는 오늘 대화 주제는,50대,DZES20000001.json,7
2,어 명절 설 명절 추석 명절 요로케 나누어서 해볼 거예,어 명절 설 명절 추석 명절 요로케 나누어서 해볼 거예요,50대,DZES20000001.json,10
3,자 그믄 이제부터 얘기해 보게예,자 그러면 이제부터 얘기해 봐요,50대,DZES20000001.json,5
7,어 그믄 언니네 설 명절 때 음식 어떵 해?,어 그러면 언니네 설 명절 때 음식 어떻게 해?,50대,DZES20000001.json,9
...,...,...,...,...,...
333782,일요일 오후 두 시 비행기,일요일 오후 두 시 비행기,30대,DZJD21002409.json,5
333784,가다가 뭐 하면 공항으로 보러 가게.,가다가 뭐 하면 공항으로 보러 가자.,30대,DZJD21002409.json,6
333790,막걸리나 한 잔 바 땡기던가,막걸리나 한 잔 바 땡기던가,30대,DZJD21002409.json,5
333797,마무리 해그넹 저녁에 잠깐 나가크라.,마무리 해서 저녁에 잠깐 나갈게.,30대,DZJD21002409.json,5


In [11]:
df = df.dropna(how='any')
with open('../datas/output/jeju_dialect_test_text.txt', 'w', encoding='utf8') as f:
    f.write('\n'.join(df['방언'] + " " + df["표준어"]))

In [12]:
spm.SentencePieceTrainer.train(input='../datas/output/jeju_dialect_test_text.txt', model_prefix='jeju_sp', vocab_size=30000)

sp_tokenizer = spm.SentencePieceProcessor(model_file='jeju_sp.model')

code_result_dialect = sp_tokenizer.encode(df["방언"].to_list(), out_type=str)
code_result_standard = sp_tokenizer.encode(df["표준어"].to_list(), out_type=str)

In [13]:
df.loc[df.index, "방언"] = [' '.join(res) for res in code_result_dialect]
df["방언"]

0          ▁언니 ▁만낭 ▁반 가 워 ▁아 ▁오늘 ▁제주 ▁방언 ▁에이 ▁아이 ▁데이터
1                        ▁어 ▁그거 ▁대화 ▁주제는 ▁오늘 ▁대화 ▁주제는
2         ▁어 ▁명절 ▁설 ▁명절 ▁추석 ▁명절 ▁요 로케 ▁나누어 서 ▁해볼 ▁거 예
3                             ▁자 ▁그믄 ▁이제부터 ▁얘기해 ▁보게 예
7                  ▁어 ▁그믄 ▁언니네 ▁설 ▁명절 ▁때 ▁음식 ▁어떵 ▁해 ?
                             ...                     
333782                            ▁일요일 ▁오후 ▁두 ▁시 ▁비행기
333784                   ▁가다가 ▁뭐 ▁하면 ▁공항 으로 ▁보러 ▁가게 .
333790                         ▁막걸리 나 ▁한 ▁잔 ▁바 ▁땡기 던가
333797                    ▁마무리 ▁해그넹 ▁저녁에 ▁잠깐 ▁나가 크라 .
333800                         ▁난 ▁차 ▁어시난 ▁우리 ▁집 ▁못 가
Name: 방언, Length: 160122, dtype: object

In [14]:
df.loc[df.index, "표준어"] = [' '.join(res) for res in code_result_standard]
df["표준어"]

0         ▁언니 ▁만나서 ▁반 가 워 ▁아 ▁오늘 ▁제주 ▁방언 ▁에이 ▁아이 ▁데이터
1                        ▁어 ▁그거 ▁대화 ▁주제는 ▁오늘 ▁대화 ▁주제는
2         ▁어 ▁명절 ▁설 ▁명절 ▁추석 ▁명절 ▁요 로케 ▁나누어 서 ▁해볼 ▁거예요
3                              ▁자 ▁그러면 ▁이제부터 ▁얘기해 ▁봐요
7                ▁어 ▁그러면 ▁언니네 ▁설 ▁명절 ▁때 ▁음식 ▁어떻게 ▁해 ?
                             ...                     
333782                            ▁일요일 ▁오후 ▁두 ▁시 ▁비행기
333784                   ▁가다가 ▁뭐 ▁하면 ▁공항 으로 ▁보러 ▁가자 .
333790                         ▁막걸리 나 ▁한 ▁잔 ▁바 ▁땡기 던가
333797                      ▁마무리 ▁해서 ▁저녁에 ▁잠깐 ▁나갈 게 .
333800                        ▁난 ▁차 ▁없으니까 ▁우리 ▁집 ▁못 가
Name: 표준어, Length: 160122, dtype: object

In [15]:
df.to_csv("../datas/output/jeju_dialect_test_SentencePiece.csv")

### 자모 토큰화

In [16]:
df = pd.read_csv("../datas/output/jeju_dialect_data_age_sampled.csv", index_col=0)
df

,방언,표준어,연령대,출처 파일,길이
0,난 진짜 벌초 할때 뱀나오잖아 뱀도,난 진짜 벌초 할때 뱀나오잖아 뱀도,50대,DZES20000002.json,6
9,아 예초하다가이 뱀 나온거야 뱀이 하나 도 아니고,아 예초하다가 뱀 나온거야 뱀이 하나 도 아니고,50대,DZES20000002.json,8
11,아 겅하니까이 예초 할땐 조심해야 돼,아 그러니까 예초 할땐 조심해야 돼,50대,DZES20000002.json,6
14,저기 돌멩이 같은 거 있잖아,저기 돌멩이 같은 거 있잖아,50대,DZES20000002.json,5
17,나 아는 사람이 벌 이거이 그거 벌초하다가 예초기가,나 아는 사람이 벌 이거 그거 벌초하다가 예초기가,50대,DZES20000002.json,8
...,...,...,...,...,...
2774248,어 루미큐브 재밌지 큐브가 어 것도 조커 두 개만 있는 줄 알았는데 여러 가지 있더...,어 루미큐브 재밌지 큐브가 어 그것도 조커 두 개만 있는 줄 알았는데 여러 가지 있...,30대,DZJD21002407.json,16
2774250,색깔 반전 시키는 것도 있댄 하고이,색깔 반전 시키는 것도 있다고 하고,30대,DZJD21002407.json,6
2774251,그리고 요즘 뭐냐 루미큐브도 루미큐브 트래블이라고 해서 여행용으로 다니고 다닐 수도...,그리고 요즘 뭐냐 루미큐브도 루미큐브 트래블이라고 해서 여행용으로 다니고 다닐 수도...,30대,DZJD21002407.json,16
2774252,name이 name이가 그거 아니 쬐그만한 거 가지고 있던데,name이 name이가 그거 아니야 작은 거 가지고 있던데,30대,DZJD21002407.json,8


In [17]:
from jamo import h2j, j2hcj # 자모화

def jamo_tokenization(sentence):
    token_list = []

    for c in sentence:
        if c == ' ':
            token_list.append("<SPC>")
        else:
            jamo_str = j2hcj(h2j(c))
            token_list.extend(jamo_str)
            token_list.append("<SEP>")
    
    return token_list

In [18]:
dialect_list = df["방언"].to_list()
dialect_list = list(map(jamo_tokenization, dialect_list))

standard_list = df["표준어"].to_list()
standard_list = list(map(jamo_tokenization, standard_list))

In [19]:
list_to_spaced_sentence = lambda li : " ".join(li)
df.loc[df.index, "방언"] = list(map(list_to_spaced_sentence, dialect_list))
df.loc[df.index, "표준어"] = list(map(list_to_spaced_sentence, standard_list))

df

,방언,표준어,연령대,출처 파일,길이
0,ㄴ ㅏ ㄴ <SEP> <SPC> ㅈ ㅣ ㄴ <SEP> ㅉ ㅏ <SEP> <SPC> ...,ㄴ ㅏ ㄴ <SEP> <SPC> ㅈ ㅣ ㄴ <SEP> ㅉ ㅏ <SEP> <SPC> ...,50대,DZES20000002.json,6
9,ㅇ ㅏ <SEP> <SPC> ㅇ ㅖ <SEP> ㅊ ㅗ <SEP> ㅎ ㅏ <SEP> ...,ㅇ ㅏ <SEP> <SPC> ㅇ ㅖ <SEP> ㅊ ㅗ <SEP> ㅎ ㅏ <SEP> ...,50대,DZES20000002.json,8
11,ㅇ ㅏ <SEP> <SPC> ㄱ ㅓ ㅇ <SEP> ㅎ ㅏ <SEP> ㄴ ㅣ <SEP...,ㅇ ㅏ <SEP> <SPC> ㄱ ㅡ <SEP> ㄹ ㅓ <SEP> ㄴ ㅣ <SEP> ...,50대,DZES20000002.json,6
14,ㅈ ㅓ <SEP> ㄱ ㅣ <SEP> <SPC> ㄷ ㅗ ㄹ <SEP> ㅁ ㅔ ㅇ <S...,ㅈ ㅓ <SEP> ㄱ ㅣ <SEP> <SPC> ㄷ ㅗ ㄹ <SEP> ㅁ ㅔ ㅇ <S...,50대,DZES20000002.json,5
17,ㄴ ㅏ <SEP> <SPC> ㅇ ㅏ <SEP> ㄴ ㅡ ㄴ <SEP> <SPC> ㅅ ...,ㄴ ㅏ <SEP> <SPC> ㅇ ㅏ <SEP> ㄴ ㅡ ㄴ <SEP> <SPC> ㅅ ...,50대,DZES20000002.json,8
...,...,...,...,...,...
2774248,ㅇ ㅓ <SEP> <SPC> ㄹ ㅜ <SEP> ㅁ ㅣ <SEP> ㅋ ㅠ <SEP> ...,ㅇ ㅓ <SEP> <SPC> ㄹ ㅜ <SEP> ㅁ ㅣ <SEP> ㅋ ㅠ <SEP> ...,30대,DZJD21002407.json,16
2774250,ㅅ ㅐ ㄱ <SEP> ㄲ ㅏ ㄹ <SEP> <SPC> ㅂ ㅏ ㄴ <SEP> ㅈ ㅓ ...,ㅅ ㅐ ㄱ <SEP> ㄲ ㅏ ㄹ <SEP> <SPC> ㅂ ㅏ ㄴ <SEP> ㅈ ㅓ ...,30대,DZJD21002407.json,6
2774251,ㄱ ㅡ <SEP> ㄹ ㅣ <SEP> ㄱ ㅗ <SEP> <SPC> ㅇ ㅛ <SEP> ...,ㄱ ㅡ <SEP> ㄹ ㅣ <SEP> ㄱ ㅗ <SEP> <SPC> ㅇ ㅛ <SEP> ...,30대,DZJD21002407.json,16
2774252,n <SEP> a <SEP> m <SEP> e <SEP> ㅇ ㅣ <SEP> <SPC...,n <SEP> a <SEP> m <SEP> e <SEP> ㅇ ㅣ <SEP> <SPC...,30대,DZJD21002407.json,8


In [20]:
df.to_csv("../datas/output/jeju_dialect_data_jamo.csv")

### 테스트셋 자모

In [21]:
df = pd.read_csv("../datas/output/jeju_dialect_test_age_processed.csv", index_col=0)
df

,방언,표준어,연령대,출처 파일,길이
0,언니 만낭 반가워 아 오늘 제주 방언 에이 아이 데이터,언니 만나서 반가워 아 오늘 제주 방언 에이 아이 데이터,50대,DZES20000001.json,10
1,어 그거 대화 주제는 오늘 대화 주제는,어 그거 대화 주제는 오늘 대화 주제는,50대,DZES20000001.json,7
2,어 명절 설 명절 추석 명절 요로케 나누어서 해볼 거예,어 명절 설 명절 추석 명절 요로케 나누어서 해볼 거예요,50대,DZES20000001.json,10
3,자 그믄 이제부터 얘기해 보게예,자 그러면 이제부터 얘기해 봐요,50대,DZES20000001.json,5
7,어 그믄 언니네 설 명절 때 음식 어떵 해?,어 그러면 언니네 설 명절 때 음식 어떻게 해?,50대,DZES20000001.json,9
...,...,...,...,...,...
333782,일요일 오후 두 시 비행기,일요일 오후 두 시 비행기,30대,DZJD21002409.json,5
333784,가다가 뭐 하면 공항으로 보러 가게.,가다가 뭐 하면 공항으로 보러 가자.,30대,DZJD21002409.json,6
333790,막걸리나 한 잔 바 땡기던가,막걸리나 한 잔 바 땡기던가,30대,DZJD21002409.json,5
333797,마무리 해그넹 저녁에 잠깐 나가크라.,마무리 해서 저녁에 잠깐 나갈게.,30대,DZJD21002409.json,5


In [22]:
dialect_list = df["방언"].to_list()
dialect_list = list(map(jamo_tokenization, dialect_list))

standard_list = df["표준어"].to_list()
standard_list = list(map(jamo_tokenization, standard_list))

In [23]:
list_to_spaced_sentence = lambda li : " ".join(li)
df.loc[df.index, "방언"] = list(map(list_to_spaced_sentence, dialect_list))
df.loc[df.index, "표준어"] = list(map(list_to_spaced_sentence, standard_list))

df

,방언,표준어,연령대,출처 파일,길이
0,ㅇ ㅓ ㄴ <SEP> ㄴ ㅣ <SEP> <SPC> ㅁ ㅏ ㄴ <SEP> ㄴ ㅏ ㅇ ...,ㅇ ㅓ ㄴ <SEP> ㄴ ㅣ <SEP> <SPC> ㅁ ㅏ ㄴ <SEP> ㄴ ㅏ <S...,50대,DZES20000001.json,10
1,ㅇ ㅓ <SEP> <SPC> ㄱ ㅡ <SEP> ㄱ ㅓ <SEP> <SPC> ㄷ ㅐ ...,ㅇ ㅓ <SEP> <SPC> ㄱ ㅡ <SEP> ㄱ ㅓ <SEP> <SPC> ㄷ ㅐ ...,50대,DZES20000001.json,7
2,ㅇ ㅓ <SEP> <SPC> ㅁ ㅕ ㅇ <SEP> ㅈ ㅓ ㄹ <SEP> <SPC> ...,ㅇ ㅓ <SEP> <SPC> ㅁ ㅕ ㅇ <SEP> ㅈ ㅓ ㄹ <SEP> <SPC> ...,50대,DZES20000001.json,10
3,ㅈ ㅏ <SEP> <SPC> ㄱ ㅡ <SEP> ㅁ ㅡ ㄴ <SEP> <SPC> ㅇ ...,ㅈ ㅏ <SEP> <SPC> ㄱ ㅡ <SEP> ㄹ ㅓ <SEP> ㅁ ㅕ ㄴ <SEP...,50대,DZES20000001.json,5
7,ㅇ ㅓ <SEP> <SPC> ㄱ ㅡ <SEP> ㅁ ㅡ ㄴ <SEP> <SPC> ㅇ ...,ㅇ ㅓ <SEP> <SPC> ㄱ ㅡ <SEP> ㄹ ㅓ <SEP> ㅁ ㅕ ㄴ <SEP...,50대,DZES20000001.json,9
...,...,...,...,...,...
333782,ㅇ ㅣ ㄹ <SEP> ㅇ ㅛ <SEP> ㅇ ㅣ ㄹ <SEP> <SPC> ㅇ ㅗ <S...,ㅇ ㅣ ㄹ <SEP> ㅇ ㅛ <SEP> ㅇ ㅣ ㄹ <SEP> <SPC> ㅇ ㅗ <S...,30대,DZJD21002409.json,5
333784,ㄱ ㅏ <SEP> ㄷ ㅏ <SEP> ㄱ ㅏ <SEP> <SPC> ㅁ ㅝ <SEP> ...,ㄱ ㅏ <SEP> ㄷ ㅏ <SEP> ㄱ ㅏ <SEP> <SPC> ㅁ ㅝ <SEP> ...,30대,DZJD21002409.json,6
333790,ㅁ ㅏ ㄱ <SEP> ㄱ ㅓ ㄹ <SEP> ㄹ ㅣ <SEP> ㄴ ㅏ <SEP> <S...,ㅁ ㅏ ㄱ <SEP> ㄱ ㅓ ㄹ <SEP> ㄹ ㅣ <SEP> ㄴ ㅏ <SEP> <S...,30대,DZJD21002409.json,5
333797,ㅁ ㅏ <SEP> ㅁ ㅜ <SEP> ㄹ ㅣ <SEP> <SPC> ㅎ ㅐ <SEP> ...,ㅁ ㅏ <SEP> ㅁ ㅜ <SEP> ㄹ ㅣ <SEP> <SPC> ㅎ ㅐ <SEP> ...,30대,DZJD21002409.json,5


In [24]:
df.to_csv("../datas/output/jeju_dialect_test_jamo.csv")

### Byte 수준 BPE

In [2]:
df = pd.read_csv("../datas/output/jeju_dialect_data_age_sampled.csv", index_col=0)
df

,방언,표준어,연령대,출처 파일,길이
0,난 진짜 벌초 할때 뱀나오잖아 뱀도,난 진짜 벌초 할때 뱀나오잖아 뱀도,50대,DZES20000002.json,6
9,아 예초하다가이 뱀 나온거야 뱀이 하나 도 아니고,아 예초하다가 뱀 나온거야 뱀이 하나 도 아니고,50대,DZES20000002.json,8
11,아 겅하니까이 예초 할땐 조심해야 돼,아 그러니까 예초 할땐 조심해야 돼,50대,DZES20000002.json,6
14,저기 돌멩이 같은 거 있잖아,저기 돌멩이 같은 거 있잖아,50대,DZES20000002.json,5
17,나 아는 사람이 벌 이거이 그거 벌초하다가 예초기가,나 아는 사람이 벌 이거 그거 벌초하다가 예초기가,50대,DZES20000002.json,8
...,...,...,...,...,...
2774248,어 루미큐브 재밌지 큐브가 어 것도 조커 두 개만 있는 줄 알았는데 여러 가지 있더...,어 루미큐브 재밌지 큐브가 어 그것도 조커 두 개만 있는 줄 알았는데 여러 가지 있...,30대,DZJD21002407.json,16
2774250,색깔 반전 시키는 것도 있댄 하고이,색깔 반전 시키는 것도 있다고 하고,30대,DZJD21002407.json,6
2774251,그리고 요즘 뭐냐 루미큐브도 루미큐브 트래블이라고 해서 여행용으로 다니고 다닐 수도...,그리고 요즘 뭐냐 루미큐브도 루미큐브 트래블이라고 해서 여행용으로 다니고 다닐 수도...,30대,DZJD21002407.json,16
2774252,name이 name이가 그거 아니 쬐그만한 거 가지고 있던데,name이 name이가 그거 아니야 작은 거 가지고 있던데,30대,DZJD21002407.json,8


In [3]:
from tokenizers import ByteLevelBPETokenizer

bpe_tokenizer = ByteLevelBPETokenizer()
bpe_tokenizer.train(
    files="../datas/output/jeju_dialect_data_text.txt",
)

In [4]:
code_result_dialect = bpe_tokenizer.encode_batch(df["방언"])
code_result_standard = bpe_tokenizer.encode_batch(df["표준어"])

In [5]:
df.loc[df.index, "방언"] = [' '.join(res.tokens) for res in code_result_dialect]
df["방언"]

0          ëĤľ Ġì§Ħì§ľ Ġë²Įì´Ī ĠíķłëķĮ Ġë±Ģ ëĤĺìĺ¤ ìŀĸìķĦ...
9          ìķĦ ĠìĺĪ ì´Ī íķĺëĭ¤ê°Ģ ìĿ´ Ġë±Ģ ĠëĤĺìĺ¨ê±°ìķ¼ ...
11         ìķĦ Ġê²ħíķĺëĭĪê¹Į ìĿ´ ĠìĺĪ ì´Ī ĠíķłëķĲ Ġì¡°ìĭ¬...
14                 ìłĢê¸° ĠëıĮë©©ìĿ´ Ġê°ĻìĿĢ Ġê±° ĠìŀĪìŀĸìķĦ
17         ëĤĺ ĠìķĦëĬĶ ĠìĤ¬ëŀĮìĿ´ Ġë²Į ĠìĿ´ê±°ìĿ´ Ġê·¸ê±°...
                                 ...                        
2774248    ìĸ´ Ġë£¨ ë¯¸ íģĲ ë¸Į Ġìŀ¬ë°Įì§Ģ ĠíģĲ ë¸Į ê°Ģ Ġ...
2774250    ìĥīê¹Ķ Ġë°ĺìłĦ ĠìĭľíĤ¤ëĬĶ Ġê²ĥëıĦ ĠìŀĪëĮĦ Ġíķĺ...
2774251    ê·¸ë¦¬ê³ł ĠìļĶì¦ĺ ĠëŃĲëĥĲ Ġë£¨ ë¯¸ íģĲ ë¸Į ëıĦ...
2774252    nameìĿ´ ĠnameìĿ´ê°Ģ Ġê·¸ê±° ĠìķĦëĭĪ Ġì¬Ĳ ê·¸ ë...
2774253             ìĸ´ Ġë§ŀìķĦ ĠëĤĺ Ġì¢Ģ Ġì¬Ĳ ëģĦë§Įíķľ ëĶĶ
Name: 방언, Length: 1375435, dtype: object

In [6]:
df.loc[df.index, "표준어"] = [' '.join(res.tokens) for res in code_result_standard]
df["표준어"]

0          ëĤľ Ġì§Ħì§ľ Ġë²Įì´Ī ĠíķłëķĮ Ġë±Ģ ëĤĺìĺ¤ ìŀĸìķĦ...
9          ìķĦ ĠìĺĪ ì´Ī íķĺëĭ¤ê°Ģ Ġë±Ģ ĠëĤĺìĺ¨ê±°ìķ¼ Ġë±Ģ...
11         ìķĦ Ġê·¸ëŁ¬ëĭĪê¹Į ĠìĺĪ ì´Ī ĠíķłëķĲ Ġì¡°ìĭ¬íķ´ì...
14                 ìłĢê¸° ĠëıĮë©©ìĿ´ Ġê°ĻìĿĢ Ġê±° ĠìŀĪìŀĸìķĦ
17         ëĤĺ ĠìķĦëĬĶ ĠìĤ¬ëŀĮìĿ´ Ġë²Į ĠìĿ´ê±° Ġê·¸ê±° Ġë...
                                 ...                        
2774248    ìĸ´ Ġë£¨ ë¯¸ íģĲ ë¸Į Ġìŀ¬ë°Įì§Ģ ĠíģĲ ë¸Į ê°Ģ Ġ...
2774250    ìĥīê¹Ķ Ġë°ĺìłĦ ĠìĭľíĤ¤ëĬĶ Ġê²ĥëıĦ ĠìŀĪëĭ¤ê³ł Ġ...
2774251    ê·¸ë¦¬ê³ł ĠìļĶì¦ĺ ĠëŃĲëĥĲ Ġë£¨ ë¯¸ íģĲ ë¸Į ëıĦ...
2774252    nameìĿ´ ĠnameìĿ´ê°Ģ Ġê·¸ê±° ĠìķĦëĭĪìķ¼ ĠìŀĳìĿĢ...
2774253                     ìĸ´ Ġë§ŀìķĦ ĠëĤĺ Ġì¢Ģ ĠìŀĳìĿĢëį°
Name: 표준어, Length: 1375435, dtype: object

In [7]:
df.to_csv("../datas/output/jeju_dialect_data_bpe.csv")

### 테스트셋 BPE

In [8]:
df = pd.read_csv("../datas/output/jeju_dialect_test_age_processed.csv", index_col=0)
df

,방언,표준어,연령대,출처 파일,길이
0,언니 만낭 반가워 아 오늘 제주 방언 에이 아이 데이터,언니 만나서 반가워 아 오늘 제주 방언 에이 아이 데이터,50대,DZES20000001.json,10
1,어 그거 대화 주제는 오늘 대화 주제는,어 그거 대화 주제는 오늘 대화 주제는,50대,DZES20000001.json,7
2,어 명절 설 명절 추석 명절 요로케 나누어서 해볼 거예,어 명절 설 명절 추석 명절 요로케 나누어서 해볼 거예요,50대,DZES20000001.json,10
3,자 그믄 이제부터 얘기해 보게예,자 그러면 이제부터 얘기해 봐요,50대,DZES20000001.json,5
7,어 그믄 언니네 설 명절 때 음식 어떵 해?,어 그러면 언니네 설 명절 때 음식 어떻게 해?,50대,DZES20000001.json,9
...,...,...,...,...,...
333782,일요일 오후 두 시 비행기,일요일 오후 두 시 비행기,30대,DZJD21002409.json,5
333784,가다가 뭐 하면 공항으로 보러 가게.,가다가 뭐 하면 공항으로 보러 가자.,30대,DZJD21002409.json,6
333790,막걸리나 한 잔 바 땡기던가,막걸리나 한 잔 바 땡기던가,30대,DZJD21002409.json,5
333797,마무리 해그넹 저녁에 잠깐 나가크라.,마무리 해서 저녁에 잠깐 나갈게.,30대,DZJD21002409.json,5


In [9]:
from tokenizers import ByteLevelBPETokenizer

bpe_tokenizer = ByteLevelBPETokenizer()
bpe_tokenizer.train(
    files="../datas/output/jeju_dialect_test_text.txt",
)

bpe_tokenizer.save_model("./")

['./vocab.json', './merges.txt']

In [10]:
code_result_dialect = bpe_tokenizer.encode_batch(df["방언"])
code_result_standard = bpe_tokenizer.encode_batch(df["표준어"])

In [11]:
df.loc[df.index, "방언"] = [' '.join(res.tokens) for res in code_result_dialect]
df["방언"]

0         ìĸ¸ëĭĪ Ġë§ĮëĤŃ Ġë°ĺê°Ģ ìĽĮ ĠìķĦ Ġìĺ¤ëĬĺ Ġìłľì£...
1         ìĸ´ Ġê·¸ê±° ĠëĮĢíĻĶ Ġì£¼ìłľëĬĶ Ġìĺ¤ëĬĺ ĠëĮĢíĻĶ...
2         ìĸ´ ĠëªħìłĪ ĠìĦ¤ ĠëªħìłĪ Ġì¶ĶìĦĿ ĠëªħìłĪ ĠìļĶë...
3          ìŀĲ Ġê·¸ë¯Ħ ĠìĿ´ìłľë¶ĢíĦ° Ġìĸĺê¸°íķ´ Ġë³´ê²Į ìĺĪ
7         ìĸ´ Ġê·¸ë¯Ħ Ġìĸ¸ëĭĪëĦ¤ ĠìĦ¤ ĠëªħìłĪ ĠëķĮ ĠìĿĮì...
                                ...                        
333782              ìĿ¼ ìļĶìĿ¼ Ġìĺ¤íĽĦ ĠëĳĲ Ġìĭľ Ġë¹Ħíĸīê¸°
333784    ê°Ģ ëĭ¤ê°Ģ ĠëŃĲ Ġíķĺë©´ Ġê³µíķŃ ìľ¼ë¡ľ Ġë³´ëŁ¬...
333790          ë§īê±¸ë¦¬ ëĤĺ Ġíķľ ĠìŀĶ Ġë°Ķ Ġëķ¡ê¸° ëįĺê°Ģ
333797    ë§Ī ë¬´ë¦¬ Ġíķ´ê·¸ëĦ¹ ĠìłĢëħģìĹĲ Ġìŀłê¹Ĳ ĠëĤĺê...
333800             ëĤľ Ġì°¨ Ġìĸ´ìĭľëĤľ Ġìļ°ë¦¬ Ġì§ĳ Ġëª»ê°Ģ
Name: 방언, Length: 160123, dtype: object

In [12]:
df.loc[df.index, "표준어"] = [' '.join(res.tokens) for res in code_result_standard]
df["표준어"]

0         ìĸ¸ëĭĪ Ġë§ĮëĤĺìĦľ Ġë°ĺê°Ģ ìĽĮ ĠìķĦ Ġìĺ¤ëĬĺ Ġìł...
1         ìĸ´ Ġê·¸ê±° ĠëĮĢíĻĶ Ġì£¼ìłľëĬĶ Ġìĺ¤ëĬĺ ĠëĮĢíĻĶ...
2         ìĸ´ ĠëªħìłĪ ĠìĦ¤ ĠëªħìłĪ Ġì¶ĶìĦĿ ĠëªħìłĪ ĠìļĶë...
3           ìŀĲ Ġê·¸ëŁ¬ë©´ ĠìĿ´ìłľë¶ĢíĦ° Ġìĸĺê¸°íķ´ Ġë´ĲìļĶ
7         ìĸ´ Ġê·¸ëŁ¬ë©´ Ġìĸ¸ëĭĪëĦ¤ ĠìĦ¤ ĠëªħìłĪ ĠëķĮ Ġì...
                                ...                        
333782              ìĿ¼ ìļĶìĿ¼ Ġìĺ¤íĽĦ ĠëĳĲ Ġìĭľ Ġë¹Ħíĸīê¸°
333784    ê°Ģ ëĭ¤ê°Ģ ĠëŃĲ Ġíķĺë©´ Ġê³µíķŃ ìľ¼ë¡ľ Ġë³´ëŁ¬...
333790          ë§īê±¸ë¦¬ ëĤĺ Ġíķľ ĠìŀĶ Ġë°Ķ Ġëķ¡ê¸° ëįĺê°Ģ
333797    ë§Ī ë¬´ë¦¬ Ġíķ´ìĦľ ĠìłĢëħģìĹĲ Ġìŀłê¹Ĳ ĠëĤĺê°Ī ...
333800          ëĤľ Ġì°¨ ĠìĹĨìľ¼ëĭĪê¹Į Ġìļ°ë¦¬ Ġì§ĳ Ġëª»ê°Ģ
Name: 표준어, Length: 160123, dtype: object

In [13]:
df.to_csv("../datas/output/jeju_dialect_test_bpe.csv")